# Cargar librerias

In [1]:
import pandas as pd
import openpyxl
import re
import os
from openpyxl import load_workbook, Workbook
import numpy as np
from numpy import ceil 
from datetime import date
import math
from datetime import datetime
import openpyxl
from openpyxl import load_workbook 
from shutil import copyfile
import calendar
from datetime import date, timedelta

##   FUNCIONES - CAMBIAR MES  

In [2]:
def calcular_saldo_acumulado(data, mes_actual):
    # Filtrar las filas correspondientes a los meses anteriores y el mes actual
    columnas = [f'Saldo Periodo {str(i).zfill(2)}' for i in range(1, 17)]
    data_filtrada = data[columnas[:mes_actual]]

    # Sumar los saldos de los meses seleccionados
    saldo_acumulado = data_filtrada.sum(axis=1)
    
    # Devolver el saldo acumulado
    return saldo_acumulado

## ESTABLECER LA RUTA DE LOS ARCHIVOS 

In [3]:
# Establecer la ruta de trabajo
ruta1 = 'D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/'

os.chdir(ruta1)

# Configurar opciones de estilo predeterminadas
pd.options.display.float_format = '{:,.2f}'.format

# Configurar opciones de formato de fecha y hora
date_format = "%m/%d/%Y"
datetime_format = "%Y-%m-%d %H:%M:%S"

# Cambiar el color de borde predeterminado
border_colour = "#4F81BD"

#  CÓDIGO PARA CARGAR Y LEER LAS BD EN FORMATO.XLSM

In [4]:
# Ruta del archivo principal
ruta_excel = "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/Fuentes de Información COVI.xlsx"

# Archivo adicional independiente
T_homologacion_ramos = pd.read_excel("T_homologacion_ramos.xlsx")

# Lectura de hojas comunes
VALIDACION_RAMOS = pd.read_excel(ruta_excel, sheet_name="VALIDACION RAMOS").drop_duplicates()
AÑO_ACTUAL = pd.read_excel(ruta_excel, sheet_name="AÑO ACTUAL")
N1 = pd.read_excel(ruta_excel, sheet_name="N-1")
N2 = pd.read_excel(ruta_excel, sheet_name="N-2")
N3 = pd.read_excel(ruta_excel, sheet_name="N-3")
BALANCE = pd.read_excel(ruta_excel, sheet_name="BALANCE")
CUIF = pd.read_excel(ruta_excel, sheet_name="CUIF")
homologacion = pd.read_excel(ruta_excel, sheet_name="HOMOLOGACION")
fechas = pd.read_excel(ruta_excel, sheet_name="FECHAS")
Ramos = pd.read_excel(ruta_excel, sheet_name="Ramos")

# Lectura y limpieza de hoja 'ACTIVOS'
activos_raw = pd.read_excel(ruta_excel, sheet_name="ACTIVOS", header=None)
headers = ['Indice', 'Nombre o Razón Social', 'Código PUC', 'Clase de Inversión', 
           'Valor Mercado ($)', 'Calificación Riesgo Crediticio', 'Ramo', 'SUBCUENTA', 'CATEGORIA']
activos_raw.columns = headers
activos = activos_raw.iloc[4:].reset_index(drop=True).drop('Indice', axis=1)
activos['Valor Mercado ($)'] = pd.to_numeric(activos['Valor Mercado ($)'])

# Lectura de reaseguros
bd_reaseguros = pd.read_excel(ruta_excel, sheet_name="REASEGURO", skiprows=11, usecols="B:N", nrows=688)

# COASEGURO - Aceptados, Cedidos y Deterioro
bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=8, usecols="B:G", nrows=31)

bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=41, usecols="B:G", nrows=31)

bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES = pd.read_excel(
    ruta_excel, sheet_name="COASEGURO", skiprows=74, usecols="B:G", nrows=15)

# Hoja INFO
Info = pd.read_excel(ruta_excel, sheet_name="INFO")
UVR = float(Info.iloc[6, 2])
mes_s = Info.iloc[4, 2]
meses = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 
         'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
mes = meses.index(mes_s) + 1 if mes_s in meses else None
Info['irow'] = range(1, len(Info) + 1)


#   Data.frame de análisis de cuentas

In [5]:
# Crear listas vacías
bd_analisis_cuentas = []
estructura = []

In [6]:
# Cambiar el directorio de trabajo
os.chdir('D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma')

# Cargar el archivo Excel existente
wb = load_workbook("Plantilla_F.3000-76.xlsx")

# Crear un nuevo libro de trabajo
new_wb = Workbook()

#  DIFERENCIADORES 

In [7]:
# Extraer y agregar primeros 4 y 6 dígitos para N2
N2['4d'] = N2['Cuenta alternativa'].astype(str).str[:4].astype(float)
N2['6d'] = N2['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Extraer y agregar primeros 4 y 6 dígitos para N3
N3['4d'] = N3['Cuenta alternativa'].astype(str).str[:4].astype(float)
N3['6d'] = N3['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Extraer y agregar primeros 4 y 6 dígitos para AÑO_ACTUAL
AÑO_ACTUAL['4d'] = AÑO_ACTUAL['Cuenta alternativa'].astype(str).str[:4].astype(float)
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Calcular acumulado para N3
N3['acumulado_mes'] = calcular_saldo_acumulado(N3, mes)

# Calcular saldos mes pasado para AÑO_ACTUAL
AÑO_ACTUAL['Saldo mes pasado'] = AÑO_ACTUAL['Total Cuenta'] - AÑO_ACTUAL.iloc[:, mes+14]

# Calcular saldos mes pasado para N1
N1['Saldo mes pasado'] = N1['Total Cuenta'] - N1.iloc[:, mes+14]

# Calcular saldos mes pasado para N2
N2['Saldo mes pasado'] = N2['Total Cuenta'] - N2.iloc[:, mes+14]

# Calcular saldos mes pasado para N3
N3['Saldo mes pasado'] = N3['Total Cuenta'] - N3.iloc[:, mes+14]

# 1005 

In [8]:
# Inicializar bd_analisis_cuentas como DataFrame vacío
bd_analisis_cuentas = pd.DataFrame()

# Convertir columnas a tipo string
AÑO_ACTUAL["Cuenta alternativa"] = AÑO_ACTUAL["Cuenta alternativa"].astype(str)
AÑO_ACTUAL["4d"] = AÑO_ACTUAL["4d"].astype(str)

# Filtrar filas que contienen "3105"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3105')]

# Crear a1005 usando operaciones de pandas
a1005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Agregar columna notas
a1005['notas'] = ""

# Agregar a1005 a bd_analisis_cuentas
bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1005], ignore_index=True)

# Calcular s1005
s1005 = abs(a1005['Saldos'].sum())

#  1040

In [9]:

# Filtrar filas que contienen "3805"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3805')]

# Crear a1040 usando operaciones de pandas
a1040 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '040'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1040 antes de proceder
if len(a1040) > 0:
    # Agregar columna notas
    a1040['notas'] = ""
    
    # Agregar a1040 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1040], ignore_index=True)

# Calcular s1040
s1040 = abs(a1040['Saldos'].sum())

# 1055

In [10]:
# Filtrar filas que contienen "3205"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3205')]

# Crear a1055 usando operaciones de pandas
a1055 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '055'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1055 antes de proceder
if len(a1055) > 0:
    # Agregar columna notas
    a1055['notas'] = ""
    
    # Agregar a1055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1055], ignore_index=True)

# Calcular s1055
s1055 = abs(a1055['Saldos'].sum())

# 1060

In [11]:
# Filtrar filas que contienen "3215"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3215')]

# Crear a1060 usando operaciones de pandas
a1060 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '060'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1060 antes de proceder
if len(a1060) > 0:
    # Agregar columna notas
    a1060['notas'] = ""
    
    # Agregar a1060 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1060], ignore_index=True)

# Calcular s1060
s1060 = abs(a1060['Saldos'].sum())

# 1070

In [12]:
# Filtrar filas que contienen "3120"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3120')]

# Crear a1070 usando operaciones de pandas
a1070 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '070'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1070 antes de proceder
if len(a1070) > 0:
    # Agregar columna notas
    a1070['notas'] = ""
    
    # Agregar a1070 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1070], ignore_index=True)

# Calcular s1070
s1070 = abs(a1070['Saldos'].sum())

# V1075

In [13]:
# Filtrar filas que contienen "3730"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3730')]

# Crear a1075 usando operaciones de pandas
a1075 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '075'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1075 antes de proceder
if len(a1075) > 0:
    # Agregar columna notas
    a1075['notas'] = ""
    
    # Agregar a1075 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1075], ignore_index=True)

# Calcular s1075
s1075 = abs(a1075['Saldos'].sum())

# V1085

In [14]:
# Filtrar filas que contienen "3705"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3705')]

# Crear a1085 usando operaciones de pandas
a1085 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '085'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1085 antes de proceder
if len(a1085) > 0:
    # Agregar columna notas
    a1085['notas'] = ""
    
    # Agregar a1085 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1085], ignore_index=True)

# Calcular s1085
s1085 = abs(a1085['Saldos'].sum())

# V1090

In [15]:
# Filtrar filas que contienen "3815"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3815')]

# Crear a1090 usando operaciones de pandas
a1090 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '090'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1090 antes de proceder
if len(a1090) > 0:
    # Guardar el signo original de los saldos para referencia si es necesario
    signos_originales = np.sign(a1090['Saldos'])
    
    # Si deseas mantener los signos originales en a1090 pero obtener s1090 positivo
    # no modifiques a1090['Saldos'], sólo usa abs() al calcular s1090
    
    # Agregar columna notas
    a1090['notas'] = ""
    
    # Agregar a1090 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1090], ignore_index=True)

# Calcular s1090 como el valor absoluto de la suma
s1090 = (a1090['Saldos'].sum()) *-1


In [16]:
s1090

np.float64(-5592492141.299999)

# V1095

In [17]:
# Filtrar filas que contienen "865" en la columna Pos_balance
mask = BALANCE['Pos_balance'].notna() & BALANCE['Pos_balance'].astype(str).str.contains('865')
BalanceSubset = BALANCE[mask]

# Obtener el primer valor de Total_periodos_de_informe y multiplicarlo por -1
s1095 = -1 * BalanceSubset['Total_periodos_de_informe'].iloc[0]

# V1100

In [20]:
# Filtrar filas que contienen "3905"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3905')]

# Crear a1100 usando operaciones de pandas
a1100 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 1, 
                'Subcuenta': '0100'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a1100 antes de proceder
if len(a1100) > 0:
    # Agregar columna notas
    a1100['notas'] = ""
    
    # Agregar a1100 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a1100], ignore_index=True)

# Calcular s1100
s1100 = abs(a1100['Saldos'].sum())

# DATAFRAME    

In [21]:
# Crear vector con los valores
v1998 = [s1005, s1040, s1055, s1060, s1070, s1075, s1085, s1090, s1095, s1100]

# Crear DataFrame
unidad1998 = pd.DataFrame(v1998, columns=['Saldos'])

# Redondear valores SIN tomar valor absoluto
unidad1998['Saldos'] = unidad1998['Saldos'].round(0)  # Se eliminó la función .abs()

# Agregar la suma al final del DataFrame
unidad1998.loc[len(unidad1998)] = unidad1998['Saldos'].sum()

# Agregar columna de Subcuenta
unidad1998['Subcuenta'] = ['005', '040', '055', '060', '070', '075', '085', '090', '095', '100', '998']

# Crear subcuentas (excluyendo s1095 que no está en la lista original)
subcuentas = pd.DataFrame({
    'Saldos': [s1005, s1040, s1055, s1060, s1070, s1085, s1090, s1100]
})

# Hacer merge
t = pd.merge(subcuentas, unidad1998)

# 2005

In [22]:
# Filtrar filas que contienen "3910"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3910', na=False)]

# Crear a2005 usando operaciones de pandas
a2005 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '005'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2005 antes de proceder
if len(a2005) > 0:
    # Agregar columna notas
    a2005['notas'] = ""
    
    # Agregar a2005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2005], ignore_index=True)

# Calcular s2005 usando Decimal para mayor precisión
from decimal import Decimal
s2005 = abs(sum(Decimal(str(x)) for x in a2005['Saldos']))

# 2010

In [23]:
# Filtrar filas que contienen "3920"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3920', na=False)]

# Crear a2010 usando operaciones de pandas
a2010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '010'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2010 antes de proceder
if len(a2010) > 0:
    # Agregar columna notas
    a2010['notas'] = ""
    
    # Agregar a2010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2010], ignore_index=True)

# Calcular s2010 usando Decimal para mayor precisión
from decimal import Decimal
s2010 = abs(sum(Decimal(str(x)) for x in a2010['Saldos']))

# 2015

In [24]:
# Filtrar filas que contienen "3930"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('3930', na=False)]

# Crear a2015 usando operaciones de pandas
a2015 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '015'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2015 antes de proceder
if len(a2015) > 0:
    # Agregar columna notas
    a2015['notas'] = ""
    
    # Agregar a2015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2015], ignore_index=True)

# Calcular s2015 usando Decimal para mayor precisión
from decimal import Decimal
s2015 = abs(sum(Decimal(str(x)) for x in a2015['Saldos']))

# 2025

In [25]:
# Buscar el valor correspondiente a "Inversión en compañías de seguros de vida"
s2025 = None
mascara = Info['Unnamed: 1'] == 'Inversión en compañías de seguros generales'
if any(mascara):
    valor = Info.loc[mascara, 'Unnamed: 2'].iloc[0]
    if not pd.isna(valor):
        s2025 = round(float(str(valor).replace(',', '')))

# 2040

In [26]:
# Filtrar filas que contienen "1910" o "2558"
cons1 = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('1910', na=False)]
cons2 = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('2558', na=False)]

# Combinar los resultados
AÑO_ACTUALSubset = pd.concat([cons1, cons2])

# Crear a2040 usando operaciones de pandas
a2040 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '040'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2040 antes de proceder
if len(a2040) > 0:
    # Agregar columna notas
    a2040['notas'] = ""
    
    # Agregar a2040 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2040], ignore_index=True)

# Calcular s2040 usando Decimal para mayor precisión
from decimal import Decimal
s2040 = sum(Decimal(str(x)) for x in a2040['Saldos'])  # Note que aquí no usamos abs()

# 2045

In [27]:
# Extraer el valor cuando irow es 25 (equivalente a ifelse en R)
s2045 = Info.loc[Info['irow'] == 25, 'Unnamed: 2'].values if any(Info['irow'] == 25) else None

# Eliminar valores NA y convertir a numérico (equivalente a na.omit y as.numeric en R)
if s2045 is not None and len(s2045) > 0:
    # Convertir a string, reemplazar comas, convertir a float y redondear
    try:
        s2045 = round(float(str(s2045[0]).replace(',', '')))
    except (ValueError, TypeError):
        s2045 = None
else:
    s2045 = None

# 2055

In [28]:
# Filtrar filas que contienen "1911"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains('1911', na=False)]

# Crear a2055 usando operaciones de pandas
a2055 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
             **{'Unidad de captura': 2, 
                'Subcuenta': '055'}
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta', 
                         'saldos_ant': 'Saldos del mes pasado'})
         .assign(
             **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
                'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
         )
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a2055 antes de proceder
if len(a2055) > 0:
    # Agregar columna notas
    a2055['notas'] = ""
    
    # Agregar a2055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a2055], ignore_index=True)

# Calcular s2055 usando Decimal para mayor precisión
from decimal import Decimal
suma_saldos = sum(Decimal(str(x)) for x in a2055['Saldos'])
s2055 = suma_saldos - (s2045 if s2045 is not None else Decimal('0'))

# Aproximar a cero si el valor absoluto es menor que un umbral (por ejemplo, 0.5)
umbral = Decimal('0.5')
if abs(s2055) < umbral:
    s2055 = Decimal('0')

# 2075

In [29]:
# Convertir la columna a string primero y luego filtrar
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('381555', na=False)]

# Extraer las columnas necesarias
v2075ca = AÑO_ACTUALSubset['Cuenta alternativa']
v2075_valores = AÑO_ACTUALSubset['Total Cuenta']

# Calcular la suma
s2075 = v2075_valores.sum()

# Aplicar condición basada en Info usando 'Unnamed: 1' 
# (segunda columna, equivalente a '...2' en el código R)
if Info.iloc[0]['Unnamed: 1'] == 'COVI':  # Primera fila, columna 'Unnamed: 1'
    s2075 = 0
    v2075 = 0
else:
    # Crear dataframe con las columnas
    v2075 = pd.DataFrame({
        'Cuenta alternativa': v2075ca,
        'saldos': v2075_valores
    })

# Agregar la columna Subcuenta (esto se hace fuera de la condición)
if isinstance(v2075, pd.DataFrame):
    v2075['Subcuenta'] = '075'

# DATAFRAME

In [30]:
from decimal import Decimal

# Convertir todos los valores a Decimal
s2998 = [Decimal(str(x)) if x is not None else Decimal('0') for x in [s2005, s2010, s2015, s2025, s2040, s2045, s2055, s2075]]

# Crear DataFrame
unidad2998 = pd.DataFrame({'Saldos': s2998})

# Redondear valores y convertir a entero
unidad2998['Saldos'] = unidad2998['Saldos'].apply(lambda x: round(x))

# Aplicar valor absoluto
unidad2998['Saldos'] = unidad2998['Saldos'].apply(lambda x: abs(x))

# Calcular la suma específica y agregarla como nueva fila
indices_suma = [0,1,2,4,5,6,7]
suma_especifica = sum(unidad2998['Saldos'].iloc[i] for i in indices_suma)
unidad2998.loc[8] = suma_especifica

# Agregar columna de Subcuenta
unidad2998['Subcuenta'] = ['005', '010', '015', '025', '040', '045', '055', '075', '998']

# Reordenar columnas
unidad2998 = unidad2998[['Saldos', 'Subcuenta']]

# Crear subcuentas
subcuentas = pd.Series([s2005, s2010, s2015, s2040])

# LIMPIEZA DE LOS DATOS

In [31]:
# Eliminar todas las variables mencionadas si existen
for var in ['s1005', 's1040', 's1055', 's1060', 's1070', 's1075', 's1085', 's1090', 
            's1100', 's2005', 's2010', 's2015', 's2040', 's2040_1', 's2040_2', 's2075', 
            'v1998', 'v2005ca', 'v2010ca', 'v2015ca', 'v2025', 'v2040ca', 'v2045', 
            'V2055', 'v2075', 'v2998', 'v2055', 'v2075', 't2']:
    if var in globals():
        del globals()[var]

#  PARTE 4 

In [32]:
# Obtener el valor de la quinta fila (índice 4) de la columna 'Saldos'
s4030 = unidad2998['Saldos'].iloc[4]

# DATAFRAME

In [33]:
# Crear lista inicial con s4030
v4998 = [s4030]

# Calcular la suma
v4998_1 = sum(v4998)

# Agregar la suma a la lista
v4998 = [s4030, v4998_1]

# Crear DataFrame
unidad4998 = pd.DataFrame({
    'Saldos': v4998,
    'Subcuenta': ['030', '999']
})

# Opcional: asegurar el orden de las columnas
unidad4998 = unidad4998[['Saldos', 'Subcuenta']]

 # SALDOS ACUMULADOS AÑO PASADO

In [34]:
# Asumiendo que calcular_saldo_acumulado es una función que ya existe
acumulado_aa = calcular_saldo_acumulado(N1, mes)
N1['acumulado_mes'] = acumulado_aa

# UNIDAD DE CAPTURA 5 

# 5010 DEMAS RAMOS

In [35]:
# Filtrar filas que contienen "412140" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412140')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
AÑO_ACTUALSubset.loc[AÑO_ACTUALSubset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset = AÑO_ACTUALSubset.dropna(subset=['Line of Buss'])

# Crear el dataframe a5010 usando funciones de pandas
a5010 = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5010 antes de proceder
if len(a5010) > 0:
    # Agregar columna notas
    a5010['notas'] = ""
    
    # Agregar a5010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5010], ignore_index=True)

# Calcular s5010 tomando el valor absoluto de la suma
s5010 = abs(a5010['Saldos'].sum())

#  5010 PASADO VENCIDO DEMAS RAMOS

In [36]:
# Extraer primeros 4 y 6 dígitos y crear nuevas columnas - primero convertir a string
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'] == 412140]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset.loc[N1Subset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5010_1 usando funciones de pandas
v5010_1 = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_1 antes de proceder
if len(v5010_1) > 0:
    # Agregar columna notas
    v5010_1['notas'] = ""
    
    # Agregar v5010_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_1], ignore_index=True)

# Calcular s5010_1 tomando el valor absoluto de la suma
s5010_1 = abs(v5010_1['Saldos'].sum())

#  5010 PASADO MES DEMAS RAMOS

In [37]:
# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset.loc[N1Subset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5010_2 usando funciones de pandas
v5010_2 = (
    N1Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_2 antes de proceder
if len(v5010_2) > 0:
    # Agregar columna notas
    v5010_2['notas'] = ""
    
    # Agregar v5010_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_2], ignore_index=True)

# Calcular s5010_2 como la suma de la columna Saldos
s5010_2 = v5010_2['Saldos'].sum()

# 5020 DEMAS RAMOS

In [38]:
# Filtrar filas que contienen "412155" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412155')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
AÑO_ACTUALSubset.loc[AÑO_ACTUALSubset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset = AÑO_ACTUALSubset.dropna(subset=['Line of Buss'])

# Crear el dataframe a5020 usando funciones de pandas
a5020 = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5020 antes de proceder
if len(a5020) > 0:
    # Agregar columna notas
    a5020['notas'] = ""
    
    # Agregar a5020 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5020], ignore_index=True)

# Calcular s5020 como la suma de la columna Saldos
s5020 = a5020['Saldos'].sum()

# 5020 PASADO VENCIDO DEMAS RAMOS 

In [39]:
# Extraer primeros 4 y 6 dígitos y crear nuevas columnas
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset.loc[N1Subset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5020_1 usando funciones de pandas
v5020_1 = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_1 antes de proceder
if len(v5020_1) > 0:
    # Agregar columna notas
    v5020_1['notas'] = ""
    
    # Agregar v5020_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_1], ignore_index=True)

# Calcular s5020_1 como la suma de la columna Saldos
s5020_1 = v5020_1['Saldos'].sum()

# 5020 PASADO MES DEMAS RAMOS 

In [40]:
# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset.loc[N1Subset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5020_2 usando funciones de pandas
v5020_2 = (
    N1Subset[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_2 antes de proceder
if len(v5020_2) > 0:
    # Agregar columna notas
    v5020_2['notas'] = ""
    
    # Agregar v5020_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_2], ignore_index=True)

# Calcular s5020_2 como la suma de la columna Saldos multiplicada por -1
s5020_2 = v5020_2['Saldos'].sum() * -1

# 5005 DEMAS RAMOS

In [41]:
# Filtrar filas que contienen "412130" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412130')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
AÑO_ACTUALSubset.loc[AÑO_ACTUALSubset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset = AÑO_ACTUALSubset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5005 usando funciones de pandas
v5005 = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5005 antes de proceder
if len(v5005) > 0:
    # Agregar columna notas
    v5005['notas'] = ""
    
    # Agregar v5005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5005], ignore_index=True)

# Calcular s5005 como el valor absoluto de la suma de la columna Saldos
s5005 = abs(v5005['Saldos'].sum())

# 5005 PASADO VENCIDO DEMAS RAMOS

In [42]:
# Filtrar filas que contienen "412130" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412130')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset.loc[N1Subset['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Crear el dataframe v5005_1 usando funciones de pandas
v5005_1 = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5005_1 antes de proceder
if len(v5005_1) > 0:
    # Agregar columna notas
    v5005_1['notas'] = ""
    
    # Agregar v5005_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5005_1], ignore_index=True)

# Calcular s5005_1 como el valor absoluto de la suma de la columna Saldos
s5005_1 = abs(v5005_1['Saldos'].sum())

# 5005 PASADO MES DEMAS RAMOS

In [43]:
# Extraer datos del dataframe N1Subset
v5005_2 = N1Subset['acumulado_mes'].values
v5005_2ca = N1Subset['Cuenta alternativa'].values

# Calcular la suma
s5005_2 = sum(v5005_2)

# Crear un nuevo dataframe con las columnas necesarias
v5005_2 = pd.DataFrame({
    'Cuenta alternativa': v5005_2ca,
    'saldos': v5005_2
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v5005_2['Cuenta'] = v5005_2['Cuenta alternativa'].astype(str).str[:6]

# Convertir la columna 'Cuenta' a tipo numérico, igual que CUIF['CUENTA']
v5005_2['Cuenta'] = pd.to_numeric(v5005_2['Cuenta'], errors='coerce')

# Asegurarnos de que CUIF['CUENTA'] también es numérico
CUIF['CUENTA'] = pd.to_numeric(CUIF['CUENTA'], errors='coerce')

# Ahora hacer el merge con tipos de datos compatibles
v5005_2 = v5005_2.merge(
    CUIF[['CUENTA', 'NOMBRE']],
    left_on='Cuenta',
    right_on='CUENTA',
    how='left'
)

# Renombrar la columna NOMBRE a 'Nombre Cuenta'
v5005_2 = v5005_2.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Eliminar la columna CUENTA si no es necesaria
if 'CUENTA' in v5005_2.columns:
    v5005_2 = v5005_2.drop('CUENTA', axis=1)

# 5015 DEMAS RAMOS

In [44]:
# Filtrar filas que contienen "414210" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414210')]

# Filtrar filas que contienen "414410" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414410')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
AÑO_ACTUALSubset_1.loc[AÑO_ACTUALSubset_1['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1.dropna(subset=['Line of Buss'])

# Crear el dataframe a5015 usando funciones de pandas
a5015 = (
    AÑO_ACTUALSubset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5015 antes de proceder
if len(a5015) > 0:
    # Agregar columna notas
    a5015['notas'] = ""
    
    # Agregar a5015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5015], ignore_index=True)

# Calcular s5015 como la suma de la columna Saldos
s5015 = a5015['Saldos'].sum()

# 5015 PASADO VENCIDO DEMAS RAMOS

In [45]:
# Filtrar filas que contienen "414210" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('414210')]

# Filtrar filas que contienen "414410" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('414410')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset_1.loc[N1Subset_1['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset_1 = N1Subset_1.dropna(subset=['Line of Buss'])

# Crear el dataframe v5015_1 usando funciones de pandas
v5015_1 = (
    N1Subset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5015_1 antes de proceder
if len(v5015_1) > 0:
    # Agregar columna notas
    v5015_1['notas'] = ""
    
    # Agregar v5015_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5015_1], ignore_index=True)

# Calcular s5015_1 como la suma de la columna Saldos
s5015_1 = v5015_1['Saldos'].sum()

#  5015 PASADO MES DEMAS RAMOS

In [46]:
# Extraer datos del dataframe N1Subset
v5015_2 = N1Subset['acumulado_mes'].values
v5015_2ca = N1Subset['Cuenta alternativa'].values

# Calcular la suma
s5015_2 = sum(v5015_2)

# Crear un nuevo dataframe con las columnas necesarias
v5015_2 = pd.DataFrame({
    'Cuenta alternativa': v5015_2ca,
    'saldos': v5015_2
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v5015_2['Cuenta'] = v5015_2['Cuenta alternativa'].astype(str).str[:6]

# Convertir a valores numéricos para asegurar compatibilidad de tipos
v5015_2['Cuenta'] = pd.to_numeric(v5015_2['Cuenta'], errors='coerce')

# Buscar coincidencias usando el método map() que es equivalente a match() en R
v5015_2['Nombre Cuenta'] = v5015_2['Cuenta'].map(
    {cuenta: nombre for cuenta, nombre in zip(CUIF['CUENTA'], CUIF['NOMBRE'])}
)


# 5025 DEMAS RAMOS

In [47]:
# Filtrar filas que contienen "515180" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515475')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
AÑO_ACTUALSubset_1.loc[AÑO_ACTUALSubset_1['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1.dropna(subset=['Line of Buss'])

# Combinar ambos dataframes
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1], ignore_index=True)

# Crear el dataframe a5025 usando funciones de pandas
a5025 = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5025 antes de proceder
if len(a5025) > 0:
    # Agregar columna notas
    a5025['notas'] = ""
    
    # Agregar a5025 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5025], ignore_index=True)

# Calcular s5025 como la suma de la columna Saldos
s5025 = a5025['Saldos'].sum()

# 5025 PASADO VENCIDO DEMAS RAMOS 

In [48]:
# Filtrar filas que contienen "515180" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('515475')]

# Crear una lista con los valores Line of Buss a filtrar
lob_to_filter = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                "E58","E59","E91","E94","E90","D16","E15"]

# Establecer a NA las filas donde Line of Buss está en la lista
N1Subset_1.loc[N1Subset_1['Line of Buss'].isin(lob_to_filter)] = np.nan

# Filtrar filas donde Line of Buss no es NA
N1Subset_1 = N1Subset_1.dropna(subset=['Line of Buss'])

# Combinar ambos dataframes
N1Subset = pd.concat([N1Subset, N1Subset_1], ignore_index=True)

# Crear el dataframe v5025_1 usando funciones de pandas
v5025_1 = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5025_1 antes de proceder
if len(v5025_1) > 0:
    # Agregar columna notas
    v5025_1['notas'] = ""
    
    # Agregar v5025_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5025_1], ignore_index=True)

# Calcular s5025_1 como la suma de la columna Saldos
s5025_1 = v5025_1['Saldos'].sum()

# 5025 PASADO MES DEMAS RAMOS

In [49]:
# Extraer datos del dataframe N1Subset
v5025_2 = N1Subset['acumulado_mes'].values
v5025_2ca = N1Subset['Cuenta alternativa'].values

# Calcular la suma
s5025_2 = sum(v5025_2)

# Crear un nuevo dataframe con las columnas necesarias
v5025_2 = pd.DataFrame({
    'Cuenta alternativa': v5025_2ca,
    'saldos': v5025_2
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v5025_2['Cuenta'] = v5025_2['Cuenta alternativa'].astype(str).str[:6]

# Convertir la columna 'Cuenta' a numérico para asegurar la compatibilidad de tipos
v5025_2['Cuenta'] = pd.to_numeric(v5025_2['Cuenta'], errors='coerce')

# Búsqueda de coincidencias usando map (equivalente a match en R)
v5025_2['Nombre Cuenta'] = v5025_2['Cuenta'].map(
    dict(zip(CUIF['CUENTA'], CUIF['NOMBRE']))
)

# SUMATORIAS DEMAS RAMOS UNIDAD 5 

In [50]:
# Calcular las sumas totales
p5005 = s5005 + s5005_1 + s5005_2
p5010 = s5010 + s5010_1 + s5010_2
p5015 = s5015 + s5015_1 + s5015_2
p5020 = s5020 + s5020_1 + s5020_2
p5025 = s5025 + s5025_1 + s5025_2

#  DATAFRAME DEMAS RAMOS UNIDAD 5

In [51]:
# Crear listas de subcuentas y saldos
subcuentas05 = ['005', '010', '015', '020', '025']
saldos05 = [p5005, p5010, p5015, p5020, p5025]

# Crear dataframe
unidad5998 = pd.DataFrame({
    'subcuentas05': subcuentas05,
    'saldos05': saldos05
})

# Redondear valores, aplicar valor absoluto y reemplazar NaN por 0
unidad5998['saldos05'] = np.round(unidad5998['saldos05'], 0)
unidad5998['saldos05'] = np.abs(unidad5998['saldos05'])
unidad5998['saldos05'] = unidad5998['saldos05'].fillna(0)

# Realizar cálculos basados en el dataframe
v5030 = min(UVR * 11000000, ((unidad5998['saldos05'].iloc[0:3].sum() - unidad5998['saldos05'].iloc[3:5].sum()))) * 0.18
v5035 = max((((unidad5998['saldos05'].iloc[0:3].sum() - unidad5998['saldos05'].iloc[3:5].sum())) - UVR * 11000000) * 0.16, 0)
v5998 = round(v5030 + v5035, 0)

# Crear dataframe adicional con los nuevos cálculos
subcuentas05_2 = ['030', '035', '998']
saldos05_2 = [v5030, v5035, v5998]
unidad5998_2 = pd.DataFrame({
    'subcuentas05': subcuentas05_2,
    'saldos05': saldos05_2
})

# Combinar ambos dataframes
unidad5998 = pd.concat([unidad5998, unidad5998_2], ignore_index=True)

# 5010 PREVISIONALES E INVALIDEZ 

In [52]:
# Filtrar filas que contienen "412140" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'E15']

# Crear el dataframe a5010pi usando funciones de pandas
a5010pi = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5010pi antes de proceder
if len(a5010pi) > 0:
    # Agregar columna notas
    a5010pi['notas'] = ""
    
    # Agregar a5010pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5010pi], ignore_index=True)

# Calcular s5010pi como la suma de la columna Saldos
s5010pi = a5010pi['Saldos'].sum()

# 5010 PREVISIONALES E INVALIDEZ 

In [53]:
# Extraer los primeros 4 y 6 dígitos de "Cuenta alternativa" y agregarlos como nuevas columnas
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Crear el dataframe v5010_1pi usando funciones de pandas
v5010_1pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_1pi antes de proceder
if len(v5010_1pi) > 0:
    # Agregar columna notas
    v5010_1pi['notas'] = ""
    
    # Agregar v5010_1pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_1pi], ignore_index=True)

# Calcular s5010_1pi como la suma de la columna Saldos
s5010_1pi = v5010_1pi['Saldos'].sum()

# 5010 PASADO MES PREVISIONALES E INVALIDEZ

In [54]:
# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Crear el dataframe v5010_2pi usando funciones de pandas
v5010_2pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_2pi antes de proceder
if len(v5010_2pi) > 0:
    # Agregar columna notas
    v5010_2pi['notas'] = ""
    
    # Agregar v5010_2pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_2pi], ignore_index=True)

# Calcular s5010_2pi como la suma de la columna Saldos
s5010_2pi = v5010_2pi['Saldos'].sum()

# 5020 PREVISIONALES E INVALIDEZ

In [55]:
# Filtrar filas que contienen "412155" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'E15']

# Crear el dataframe a5020pi usando funciones de pandas
a5020pi = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5020pi antes de proceder
if len(a5020pi) > 0:
    # Agregar columna notas
    a5020pi['notas'] = ""
    
    # Agregar a5020pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5020pi], ignore_index=True)

# Calcular s5020pi como la suma de la columna Saldos
s5020pi = a5020pi['Saldos'].sum()

# 5020 PASADO VENCIDO PREVISIONALES E INVALIDEZ 

In [56]:
# Extraer los primeros 4 y 6 dígitos de "Cuenta alternativa" y agregarlos como nuevas columnas
N1['4d'] = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['6d'] = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)

# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Crear el dataframe v5020_1pi usando funciones de pandas
v5020_1pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_1pi antes de proceder
if len(v5020_1pi) > 0:
    # Agregar columna notas
    v5020_1pi['notas'] = ""
    
    # Agregar v5020_1pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_1pi], ignore_index=True)

# Calcular s5020_1pi como la suma de la columna Saldos
s5020_1pi = v5020_1pi['Saldos'].sum()

#  5020 PASADO MES PREVISIONALES E INVALIDEZ

In [57]:
# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Crear el dataframe v5020_2pi usando funciones de pandas
v5020_2pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_2pi antes de proceder
if len(v5020_2pi) > 0:
    # Agregar columna notas
    v5020_2pi['notas'] = ""
    
    # Agregar v5020_2pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_2pi], ignore_index=True)

# Calcular s5020_2pi como la suma de la columna Saldos
s5020_2pi = v5020_2pi['Saldos'].sum()

# 5005 PREVISIONALES E INVALIDEZ 

In [58]:
# Filtrar filas que contienen "412115" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412115')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'E15']

# Crear el dataframe a5005pi usando funciones de pandas
a5005pi = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5005pi antes de proceder
if len(a5005pi) > 0:
    # Agregar columna notas
    a5005pi['notas'] = ""
    
    # Agregar a5005pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5005pi], ignore_index=True)

# Calcular s5005pi como la suma de la columna Saldos
s5005pi = a5005pi['Saldos'].sum()

# 5005 PASADO VENCIDO PREVISIONALES E INVALIDEZ

In [59]:
# Filtrar filas que contienen "412115" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412115')]

# Filtrar adicionalmente para incluir solo líneas de negocio "E15"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Crear el dataframe v5005_1pi usando funciones de pandas
v5005_1pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5005_1pi antes de proceder
if len(v5005_1pi) > 0:
    # Agregar columna notas
    v5005_1pi['notas'] = ""
    
    # Agregar v5005_1pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5005_1pi], ignore_index=True)

# Calcular s5005_1pi como la suma de la columna Saldos
s5005_1pi = v5005_1pi['Saldos'].sum()

# 5005 PASADO MES PREVISIONALES E INVALIDEZ

In [60]:
# Extraer 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5005_2pi = N1Subset['acumulado_mes'].values
v5005_2capi = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5005_2pi
s5005_2pi = sum(v5005_2pi)

# Crear dataframe con v5005_2capi y v5005_2pi
v5005_2pi = pd.DataFrame({
    'Cuenta alternativa': v5005_2capi,
    'saldos': v5005_2pi
})

# Extraer los primeros 6 dígitos de 'Cuenta alternativa'
v5005_2pi['Cuenta'] = v5005_2pi['Cuenta alternativa'].astype(str).str[:6]

# Buscar coincidencias en CUIF para obtener el nombre de la cuenta
# Equivalente a match() en R
match_indices = pd.Series([CUIF.index[CUIF['CUENTA'] == cuenta][0] 
                           if any(CUIF['CUENTA'] == cuenta) else None 
                           for cuenta in v5005_2pi['Cuenta']])

# Obtener los nombres de cuenta correspondientes
nombre_cuenta = [CUIF.loc[idx, 'NOMBRE'] if idx is not None else None for idx in match_indices]

# Agregar la columna 'Nombre Cuenta' al dataframe
v5005_2pi['Nombre Cuenta'] = nombre_cuenta

# 5015 PREVISIONALES E INVALIDEZ 

In [61]:
# Filtrar filas que contienen "414215" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414215')]

# Filtrar filas que contienen "414415" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414415')]

# Filtrar adicionalmente AÑO_ACTUALSubset_1 para incluir solo líneas de negocio "E15"
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1[AÑO_ACTUALSubset_1['Line of Buss'] == 'E15']

# Combinar los dos dataframes
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1], ignore_index=True)

# Crear el dataframe a5015pi usando funciones de pandas
a5015pi = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5015pi antes de proceder
if len(a5015pi) > 0:
    # Agregar columna notas
    a5015pi['notas'] = ""
    
    # Agregar a5015pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5015pi], ignore_index=True)

# Calcular s5015pi como la suma de la columna Saldos
s5015pi = a5015pi['Saldos'].sum()

# 5015 PASADO VENCIDO PREVISIONALES E INVALIDEZ 

In [62]:
# Filtrar filas que contienen "414215" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('414215')]

# Filtrar filas que contienen "414415" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('414415')]

# Filtrar adicionalmente N1Subset_1 para incluir solo líneas de negocio "E15"
N1Subset_1 = N1Subset_1[N1Subset_1['Line of Buss'] == 'E15']

# Crear el dataframe v5015_1pi usando funciones de pandas (solo usando N1Subset_1)
v5015_1pi = (
    N1Subset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5015_1pi antes de proceder
if len(v5015_1pi) > 0:
    # Agregar columna notas
    v5015_1pi['notas'] = ""
    
    # Agregar v5015_1pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5015_1pi], ignore_index=True)

# Calcular s5015_1pi como la suma de la columna Saldos
s5015_1pi = v5015_1pi['Saldos'].sum()

# 5015 PASADO MES PREVISIONALES E INVALIDEZ

In [63]:
# Extraer 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5015_2pi = N1Subset['acumulado_mes'].values
v5015_2capi = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5015_2pi
s5015_2pi = sum(v5015_2pi)

# Crear dataframe con v5015_2capi y v5015_2pi
v5015_2pi = pd.DataFrame({
    'Cuenta alternativa': v5015_2capi,
    'saldos': v5015_2pi
})

# Extraer los primeros 6 dígitos de 'Cuenta alternativa'
v5015_2pi['Cuenta'] = v5015_2pi['Cuenta alternativa'].astype(str).str[:6]

# Buscar coincidencias en CUIF para obtener el nombre de la cuenta
# Equivalente a match() en R
match_indices = pd.Series([CUIF.index[CUIF['CUENTA'] == cuenta][0] 
                          if any(CUIF['CUENTA'] == cuenta) else None 
                          for cuenta in v5015_2pi['Cuenta']])

# Obtener los nombres de cuenta correspondientes
nombre_cuenta = [CUIF.loc[idx, 'NOMBRE'] if idx is not None else None for idx in match_indices]

# Agregar la columna 'Nombre Cuenta' al dataframe
v5015_2pi['Nombre Cuenta'] = nombre_cuenta

# 5025 PREVISIONALES E INVALIDEZ

In [64]:
# Filtrar filas que contienen "515180" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515475')]

# Filtrar adicionalmente AÑO_ACTUALSubset_1 para incluir solo líneas de negocio "E15"
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1[AÑO_ACTUALSubset_1['Line of Buss'] == 'E15']

# Crear el dataframe a5025pi usando solo AÑO_ACTUALSubset_1
a5025pi = (
    AÑO_ACTUALSubset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5025pi antes de proceder
if len(a5025pi) > 0:
    # Agregar columna notas
    a5025pi['notas'] = ""
    
    # Agregar a5025pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5025pi], ignore_index=True)

# Calcular s5025pi como la suma de la columna Saldos
s5025pi = a5025pi['Saldos'].sum()

# 5025 PASADO VENCIDO PREVISIONALES E INVALIDEZ

In [65]:
# Filtrar filas que contienen "515180" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('515475')]

# Filtrar adicionalmente N1Subset_1 para incluir solo líneas de negocio "E15"
N1Subset_1 = N1Subset_1[N1Subset_1['Line of Buss'] == 'E15']

# Combinar los dos dataframes
N1Subset = pd.concat([N1Subset, N1Subset_1], ignore_index=True)

# Crear el dataframe v5025_1pi usando funciones de pandas
v5025_1pi = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5025_1pi antes de proceder
if len(v5025_1pi) > 0:
    # Agregar columna notas
    v5025_1pi['notas'] = ""
    
    # Agregar v5025_1pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5025_1pi], ignore_index=True)

# Calcular s5025_1pi como la suma de la columna Saldos
s5025_1pi = v5025_1pi['Saldos'].sum()

# 5025 PASADO MES PREVISIONALES E INVALIDEZ

In [66]:
# Extraer 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5025_2pi = N1Subset['acumulado_mes'].values
v5025_2capi = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5025_2pi
s5025_2pi = sum(v5025_2pi)

# Crear dataframe con v5025_2capi y v5025_2pi
v5025_2pi = pd.DataFrame({
    'Cuenta alternativa': v5025_2capi,
    'saldos': v5025_2pi
})

# Extraer los primeros 6 dígitos de 'Cuenta alternativa'
v5025_2pi['Cuenta'] = v5025_2pi['Cuenta alternativa'].astype(str).str[:6]

# Buscar coincidencias en CUIF para obtener el nombre de la cuenta
# Equivalente a match() en R
match_indices = pd.Series([CUIF.index[CUIF['CUENTA'] == cuenta][0] 
                          if any(CUIF['CUENTA'] == cuenta) else None 
                          for cuenta in v5025_2pi['Cuenta']])

# Obtener los nombres de cuenta correspondientes
nombre_cuenta = [CUIF.loc[idx, 'NOMBRE'] if idx is not None else None for idx in match_indices]

# Agregar la columna 'Nombre Cuenta' al dataframe
v5025_2pi['Nombre Cuenta'] = nombre_cuenta

# SUMATORIAS UNIDAD 5 PI

In [67]:
# Calcular la suma de los valores para cada grupo
p5005pi = s5005pi + s5005_1pi + s5005_2pi
p5010pi = s5010pi + s5010_1pi + s5010_2pi
p5015pi = s5015pi + s5015_1pi + s5015_2pi
p5020pi = s5020pi + s5020_1pi + s5020_2pi
p5025pi = s5025pi + s5025_1pi + s5025_2pi

#  DATAFRAME UNIDAD 5 PI

In [68]:
# Crear dataframe con subcuentas y saldos
subcuentas05pi = ['005', '010', '015', '020', '025']
saldos05pi = [p5005pi, p5010pi, p5015pi, p5020pi, p5025pi]
unidad5998pi = pd.DataFrame({
    'subcuentas05pi': subcuentas05pi,
    'saldos05pi': saldos05pi
})

# Redondear los valores a enteros
unidad5998pi['saldos05pi'] = unidad5998pi['saldos05pi'].round(0)

# Tomar valores absolutos
unidad5998pi['saldos05pi'] = unidad5998pi['saldos05pi'].abs()

# Reemplazar valores NA con 0
unidad5998pi['saldos05pi'] = unidad5998pi['saldos05pi'].fillna(0)

# Realizar cálculos basados en el dataframe
# Primero, calculamos la suma de los primeros 3 valores (índices 0, 1, 2) y los últimos 2 (índices 3, 4)
suma_primeros_tres = unidad5998pi['saldos05pi'].iloc[0:3].sum()
suma_ultimos_dos = unidad5998pi['saldos05pi'].iloc[3:5].sum()
diferencia = suma_primeros_tres - suma_ultimos_dos

# Calcular valores según las fórmulas dadas
v5030pi = min(UVR * 41000000, diferencia) * 0.18
v5035pi = max((diferencia - UVR * 41000000) * 0.16, 0)
v5998pi = v5030pi + v5035pi

# Crear dataframe para los nuevos valores
subcuentas05_2pi = ['030', '035', '998']
saldos05_2pi = [v5030pi, v5035pi, v5998pi]
unidad5998_2pi = pd.DataFrame({
    'subcuentas05pi': subcuentas05_2pi,
    'saldos05pi': saldos05_2pi
})

# Combinar los dos dataframes
unidad5998pi = pd.concat([unidad5998pi, unidad5998_2pi], ignore_index=True)

#  5010 RIESGOS PROFESIONALES

In [69]:
# Filtrar filas que contienen "412140" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'D16']

# Crear el dataframe a5010rp usando funciones de pandas
a5010rp = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5010rp antes de proceder
if len(a5010rp) > 0:
    # Agregar columna notas
    a5010rp['notas'] = ""
    
    # Agregar a5010rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5010rp], ignore_index=True)

# Calcular s5010rp como la suma de la columna Saldos
# Nota: Hay un posible error en el código original, ya que utiliza v5025_1pi en lugar de a5010rp
s5010rp = v5025_1pi['Saldos'].sum()  # Posible error en el código original
# Si se pretendía usar a5010rp, debería ser:
# s5010rp = a5010rp['Saldos'].sum()

# 5010 RIESGOS PROFESIONALES

In [70]:
# Extraer los primeros 4 dígitos de "Cuenta alternativa" y convertirlos a numéricos
primeros_4_digitos = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['4d'] = primeros_4_digitos

# Extraer los primeros 6 dígitos de "Cuenta alternativa" y convertirlos a numéricos
primeros_6_digitos = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)
N1['6d'] = primeros_6_digitos

# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'D16']

# Crear el dataframe v5010_1rp usando funciones de pandas
v5010_1rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_1rp antes de proceder
if len(v5010_1rp) > 0:
    # Agregar columna notas
    v5010_1rp['notas'] = ""
    
    # Agregar v5010_1rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_1rp], ignore_index=True)

# Calcular s5010_1rp como la suma de la columna Saldos
s5010_1rp = v5010_1rp['Saldos'].sum()

# 5010 PASADO MES RIESGOS PROFESIONALES

In [71]:
# Filtrar filas que contienen "412140" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412140')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'D16']

# Crear el dataframe v5010_2rp usando funciones de pandas
v5010_2rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '010'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5010_2rp antes de proceder
if len(v5010_2rp) > 0:
    # Agregar columna notas
    v5010_2rp['notas'] = ""
    
    # Agregar v5010_2rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5010_2rp], ignore_index=True)

# Calcular s5010_2rp como la suma de la columna Saldos
s5010_2rp = v5010_2rp['Saldos'].sum()

# 5020 RIESGOS PROFESIONALES 

In [72]:
# Filtrar filas que contienen "412155" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'D16']

# Crear el dataframe a5020rp usando funciones de pandas
a5020rp = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5020rp antes de proceder
if len(a5020rp) > 0:
    # Agregar columna notas
    a5020rp['notas'] = ""
    
    # Agregar a5020rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5020rp], ignore_index=True)

# Calcular s5020rp como la suma de la columna Saldos
s5020rp = a5020rp['Saldos'].sum()

# 5020 PASADO VENCIDO RIESGOS PROFESIONALES

In [73]:
# Extraer los primeros 4 dígitos de "Cuenta alternativa" y convertirlos a numéricos
primeros_4_digitos = N1['Cuenta alternativa'].astype(str).str[:4].astype(float)
N1['4d'] = primeros_4_digitos

# Extraer los primeros 6 dígitos de "Cuenta alternativa" y convertirlos a numéricos
primeros_6_digitos = N1['Cuenta alternativa'].astype(str).str[:6].astype(float)
N1['6d'] = primeros_6_digitos

# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'D16']

# Crear el dataframe v5020_1rp usando funciones de pandas
v5020_1rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_1rp antes de proceder
if len(v5020_1rp) > 0:
    # Agregar columna notas
    v5020_1rp['notas'] = ""
    
    # Agregar v5020_1rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_1rp], ignore_index=True)

# Calcular s5020_1rp como la suma de la columna Saldos
s5020_1rp = v5020_1rp['Saldos'].sum()

# 5020 PASADO MES RIESGOS PROFESIONALES

In [74]:
# Filtrar filas que contienen "412155" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412155')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'D16']

# Crear el dataframe v5020_2rp usando funciones de pandas
v5020_2rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '020'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5020_2rp antes de proceder
if len(v5020_2rp) > 0:
    # Agregar columna notas
    v5020_2rp['notas'] = ""
    
    # Agregar v5020_2rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5020_2rp], ignore_index=True)

# Calcular s5020_2rp como la suma de la columna Saldos
s5020_2rp = v5020_2rp['Saldos'].sum()

# 5005 RIESGOS PROFESIONALES 

In [75]:
# Filtrar filas que contienen "412120" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('412120')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'D16']

# Crear el dataframe a5005rp usando funciones de pandas
a5005rp = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5005rp antes de proceder
if len(a5005rp) > 0:
    # Agregar columna notas
    a5005rp['notas'] = ""
    
    # Agregar a5005rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5005rp], ignore_index=True)

# Calcular s5005rp como la suma de la columna Saldos
s5005rp = a5005rp['Saldos'].sum()

# 5005 PASADO VENCIDO RIESGOS PROFESIONALES

In [76]:
# Filtrar filas que contienen "412120" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('412120')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'D16']

# Crear el dataframe v5005_1rp usando funciones de pandas
v5005_1rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '005'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5005_1rp antes de proceder
if len(v5005_1rp) > 0:
    # Agregar columna notas
    v5005_1rp['notas'] = ""
    
    # Agregar v5005_1rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5005_1rp], ignore_index=True)

# Calcular s5005_1rp como la suma de la columna Saldos
s5005_1rp = v5005_1rp['Saldos'].sum()

# 5005 PASADO MES RIESGOS PROFESIONALES

In [77]:
# Extraer los valores de 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5005_2rp = N1Subset['acumulado_mes'].values
v5005_2carp = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5005_2rp
s5005_2rp = sum(v5005_2rp)

# Crear un dataframe con las columnas 'Cuenta alternativa' y 'saldos'
v5005_2rp = pd.DataFrame({
    'Cuenta alternativa': v5005_2carp,
    'saldos': v5005_2rp
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y crear una nueva columna 'Cuenta'
v5005_2rp['Cuenta'] = v5005_2rp['Cuenta alternativa'].astype(str).str[:6]

# Hacer coincidir los valores de 'Cuenta' con 'CUENTA' del dataframe CUIF
# y extraer los nombres de cuenta correspondientes
v5005_2rp['Nombre Cuenta'] = v5005_2rp['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# 5015 RIESGOS PROFESIONALES

In [78]:
# Filtrar filas que contienen "414220" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414220')]

# Filtrar filas que contienen "414420" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('414420')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16" en el segundo subset
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1[AÑO_ACTUALSubset_1['Line of Buss'] == 'D16']

# Combinar los dos subsets
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1], ignore_index=True)

# Crear el dataframe a5015rp usando funciones de pandas
a5015rp = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5015rp antes de proceder
if len(a5015rp) > 0:
    # Agregar columna notas
    a5015rp['notas'] = ""
    
    # Agregar a5015rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5015rp], ignore_index=True)

# Calcular s5015rp como la suma de la columna Saldos
s5015rp = a5015rp['Saldos'].sum()

# 5015 PASADO VENCIDO RIESGOS PROFESIONALES

In [79]:
# Filtrar filas que contienen "414220" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('414220')]

# Filtrar filas que contienen "414420" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('414420')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16" en el segundo subset
N1Subset_1 = N1Subset_1[N1Subset_1['Line of Buss'] == 'D16']

# Crear el dataframe v5015_1rp usando funciones de pandas
v5015_1rp = (
    N1Subset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '015'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5015_1rp antes de proceder
if len(v5015_1rp) > 0:
    # Agregar columna notas
    v5015_1rp['notas'] = ""
    
    # Agregar v5015_1rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5015_1rp], ignore_index=True)

# Calcular s5015_1rp como la suma de la columna Saldos
s5015_1rp = v5015_1rp['Saldos'].sum()

# 5015 PASADO MES RIESGOS PROFESIONALES 

In [80]:
# Extraer los valores de 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5015_2rp = N1Subset['acumulado_mes'].values
v5015_2carp = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5015_2rp
s5015_2rp = sum(v5015_2rp)

# Crear un dataframe con las columnas 'Cuenta alternativa' y 'saldos'
v5015_2rp = pd.DataFrame({
    'Cuenta alternativa': v5015_2carp,
    'saldos': v5015_2rp
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y crear una nueva columna 'Cuenta'
v5015_2rp['Cuenta'] = v5015_2rp['Cuenta alternativa'].astype(str).str[:6]

# Hacer coincidir los valores de 'Cuenta' con 'CUENTA' del dataframe CUIF
# y extraer los nombres de cuenta correspondientes
v5015_2rp['Nombre Cuenta'] = v5015_2rp['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# 5025 RIESGOS PROFESIONALES 

In [81]:
# Filtrar filas que contienen "515180" en la columna "6d"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515475')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16" en el segundo subset
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1[AÑO_ACTUALSubset_1['Line of Buss'] == 'D16']

# Combinar los dos subsets
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1], ignore_index=True)

# Crear el dataframe a5025rp usando funciones de pandas
a5025rp = (
    AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en a5025rp antes de proceder
if len(a5025rp) > 0:
    # Agregar columna notas
    a5025rp['notas'] = ""
    
    # Agregar a5025rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a5025rp], ignore_index=True)

# Calcular s5025rp como la suma de la columna Saldos
s5025rp = a5025rp['Saldos'].sum()

# 5025 PASADO VENCIDO RIESGOS PROFESIONALES

In [82]:
# Filtrar filas que contienen "515180" en la columna "6d"
N1Subset = N1[N1['6d'].astype(str).str.contains('515180')]

# Filtrar filas que contienen "515475" en la columna "6d"
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('515475')]

# Filtrar adicionalmente para incluir solo líneas de negocio "D16" en el segundo subset
N1Subset_1 = N1Subset_1[N1Subset_1['Line of Buss'] == 'D16']

# Combinar los dos subsets
N1Subset = pd.concat([N1Subset, N1Subset_1], ignore_index=True)

# Crear el dataframe v5025_1rp usando funciones de pandas
v5025_1rp = (
    N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
    .rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})
    .assign(
        Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float),
        **{'Unidad de captura': 5, 'Subcuenta': '025'}
    )
    .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
    .rename(columns={'NOMBRE': 'Nombre Cuenta', 'saldos_ant': 'Saldos del mes pasado'})
    .assign(
        **{'Diferencia mes a mes': lambda x: x['Saldos'] - x['Saldos del mes pasado'],
           'Diferencia %': lambda x: (x['Saldos'] - x['Saldos del mes pasado']) / x['Saldos']}
    )
    [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
    .reset_index(drop=True)
)

# Verificar si hay filas en v5025_1rp antes de proceder
if len(v5025_1rp) > 0:
    # Agregar columna notas
    v5025_1rp['notas'] = ""
    
    # Agregar v5025_1rp a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v5025_1rp], ignore_index=True)

# Calcular s5025_1rp como la suma de la columna Saldos
s5025_1rp = v5025_1rp['Saldos'].sum()

# 5025 PASADO MES RIESGOS PROFESIONALES

In [83]:
# Extraer los valores de 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v5025_2rp = N1Subset['acumulado_mes'].values
v5025_2carp = N1Subset['Cuenta alternativa'].values

# Calcular la suma de v5025_2rp
s5025_2rp = sum(v5025_2rp)

# Crear un dataframe con las columnas 'Cuenta alternativa' y 'saldos'
v5025_2rp = pd.DataFrame({
    'Cuenta alternativa': v5025_2carp,
    'saldos': v5025_2rp
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y crear una nueva columna 'Cuenta'
v5025_2rp['Cuenta'] = v5025_2rp['Cuenta alternativa'].astype(str).str[:6]

# Hacer coincidir los valores de 'Cuenta' con 'CUENTA' del dataframe CUIF
# y extraer los nombres de cuenta correspondientes
v5025_2rp['Nombre Cuenta'] = v5025_2rp['Cuenta'].map(dict(zip(CUIF['CUENTA'], CUIF['NOMBRE'])))

# SUMATORIAS UNIDAD 5 RP 

In [84]:
# Calcular totales sumando valores relacionados
p5005rp = s5005rp + s5005_1rp + s5005_2rp
p5010rp = s5010rp + s5010_1rp + s5010_2rp
p5015rp = s5015rp + s5015_1rp + s5015_2rp
p5020rp = s5020rp + s5020_1rp + s5020_2rp
p5025rp = s5025rp + s5025_1rp + s5025_2rp

# DATAFRAME UNIDAD 5 RP

In [85]:
# Crear arrays con los datos de subcuentas y saldos
subcuentas05rp = ['005', '010', '015', '020', '025']
saldos05rp = [p5005rp, p5010rp, p5015rp, p5020rp, p5025rp]

# Crear dataframe con subcuentas y saldos
unidad5998rp = pd.DataFrame({
    'subcuentas05rp': subcuentas05rp,
    'saldos05rp': saldos05rp
})

# Redondear valores, convertir a valores absolutos y reemplazar NaN por 0
unidad5998rp['saldos05rp'] = unidad5998rp['saldos05rp'].round(0)
unidad5998rp['saldos05rp'] = unidad5998rp['saldos05rp'].abs()
unidad5998rp['saldos05rp'] = unidad5998rp['saldos05rp'].fillna(0)

# Realizar cálculos basados en el dataframe
v5030rp = min(UVR * 41000000, ((unidad5998rp['saldos05rp'].iloc[0:3].sum() - unidad5998rp['saldos05rp'].iloc[3:5].sum()))) * 0.18
v5035rp = max((((unidad5998rp['saldos05rp'].iloc[0:3].sum() - unidad5998rp['saldos05rp'].iloc[3:5].sum())) - UVR * 41000000), 0) * 0.16
v5998rp = v5030rp + v5035rp

# Crear dataframe adicional y agregarlo al dataframe original
subcuentas05_2rp = ['030', '035', '998']
saldos05_2rp = [v5030rp, v5035rp, v5998rp]
unidad5998_2rp = pd.DataFrame({
    'subcuentas05rp': subcuentas05_2rp,
    'saldos05rp': saldos05_2rp
})

# Combinar dataframes
unidad5998rp = pd.concat([unidad5998rp, unidad5998_2rp], ignore_index=True)

# 6020 DEMAS RAMOS


In [86]:
# Filtrar para obtener los registros que contienen "168010"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('168010', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
AÑO_ACTUALSubset.loc[AÑO_ACTUALSubset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
AÑO_ACTUALSubset = AÑO_ACTUALSubset.dropna(subset=['Line of Buss'])

# Crear a6020 usando operaciones de pandas
a6020 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '020',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6020 antes de proceder
if len(a6020) > 0:
    # Agregar columna notas
    a6020['notas'] = ""
    
    # Agregar a6020 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6020], ignore_index=True)

# Calcular s6020 con redondeo a entero
s6020 = int(round(a6020['Saldos'].sum()))

# 6015 DEMAS RAMOS 

In [87]:
# Filtrar para obtener los registros que contienen "265010" o "168010"
AÑO_ACTUALSubset = AÑO_ACTUAL[
    AÑO_ACTUAL['6d'].astype(str).str.contains('^265010') | 
    AÑO_ACTUAL['6d'].astype(str).str.contains('^168010')
]

# Opción 1: Eliminar completamente esta línea
# Opción 2: Guardarla en una variable pero no imprimirla
tabla_frecuencias = AÑO_ACTUALSubset['6d'].value_counts()  # Se guarda pero no se imprime

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
AÑO_ACTUALSubset.loc[AÑO_ACTUALSubset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
AÑO_ACTUALSubset = AÑO_ACTUALSubset.dropna(subset=['Line of Buss'])

# Crear a6015 usando operaciones de pandas
a6015 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '015',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6015 antes de proceder
if len(a6015) > 0:
    # Agregar columna notas
    a6015['notas'] = ""
    
    # Agregar a6015 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6015], ignore_index=True)

# Calcular s6015 con valor absoluto y redondeado a entero
s6015 = int(round(abs(a6015['Saldos'].sum())))

# 6025 PASADO VENCIDO DEMAS RAMOS

In [88]:
# Filtrar para obtener los registros que contienen "265010" o "168010"
N3Subset = N3[
    N3['6d'].astype(str).str.contains('^265010') | 
    N3['6d'].astype(str).str.contains('^168010')
]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset.loc[N3Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Crear v6025_2 usando operaciones de pandas
v6025_2 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
         .rename(columns={'acumulado_mes': 'Saldos', 
                          'Arrastre saldos': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '025',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6025_2 antes de proceder
if len(v6025_2) > 0:
    # Agregar columna notas
    v6025_2['notas'] = ""
    
    # Agregar v6025_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6025_2], ignore_index=True)

# Calcular s6025_2 sumando los saldos actuales y los del mes pasado, aplicando valor absoluto
s6025_2 = int(round(abs(v6025_2['Saldos'].sum() + v6025_2['Saldos del mes pasado'].sum())))

# 6030 PERÍODO ANTERIOR DEMAS RAMOS 

In [89]:
# Filtrar para obtener solo los registros que contienen "168010"
N3Subset = N3[N3['6d'].astype(str).str.contains('^168010')]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset.loc[N3Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Crear v6030_2 usando operaciones de pandas
v6030_2 = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
         .rename(columns={'acumulado_mes': 'Saldos', 
                          'Arrastre saldos': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '030',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6030_2 antes de proceder
if len(v6030_2) > 0:
    # Agregar columna notas
    v6030_2['notas'] = ""
    
    # Agregar v6030_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6030_2], ignore_index=True)

# Calcular s6030_2 sumando los saldos actuales y los del mes pasado
# Usamos valor absoluto para asegurar un resultado positivo
s6030_2 = int(round(abs(v6030_2['Saldos'].sum() + v6030_2['Saldos del mes pasado'].sum())))

# 6005 MES ACTUAL DEMAS RAMOS 

In [90]:
# Filtrar para obtener registros que contienen "512110"
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('512110', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
AÑO_ACTUALSubset_2.loc[AÑO_ACTUALSubset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2.dropna(subset=['Line of Buss'])

# Crear a6005 usando operaciones de pandas
a6005 = (AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '005',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6005 antes de proceder
if len(a6005) > 0:
    # Agregar columna notas
    a6005['notas'] = ""
    
    # Agregar a6005 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6005], ignore_index=True)

# Calcular s6005 como la suma de los saldos y redondear a entero
s6005 = int(round(a6005['Saldos'].sum()))

# 6005 PASADO VENCIDO DEMAS RAMOS

In [91]:
# Filtrar para obtener registros que contienen "512110"
N1Subset_2 = N1[N1['6d'].astype(str).str.contains('512110', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N1Subset_2.loc[N1Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N1Subset_2 = N1Subset_2.dropna(subset=['Line of Buss'])

# Crear v6005_1 usando operaciones de pandas
v6005_1 = (N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '005',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6005_1 antes de proceder
if len(v6005_1) > 0:
    # Agregar columna notas
    v6005_1['notas'] = ""
    
    # Agregar v6005_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_1], ignore_index=True)

# Calcular s6005_1 como la suma de los saldos
s6005_1 = int(round(v6005_1['Saldos'].sum()))

# 6005 #N-2 VENCIDO DEMAS RAMOS

In [92]:
# Filtrar para obtener registros que contienen "512110"
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('512110', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N2Subset_2.loc[N2Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N2Subset_2 = N2Subset_2.dropna(subset=['Line of Buss'])

# Crear v6005_2 usando operaciones de pandas
v6005_2 = (N2Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '005',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6005_2 antes de proceder
if len(v6005_2) > 0:
    # Agregar columna notas
    v6005_2['notas'] = ""
    
    # Agregar v6005_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_2], ignore_index=True)

# Calcular s6005_2 como la suma de los saldos
s6005_2 = int(round(v6005_2['Saldos'].sum()))

# 6005 #N-3 VENCIDO DEMAS RAMOS 

In [93]:
# Filtrar para obtener registros que contienen "512110"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('512110', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset_2.loc[N3Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset_2 = N3Subset_2.dropna(subset=['Line of Buss'])

# Crear v6005_3 usando operaciones de pandas
v6005_3 = (N3Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '005',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6005_3 antes de proceder
if len(v6005_3) > 0:
    # Agregar columna notas
    v6005_3['notas'] = ""
    
    # Agregar v6005_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_3], ignore_index=True)

# Calcular s6005_3 como la suma de los saldos
s6005_3 = int(round(v6005_3['Saldos'].sum()))

# 6005 #N-3 MES CORRIDO DEMAS RAMOS

In [94]:
# Filtrar para obtener registros que contienen "512110"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('512110', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset_2.loc[N3Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset_2 = N3Subset_2.dropna(subset=['Line of Buss'])

# Crear v6005_4 usando operaciones de pandas
v6005_4 = (N3Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'acumulado_mes': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '005',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6005_4 antes de proceder
if len(v6005_4) > 0:
    # Agregar columna notas
    v6005_4['notas'] = ""
    
    # Agregar v6005_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_4], ignore_index=True)

# Calcular s6005_4 como el valor absoluto de la suma de los saldos multiplicado por -1
# Utilizando int() y round() para convertir a entero sin decimales
s6005_4 = int(round(abs(v6005_4['Saldos'].sum()) * -1))

# 6010 DEMAS RAMOS 

In [95]:
# Filtrar para obtener registros que contienen diferentes códigos
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515150', na=False)]
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515155', na=False)]
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515445', na=False)]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515450', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas del subset 3 que tienen 'Line of Buss' en la lista de valores_a_excluir
AÑO_ACTUALSubset_3.loc[AÑO_ACTUALSubset_3['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3.dropna(subset=['Line of Buss'])

# Concatenar todos los subsets
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1, AÑO_ACTUALSubset_2, AÑO_ACTUALSubset_3], ignore_index=True)

# Crear a6010 usando operaciones de pandas
a6010 = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '010',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en a6010 antes de proceder
if len(a6010) > 0:
    # Agregar columna notas
    a6010['notas'] = ""
    
    # Agregar a6010 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6010], ignore_index=True)

# Calcular s6010 como la suma de los saldos
s6010 = int(round(a6010['Saldos'].sum()))

# 6010 PASADO VENCIDO DEMAS RAMOS 

In [96]:
# Filtrar para obtener registros que contienen diferentes códigos
N1Subset_2 = N1[N1['6d'].astype(str).str.contains('515150', na=False)]
N1Subset_3 = N1[N1['6d'].astype(str).str.contains('515155', na=False)]
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('515445', na=False)]
N1Subset = N1[N1['6d'].astype(str).str.contains('515450', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N1Subset.loc[N1Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Concatenar todos los subsets
N1Subset = pd.concat([N1Subset, N1Subset_1, N1Subset_2, N1Subset_3], ignore_index=True)

# Crear v6010_1 usando operaciones de pandas
v6010_1 = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '010',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6010_1 antes de proceder
if len(v6010_1) > 0:
    # Agregar columna notas
    v6010_1['notas'] = ""
    
    # Agregar v6010_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_1], ignore_index=True)

# Calcular s6010_1 como el valor absoluto de la suma de los saldos
s6010_1 = int(round(abs(v6010_1['Saldos'].sum())))

# 6010 #N-2 VENCIDO DEMAS RAMOS

In [97]:
# Filtrar para obtener registros que contienen diferentes códigos
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('515150', na=False)]
N2Subset_3 = N2[N2['6d'].astype(str).str.contains('515155', na=False)]
N2Subset_1 = N2[N2['6d'].astype(str).str.contains('515445', na=False)]
N2Subset = N2[N2['6d'].astype(str).str.contains('515450', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N2Subset.loc[N2Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N2Subset = N2Subset.dropna(subset=['Line of Buss'])

# Concatenar todos los subsets
N2Subset = pd.concat([N2Subset, N2Subset_1, N2Subset_2, N2Subset_3], ignore_index=True)

# Crear v6010_2 usando operaciones de pandas
v6010_2 = (N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '010',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6010_2 antes de proceder
if len(v6010_2) > 0:
    # Agregar columna notas
    v6010_2['notas'] = ""
    
    # Agregar v6010_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_2], ignore_index=True)

# Calcular s6010_2 como el valor absoluto de la suma de los saldos
s6010_2 = int(round(abs(v6010_2['Saldos'].sum())))

# 6010 #N-3 VENCIDO DEMAS RAMOS

In [98]:
# Filtrar para obtener registros que contienen diferentes códigos
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('515150', na=False)]
N3Subset_3 = N3[N3['6d'].astype(str).str.contains('515155', na=False)]
N3Subset_1 = N3[N3['6d'].astype(str).str.contains('515445', na=False)]
N3Subset = N3[N3['6d'].astype(str).str.contains('515450', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset.loc[N3Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Concatenar todos los subsets
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], ignore_index=True)

# Crear v6010_3 usando operaciones de pandas
v6010_3 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '010',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6010_3 antes de proceder
if len(v6010_3) > 0:
    # Agregar columna notas
    v6010_3['notas'] = ""
    
    # Agregar v6010_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_3], ignore_index=True)

# Calcular s6010_3 como el valor absoluto de la suma de los saldos
s6010_3 = int(round(abs(v6010_3['Saldos'].sum())))

# 6010 #N-3 MES CORRIDO DEMAS RAMOS

In [99]:
# Filtrar para obtener registros que contienen diferentes códigos
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('515150', na=False)]
N3Subset_3 = N3[N3['6d'].astype(str).str.contains('515155', na=False)]
N3Subset_1 = N3[N3['6d'].astype(str).str.contains('515445', na=False)]
N3Subset = N3[N3['6d'].astype(str).str.contains('515450', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset.loc[N3Subset['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Concatenar todos los subsets
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], ignore_index=True)

# Crear v6010_4 usando operaciones de pandas
v6010_4 = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '010',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6010_4 antes de proceder
if len(v6010_4) > 0:
    # Agregar columna notas
    v6010_4['notas'] = ""
    
    # Agregar v6010_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_4], ignore_index=True)

# Calcular s6010_4 como el valor absoluto de la suma de los saldos multiplicado por -1
s6010_4 = int(round(abs(v6010_4['Saldos'].sum()) * -1))

# 6055 DEMAS RAMOS

In [100]:
# Filtrar para obtener registros que contienen el código "419530"
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('419530', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
AÑO_ACTUALSubset_2.loc[AÑO_ACTUALSubset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2.dropna(subset=['Line of Buss'])

# Crear a6055 usando operaciones de pandas
a6055 = (AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
        .rename(columns={'Total Cuenta': 'Saldos', 
                         'Saldo mes pasado': 'saldos_ant'})
        .assign(
            Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
        )
        .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
        .rename(columns={'NOMBRE': 'Nombre Cuenta'})
        .assign(
            **{'Unidad de captura': 6, 
               'Subcuenta': '055',
               'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
               'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
        )
        .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
        [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
          'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
          'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
        .reset_index(drop=True))

# Verificar si hay filas en a6055 antes de proceder
if len(a6055) > 0:
    # Agregar columna notas
    a6055['notas'] = ""
    
    # Agregar a6055 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6055], ignore_index=True)

# Calcular s6055 como el valor absoluto de la suma de los saldos
s6055 = int(round(abs(a6055['Saldos'].sum())))

# 6055 PASADO VENCIDO DEMAS RAMOS 

In [101]:
# Filtrar para obtener registros que contienen el código "419530"
N1Subset_2 = N1[N1['6d'].astype(str).str.contains('419530', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N1Subset_2.loc[N1Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N1Subset_2 = N1Subset_2.dropna(subset=['Line of Buss'])

# Crear v6055_1 usando operaciones de pandas
v6055_1 = (N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '055',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6055_1 antes de proceder
if len(v6055_1) > 0:
    # Agregar columna notas
    v6055_1['notas'] = ""
    
    # Agregar v6055_1 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_1], ignore_index=True)

# Calcular s6055_1 como el valor absoluto de la suma de los saldos
s6055_1 = int(round(abs(v6055_1['Saldos'].sum())))

#  6055 #N-2 VENCIDO DEMAS RAMOS 

In [102]:
# Filtrar para obtener registros que contienen el código "419530"
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('419530', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N2Subset_2.loc[N2Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N2Subset_2 = N2Subset_2.dropna(subset=['Line of Buss'])

# Crear v6055_2 usando operaciones de pandas
v6055_2 = (N2Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': '055',
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6055_2 antes de proceder
if len(v6055_2) > 0:
    # Agregar columna notas
    v6055_2['notas'] = ""
    
    # Agregar v6055_2 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_2], ignore_index=True)

# Calcular s6055_2 como el valor absoluto de la suma de los saldos
s6055_2 = int(round(abs(v6055_2['Saldos'].sum())))

# 6055 #N-3 VENCIDO DEMAS RAMOS 

In [103]:
# Filtrar para obtener registros que contienen el código "419530"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('419530', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset_2.loc[N3Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset_2 = N3Subset_2.dropna(subset=['Line of Buss'])

# Crear v6055_3 usando operaciones de pandas
v6055_3 = (N3Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'Total Cuenta': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': "055",
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6055_3 antes de proceder
if len(v6055_3) > 0:
    # Agregar columna notas
    v6055_3['notas'] = ""
    
    # Agregar v6055_3 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_3], ignore_index=True)

# Calcular s6055_3 como el valor absoluto de la suma de los saldos
# MODIFICACIÓN: Convertir a entero redondeado
s6055_3 = int(round(abs(v6055_3['Saldos'].sum())))

#  6055 #N-3 CORRIDO DEMAS RAMOS

In [104]:
# Filtrar para obtener registros que contienen el código "419530"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('419530', na=False)]

# Lista de valores para filtrar
valores_a_excluir = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                     "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                     "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                     "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                     "E58","E59","E91","E94","E90","D16","E15"]

# Establecer como NA las filas que tienen 'Line of Buss' en la lista de valores_a_excluir
N3Subset_2.loc[N3Subset_2['Line of Buss'].isin(valores_a_excluir), :] = np.nan

# Eliminar las filas con NA en 'Line of Buss'
N3Subset_2 = N3Subset_2.dropna(subset=['Line of Buss'])

# Crear v6055_4 usando operaciones de pandas
v6055_4 = (N3Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
         .rename(columns={'acumulado_mes': 'Saldos', 
                          'Saldo mes pasado': 'saldos_ant'})
         .assign(
             Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
         )
         .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
         .rename(columns={'NOMBRE': 'Nombre Cuenta'})
         .assign(
             **{'Unidad de captura': 6, 
                'Subcuenta': "055",
                'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
         )
         .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
         [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
           'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
           'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
         .reset_index(drop=True))

# Verificar si hay filas en v6055_4 antes de proceder
if len(v6055_4) > 0:
    # Agregar columna notas
    v6055_4['notas'] = ""
    
    # Agregar v6055_4 a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_4], ignore_index=True)

# Calcular s6055_4 como la suma de los saldos (sin valor absoluto en este caso)
s6055_4 = int(round(v6055_4['Saldos'].sum()))

# SUMATORIAS DEMAS RAMOS UNIDAD 6

In [105]:
# Sumar los saldos de v6025_2 y v6030_2
p6025 = int(round(v6025_2['Saldos'].sum()))
p6030 = int(round(v6030_2['Saldos'].sum()))

# Sumar las variables s para cada grupo
p6005 = s6005 + s6005_1 + s6005_2 + s6005_3 + s6005_4
p6010 = s6010 + s6010_1 + s6010_2 + s6010_3 + s6010_4
p6055 = s6055 + s6055_1 + s6055_2 + s6055_3 + s6055_4

# DATAFRAME DEMAS RAMOS UNIDAD 6

In [106]:
# Crear dataframe unidad6998
saldos06 = [p6005, p6010, s6015, s6020, s6025_2, s6030_2, p6055]
subcuentas06 = ['005', '010', '015', '020', '025', '030', '055']
unidad6998 = pd.DataFrame({
    'subcuentas06': subcuentas06,
    'saldos06': saldos06
})

# Redondear y tomar valor absoluto
unidad6998['saldos06'] = np.round(unidad6998['saldos06'], 0).astype(int)
unidad6998['saldos06'] = np.abs(unidad6998['saldos06'])

# Cálculos extra
v6060 = unidad6998['saldos06'].iloc[0:4].sum() - unidad6998['saldos06'].iloc[4:7].sum()
v6065 = int(round(v6060 / 3))
v6070 = round(min(UVR * 6600000, v6065) * 0.27, 0)
v6075 = round(max(v6065 - UVR * 6600000, 0) * 0.24, 0)
v6115 = round(v6070 + v6075, 0)

# Crear dataframe adicional y concatenarlo con el primero
subcuentas06_2 = ['060', '065', '070', '075', '115']
saldos06_2 = [v6060, v6065, v6070, v6075, v6115]
unidad6998_2 = pd.DataFrame({
    'subcuentas06': subcuentas06_2,
    'saldos06': saldos06_2
})

# Combinar los dataframes
unidad6998 = pd.concat([unidad6998, unidad6998_2], ignore_index=True)

#  6020 Previsionales de invalidez

In [107]:
# Filtrar AÑO_ACTUAL para registros que comienzan con "168010" en la columna "6d"
# Convertir la columna a string primero
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('^168010', regex=True)]

# Filtrar solo las filas donde "Line of Buss" es "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'E15']

# Crear a6020pi usando operaciones de pandas
a6020pi = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                            'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta'})
           .assign(
               **{'Unidad de captura': 6, 
                  'Subcuenta': "020",
                  'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
           )
           .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en a6020pi antes de proceder
if len(a6020pi) > 0:
    # Agregar columna notas
    a6020pi['notas'] = ""
    
    # Agregar a6020pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6020pi], ignore_index=True)

# Calcular s6020pi como la suma de los saldos y convertirlo a entero
s6020pi = int(round(a6020pi['Saldos'].sum())) if len(a6020pi) > 0 else 0

# 6015 Previsionales de invalidez 

In [108]:
# Filtrar AÑO_ACTUAL para registros que comienzan con "^265015" o "^168010" en la columna "6d"
mask_265015 = AÑO_ACTUAL['6d'].astype(str).str.contains('^265015', regex=True)
mask_168010 = AÑO_ACTUAL['6d'].astype(str).str.contains('^168010', regex=True)
AÑO_ACTUALSubset = AÑO_ACTUAL[mask_265015 | mask_168010]

# Filtrar solo las filas donde "Line of Buss" es "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == 'E15']

# Crear a6015pi usando operaciones de pandas
a6015pi = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                            'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta'})
           .assign(
               **{'Unidad de captura': 6, 
                  'Subcuenta': "015",
                  'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
           )
           .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en a6015pi antes de proceder
if len(a6015pi) > 0:
    # Agregar columna notas
    a6015pi['notas'] = ""
    
    # Agregar a6015pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6015pi], ignore_index=True)

# Calcular s6015pi como el valor absoluto de la suma de los saldos convertido a entero
s6015pi = int(abs(a6015pi['Saldos'].sum())) if len(a6015pi) > 0 else 0

# 6025 PASADO VENCIDO Previsionales de invalidez 

In [109]:
# Filtrar N3 para registros que comienzan con "^265015" o "^168010" en la columna "6d"
mask_265015 = N3['6d'].astype(str).str.contains('^265015', regex=True)
mask_168010 = N3['6d'].astype(str).str.contains('^168010', regex=True)
N3Subset = N3[mask_265015 | mask_168010]

# Filtrar solo las filas donde "Line of Buss" es "E15"
N3Subset = N3Subset[N3Subset['Line of Buss'] == 'E15']

# Crear v6025_2pi usando operaciones de pandas
v6025_2pi = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
            .rename(columns={'acumulado_mes': 'Saldos', 
                             'Arrastre saldos': 'saldos_ant'})
            .assign(
                Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
            )
            .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
            .rename(columns={'NOMBRE': 'Nombre Cuenta'})
            .assign(
                **{'Unidad de captura': 6, 
                   'Subcuenta': "025",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
            )
            .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
            [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
              'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
              'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
            .reset_index(drop=True))

# Verificar si hay filas en v6025_2pi antes de proceder
if len(v6025_2pi) > 0:
    # Agregar columna notas
    v6025_2pi['notas'] = ""
    
    # Concatenar v6025_2pi a bd_analisis_cuentas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6025_2pi], ignore_index=True)

# Calcular s6025_2pi como la suma de 'Saldos' y 'Saldos del mes pasado' y convertir a entero
s6025_2pi = int(v6025_2pi['Saldos'].sum() + v6025_2pi['Saldos del mes pasado'].sum()) if len(v6025_2pi) > 0 else 0

# 6030 PERÍODO ANTERIOR Previsionales de invalidez 

In [110]:
# Filtrar N3 para registros que comienzan con "^168010" en la columna "6d"
mask_168010 = N3['6d'].astype(str).str.contains('^168010', regex=True)
N3Subset = N3[mask_168010]

# Filtrar solo las filas donde "Line of Buss" es "E15"
N3Subset = N3Subset[N3Subset['Line of Buss'] == 'E15']

# Crear v6030_2pi usando operaciones de pandas
v6030_2pi = (N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']]
            .rename(columns={'acumulado_mes': 'Saldos', 
                             'Arrastre saldos': 'saldos_ant'})
            .assign(
                Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
            )
            .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
            .rename(columns={'NOMBRE': 'Nombre Cuenta'})
            .assign(
                **{'Unidad de captura': 6, 
                   'Subcuenta': "025",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
            )
            .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
            [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
              'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
              'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
            .reset_index(drop=True))

# Verificar si hay filas en v6030_2pi antes de proceder
if len(v6030_2pi) > 0:
    # Agregar columna notas
    v6030_2pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6030_2pi.columns)))
    
    # Usar solo las columnas comunes para la concatenación
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6030_2pi[common_cols]], 
                                    ignore_index=True)

# Calcular s6030_2pi como la suma de 'Saldos' y 'Saldos del mes pasado'
s6030_2pi = int(v6030_2pi['Saldos'].sum() + v6030_2pi['Saldos del mes pasado'].sum()) if len(v6030_2pi) > 0 else 0

# 6005 Previsionales de invalidez 

In [111]:
# Filtrar AÑO_ACTUAL para encontrar filas donde la columna '6d' contiene "512115"
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('512115')]

# Filtrar para mantener solo las filas donde "Line of Buss" es "E15"
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2[AÑO_ACTUALSubset_2['Line of Buss'] == 'E15']

# Crear a6005pi usando operaciones de pandas
a6005pi = (AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                            'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta'})
           .assign(
               **{'Unidad de captura': 6, 
                  'Subcuenta': "005",
                  'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                  'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
           )
           .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
           .reset_index(drop=True))

# Verificar si hay filas en a6005pi antes de proceder
if len(a6005pi) > 0:
    # Agregar columna notas
    a6005pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(a6005pi.columns)))
    
    # Usar solo las columnas comunes para la concatenación
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], a6005pi[common_cols]], 
                                    ignore_index=True)

# Calcular s6005pi como el valor absoluto de la suma de 'Saldos', redondeado al entero más cercano
s6005pi = int(round(abs(a6005pi['Saldos'].sum()))) if len(a6005pi) > 0 else 0

#  6005 PASADO VENCIDO Previsionales de invalidez

In [112]:
# Filtrar N1 para encontrar filas donde la columna '6d' contiene "512115"
N1Subset_2 = N1[N1['6d'].astype(str).str.contains('512115')]

# Filtrar para mantener solo las filas donde "Line of Buss" es "E15"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'] == 'E15']

# Crear v6005_1pi usando operaciones de pandas
v6005_1pi = (N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                             'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                    'Subcuenta': "005",
                    'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                    'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
             .reset_index(drop=True))

# Verificar si hay filas en v6005_1pi antes de proceder
if len(v6005_1pi) > 0:
    # Agregar columna notas
    v6005_1pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6005_1pi.columns)))
    
    # Usar solo las columnas comunes para la concatenación
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6005_1pi[common_cols]], 
                                    ignore_index=True)

# Calcular s6005_1pi como el valor absoluto de la suma de 'Saldos'
s6005_1pi = int(round(abs(v6005_1pi['Saldos'].sum()))) if len(v6005_1pi) > 0 else 0

# 6005 #N-2 VENCIDO Previsionales de invalidez

In [113]:
# Filtrar N2 para filas donde '6d' contiene "512115"
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('512115')]

# Filtrar para filas donde "Line of Buss" es "E15"
N2Subset_2 = N2Subset_2[N2Subset_2['Line of Buss'] == 'E15']

# Crear v6005_2pi con transformaciones
v6005_2pi = (N2Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                              'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "005",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6005_2pi) > 0:
    # Agregar columna notas
    v6005_2pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6005_2pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6005_2pi[common_cols]], 
                                    ignore_index=True)

# Calcular valor absoluto de la suma con redondeo a entero
s6005_2pi = int(round(abs(v6005_2pi['Saldos'].sum()))) if len(v6005_2pi) > 0 else 0

#  6005 #N-3 VENCIDO Previsionales de invalidez 

In [114]:
# Filtrar N3 para filas donde '6d' contiene "512115"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('512115')]

# Filtrar para filas donde "Line of Buss" es "E15"
N3Subset_2 = N3Subset_2[N3Subset_2['Line of Buss'] == 'E15']

# Crear v6005_3pi con transformaciones
v6005_3pi = (N3Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                              'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "005",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6005_3pi) > 0:
    # Agregar columna notas
    v6005_3pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6005_3pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6005_3pi[common_cols]], 
                                    ignore_index=True)

# Calcular valor absoluto de la suma con redondeo a entero
s6005_3pi = int(round(abs(v6005_3pi['Saldos'].sum()))) if len(v6005_3pi) > 0 else 0

# 6005 #N-3 MES CORRIDO Previsionales de invalidez

In [115]:
# Filtrar N3 para filas donde '6d' contiene "512115"
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('512115')]

# Filtrar para filas donde "Line of Buss" es "E15"
N3Subset_2 = N3Subset_2[N3Subset_2['Line of Buss'] == 'E15']

# Crear v6005_4pi con transformaciones
v6005_4pi = (N3Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'acumulado_mes': 'Saldos', 
                              'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "005",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6005_4pi) > 0:
    # Agregar columna notas
    v6005_4pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6005_4pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6005_4pi[common_cols]], 
                                    ignore_index=True)

# Calcular valor absoluto de la suma con redondeo a entero y multiplicar por -1
s6005_4pi = int(round(abs(v6005_4pi['Saldos'].sum()))) * -1 if len(v6005_4pi) > 0 else 0

#  6010 Previsionales de invalidez

In [116]:
# Filtrar datos de AÑO_ACTUAL
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515150')]
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515155')]
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515445')]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('515450')]

# Filtrar AÑO_ACTUALSubset_3 por "E15" en "Line of Buss"
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3.dropna(subset=['Line of Buss'])

# Combinar todos los subsets
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1, 
                             AÑO_ACTUALSubset_2, AÑO_ACTUALSubset_3], 
                             ignore_index=True)

# Crear a6010pi con transformaciones
a6010pi = (AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta'})
           .assign(
               **{'Unidad de captura': 6, 
                 'Subcuenta': "010",
                 'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                 'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
           )
           .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                           'Ejercicio': 'mes'})
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'mes']]
           .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(a6010pi) > 0:
    # Agregar columna notas
    a6010pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(a6010pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], a6010pi[common_cols]], 
                                    ignore_index=True)

# Calcular valor absoluto de la suma redondeado al entero más cercano
s6010pi = int(round(abs(a6010pi['Saldos'].sum()))) if len(a6010pi) > 0 else 0

# 6010 PASADO VENCIDO Previsionales de invalidez 

In [117]:
# Filtrar datos de N1
N1Subset_2 = N1[N1['6d'].astype(str).str.contains('515150')]
N1Subset_3 = N1[N1['6d'].astype(str).str.contains('515155')]
N1Subset_1 = N1[N1['6d'].astype(str).str.contains('515445')]
N1Subset = N1[N1['6d'].astype(str).str.contains('515450')]

# Filtrar N1Subset por "E15" en "Line of Buss"
N1Subset = N1Subset[N1Subset['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Combinar todos los subsets
N1Subset = pd.concat([N1Subset, N1Subset_1, N1Subset_2, N1Subset_3], 
                     ignore_index=True)

# Crear v6010_1pi con transformaciones
v6010_1pi = (N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                             'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "010",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                             'Ejercicio': 'mes'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'mes']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6010_1pi) > 0:
    # Agregar columna notas
    v6010_1pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6010_1pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6010_1pi[common_cols]], 
                                   ignore_index=True)

# Calcular valor absoluto de la suma
s6010_1pi = int(round(abs(v6010_1pi['Saldos'].sum()))) if len(v6010_1pi) > 0 else 0

# 6010 #N-2 VENCIDO Previsionales de invalidez

In [118]:
# Filtrar datos de N2
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('515150')]
N2Subset_3 = N2[N2['6d'].astype(str).str.contains('515155')]
N2Subset_1 = N2[N2['6d'].astype(str).str.contains('515445')]
N2Subset = N2[N2['6d'].astype(str).str.contains('515450')]

# Filtrar N2Subset por "E15" en "Line of Buss"
N2Subset = N2Subset[N2Subset['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
N2Subset = N2Subset.dropna(subset=['Line of Buss'])

# Combinar todos los subsets
N2Subset = pd.concat([N2Subset, N2Subset_1, N2Subset_2, N2Subset_3], 
                     ignore_index=True)

# Crear v6010_2pi con transformaciones
v6010_2pi = (N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                             'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "010",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                             'Ejercicio': 'mes'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'mes']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6010_2pi) > 0:
    # Agregar columna notas
    v6010_2pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6010_2pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6010_2pi[common_cols]], 
                                   ignore_index=True)

# Calcular valor absoluto de la suma 
s6010_2pi = int(round(abs(v6010_2pi['Saldos'].sum()))) if len(v6010_2pi) > 0 else 0

# 6010 #N-3 Previsionales de invalidez 

In [119]:
# Filtrar datos de N3
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('515150')]
N3Subset_3 = N3[N3['6d'].astype(str).str.contains('515155')]
N3Subset_1 = N3[N3['6d'].astype(str).str.contains('515445')]
N3Subset = N3[N3['6d'].astype(str).str.contains('515450')]

# Filtrar N3Subset por "E15" en "Line of Buss"
N3Subset = N3Subset[N3Subset['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Combinar todos los subsets
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], 
                     ignore_index=True)

# Crear v6010_3pi con transformaciones
v6010_3pi = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                             'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "010",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                             'Ejercicio': 'mes'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'mes']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6010_3pi) > 0:
    # Agregar columna notas
    v6010_3pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6010_3pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6010_3pi[common_cols]], 
                                   ignore_index=True)

# Calcular valor absoluto de la suma
s6010_3pi = int(round(abs(v6010_3pi['Saldos'].sum()))) if len(v6010_3pi) > 0 else 0

# 6010 #N-3 MES CORRIDO Previsionales de invalidez 

In [120]:
# Filtrar datos de N3
N3Subset_2 = N3[N3['6d'].astype(str).str.contains('515150')]
N3Subset_3 = N3[N3['6d'].astype(str).str.contains('515155')]
N3Subset_1 = N3[N3['6d'].astype(str).str.contains('515445')]
N3Subset = N3[N3['6d'].astype(str).str.contains('515450')]

# Filtrar N3Subset por "E15" en "Line of Buss"
N3Subset = N3Subset[N3Subset['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
N3Subset = N3Subset.dropna(subset=['Line of Buss'])

# Combinar todos los subsets
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], 
                     ignore_index=True)

# Crear v6010_4pi con transformaciones
v6010_4pi = (N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
             .rename(columns={'Total Cuenta': 'Saldos', 
                             'Saldo mes pasado': 'saldos_ant'})
             .assign(
                 Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
             )
             .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
             .rename(columns={'NOMBRE': 'Nombre Cuenta'})
             .assign(
                 **{'Unidad de captura': 6, 
                   'Subcuenta': "010",
                   'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                   'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
             )
             .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                             'Ejercicio': 'mes'})
             [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
               'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
               'Diferencia mes a mes', 'Diferencia %', 'mes']]
             .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(v6010_4pi) > 0:
    # Agregar columna notas
    v6010_4pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(v6010_4pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], v6010_4pi[common_cols]], 
                                   ignore_index=True)

# Calcular valor absoluto de la suma y multiplicar por -1
s6010_4pi = abs(v6010_4pi['Saldos'].sum()) * -1 if len(v6010_4pi) > 0 else 0

# 6055 Previsionales de invalidez

In [121]:
# Filtrar datos de AÑO_ACTUAL
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('419530')]

# Filtrar por "E15" en "Line of Buss"
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2[AÑO_ACTUALSubset_2['Line of Buss'] == 'E15']

# Eliminar filas con valores NA en 'Line of Buss'
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2.dropna(subset=['Line of Buss'])

# Crear a6055pi con transformaciones
a6055pi = (AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
           .rename(columns={'Total Cuenta': 'Saldos', 
                           'Saldo mes pasado': 'saldos_ant'})
           .assign(
               Cuenta=lambda x: x['Cuenta alternativa'].astype(str).str[:6].astype(float)
           )
           .merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
           .rename(columns={'NOMBRE': 'Nombre Cuenta'})
           .assign(
               **{'Unidad de captura': 6, 
                 'Subcuenta': "055",
                 'Diferencia mes a mes': lambda x: x['Saldos'] - x['saldos_ant'],
                 'Diferencia %': lambda x: (x['Saldos'] - x['saldos_ant']) / x['Saldos']}
           )
           .rename(columns={'saldos_ant': 'Saldos del mes pasado',
                           'Ejercicio': 'mes'})
           [['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
             'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
             'Diferencia mes a mes', 'Diferencia %', 'mes']]
           .reset_index(drop=True))

# Verificar si hay filas antes de proceder
if len(a6055pi) > 0:
    # Agregar columna notas
    a6055pi['notas'] = ""
    
    # Encontrar columnas comunes entre ambos dataframes
    common_cols = list(set(bd_analisis_cuentas.columns).intersection(set(a6055pi.columns)))
    
    # Concatenar usando solo columnas comunes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas[common_cols], a6055pi[common_cols]], 
                                   ignore_index=True)

# Calcular valor absoluto de la suma y convertir a int o float de Python
if len(a6055pi) > 0:
    # Opción 1: Convertir a entero si el número debería ser entero
    s6055pi = int(abs(a6055pi['Saldos'].sum()))
    
    # Opción 2: Alternativamente, si necesitas mantenerlo como float pero en formato Python nativo
    # s6055pi = float(abs(a6055pi['Saldos'].sum()))
else:
    s6055pi = 0

# 6055 PASADO VENCIDO Previsionales de invalidez

In [122]:
# Filtrar N1 para obtener N1Subset_1
N1Subset_1 = N1[N1['6d'].astype(str).str.contains("419530")]
N1Subset_1 = N1Subset_1[N1Subset_1['Line of Buss'].isin(["E15"])]

# Seleccionar las columnas necesarias y realizar las transformaciones
v6055_1pi = N1Subset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6055_1pi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)
v6055_1pi['Cuenta alternativa'] = v6055_1pi['Cuenta alternativa'].astype(str)  # Asegurarse de que sea tipo cadena
v6055_1pi['Cuenta'] = v6055_1pi['Cuenta alternativa'].str[:6].astype(int)
v6055_1pi = v6055_1pi.merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6055_1pi.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)
v6055_1pi['Unidad de captura'] = 6
v6055_1pi['Subcuenta'] = "055"
v6055_1pi['Diferencia mes a mes'] = v6055_1pi['Saldos'] - v6055_1pi['saldos_ant']
v6055_1pi['Diferencia %'] = (v6055_1pi['Saldos'] - v6055_1pi['saldos_ant']) / v6055_1pi['Saldos']
v6055_1pi.rename(columns={'saldos_ant': 'Saldos del mes pasado', 'Ejercicio': 'mes'}, inplace=True)
v6055_1pi = v6055_1pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                       'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'mes']]

# Verificar si hay filas en v6055_1pi
if not v6055_1pi.empty:
    v6055_1pi['notas'] = ""  # Agregar nota si aplica y cambiar nombre de consulta
    
    # Verificar si bd_analisis_cuentas existe
    if 'bd_analisis_cuentas' not in globals():
        bd_analisis_cuentas = pd.DataFrame()
    
    # Asegurar compatibilidad de nombres de columnas
    if not bd_analisis_cuentas.empty:
        # Verificar si necesitamos cambiar "mes" por "Ejercicio" para mantener consistencia
        if 'Ejercicio' in bd_analisis_cuentas.columns and 'Ejercicio' not in v6055_1pi.columns:
            v6055_1pi.rename(columns={'mes': 'Ejercicio'}, inplace=True)
    
    # Intentar combinar los dataframes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_1pi], ignore_index=True)

# Corregir el acceso a la columna "Saldos del mes pasado"
s6055_1pi = float(v6055_1pi['Saldos'].sum())

# SUMATORIAS Previsionales de invalidez UNIDAD 6

In [123]:
# Suma de variables para calcular los totales
p6005pi = s6005pi + s6005_1pi + s6005_2pi + s6005_3pi + s6005_4pi
p6010pi = s6010pi + s6010_1pi + s6010_2pi + s6010_3pi + s6010_4pi
p6055pi = s6055pi + s6055_1pi

# Calcular suma de la columna de saldos, verificando diferentes posibles nombres
try:
    # Intentar con 'saldos' (minúscula)
    p6025pi = sum(v6025_2pi['saldos'])
except KeyError:
    try:
        # Intentar con 'Saldos' (mayúscula)
        p6025pi = sum(v6025_2pi['Saldos'])
    except KeyError:
        # Si no encuentra ninguna, imprimir columnas disponibles y asignar 0
        print(f"Columnas disponibles en v6025_2pi: {v6025_2pi.columns.tolist()}")
        p6025pi = 0

try:
    # Intentar con 'saldos' (minúscula)
    p6030pi = sum(v6030_2pi['saldos'])
except KeyError:
    try:
        # Intentar con 'Saldos' (mayúscula)
        p6030pi = sum(v6030_2pi['Saldos'])
    except KeyError:
        # Si no encuentra ninguna, imprimir columnas disponibles y asignar 0
        print(f"Columnas disponibles en v6030_2pi: {v6030_2pi.columns.tolist()}")
        p6030pi = 0

# DATAFRAME Previsionales de invalidez UNIDAD 6 

In [124]:
# Crear listas para las subcuentas y saldos
saldos06pi = [p6005pi, p6010pi, s6015pi, s6020pi, s6025_2pi, s6030_2pi, p6055pi]
subcuentas06pi = ['005', '010', '015', '020', '025', '030', '055']

# Crear DataFrame con los datos
unidad6998pi = pd.DataFrame({
    'subcuentas06pi': subcuentas06pi,
    'saldos06pi': saldos06pi
})

# Redondear los valores a 0 decimales
unidad6998pi['saldos06pi'] = unidad6998pi['saldos06pi'].round(0)

# Tomar el valor absoluto
unidad6998pi['saldos06pi'] = unidad6998pi['saldos06pi'].abs()

# Cálculos extra
v6060pi = unidad6998pi['saldos06pi'].iloc[0:4].sum() - unidad6998pi['saldos06pi'].iloc[4:7].sum()
v6065pi = v6060pi / 3
v6070pi = round(min(UVR * 6600000, v6065pi) * 0.27, 0)
v6075pi = round(max(v6065pi - UVR * 6600000, 0) * 0.24, 0)
v6115pi = v6075pi + v6070pi

# Crear segundo DataFrame con resultados adicionales
subcuentas06_2pi = ['060', '065', '070', '075', '115']
saldos06_2pi = [v6060pi, v6065pi, v6070pi, v6075pi, v6115pi]
unidad6998_2pi = pd.DataFrame({
    'subcuentas06pi': subcuentas06_2pi,
    'saldos06_2pi': saldos06_2pi
})

# Renombrar columnas del segundo DataFrame para que coincidan con el primero
unidad6998_2pi = unidad6998_2pi.rename(columns={'saldos06_2pi': 'saldos06pi'})

# Concatenar ambos DataFrames
unidad6998pi = pd.concat([unidad6998pi, unidad6998_2pi], ignore_index=True)

# 6020 Riesgos Profesionales 

In [125]:
# Asumiendo que AÑO_ACTUAL y CUIF son DataFrames ya cargados en Python
import pandas as pd
import numpy as np

# Equivalente a: AÑO_ACTUALSubset <- AÑO_ACTUAL %>% filter(stringr::str_detect(`6d`,"^168010"))
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('^168010', regex=True)]

# Equivalente a: table(AÑO_ACTUALSubset$`6d`)
conteo_6d = AÑO_ACTUALSubset['6d'].value_counts()

# Equivalente a: AÑO_ACTUALSubset <- AÑO_ACTUALSubset[which(AÑO_ACTUALSubset$`Line of Buss` %in% c("D16")),]
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'].isin(['D16'])]

# Equivalente a todo el pipeline de transformación que define a6020rp
a6020rp = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a6020rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Crear columna Cuenta
a6020rp['Cuenta'] = a6020rp['Cuenta alternativa'].str[:6]
a6020rp['Cuenta'] = pd.to_numeric(a6020rp['Cuenta'], errors='coerce')

# Unir con CUIF
a6020rp = pd.merge(a6020rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
a6020rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
a6020rp['Unidad de captura'] = 6
a6020rp['Subcuenta'] = "020"
a6020rp['Diferencia mes a mes'] = a6020rp['Saldos'] - a6020rp['saldos_ant']
a6020rp['Diferencia %'] = (a6020rp['Saldos'] - a6020rp['saldos_ant']) / a6020rp['Saldos']
a6020rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
a6020rp = a6020rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Convertir a DataFrame (aunque ya es un DataFrame)
a6020rp = pd.DataFrame(a6020rp)

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(a6020rp) > 0:
    a6020rp['notas'] = ""  # Agregar columna de notas vacía
    # Concatenar con bd_analisis_cuentas (suponiendo que ya existe)
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6020rp], ignore_index=True)
    except NameError:
        # Si bd_analisis_cuentas no existe, inicializarlo
        bd_analisis_cuentas = a6020rp.copy()

# Calcular la suma y convertirla a entero
s6020rp = int(a6020rp['Saldos'].sum()) if len(a6020rp) > 0 else 0

# 6015 Riesgos Profesionales

In [126]:
# Asumiendo que AÑO_ACTUAL y CUIF son DataFrames ya cargados en Python

# Primero, convierte la columna '6d' a string para poder usar métodos de string
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)

# Ahora filtra con str.contains
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('^265020|^168010', regex=True)]

# Equivalente a: table(AÑO_ACTUALSubset$`6d`)
conteo_6d = AÑO_ACTUALSubset['6d'].value_counts()

# Equivalente a: AÑO_ACTUALSubset<-AÑO_ACTUALSubset[which(AÑO_ACTUALSubset$`Line of Buss` %in% c("D16")),]
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'].isin(['D16'])]

# Creando a6015rp
a6015rp = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a6015rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Crear columna Cuenta - asegúrate de que 'Cuenta alternativa' es string también
a6015rp['Cuenta alternativa'] = a6015rp['Cuenta alternativa'].astype(str)
a6015rp['Cuenta'] = a6015rp['Cuenta alternativa'].str[:6]
a6015rp['Cuenta'] = pd.to_numeric(a6015rp['Cuenta'], errors='coerce')

# Unir con CUIF
a6015rp = pd.merge(a6015rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
a6015rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
a6015rp['Unidad de captura'] = 6
a6015rp['Subcuenta'] = "015"
a6015rp['Diferencia mes a mes'] = a6015rp['Saldos'] - a6015rp['saldos_ant']

# Manejo seguro para la división (evitar división por cero)
a6015rp['Diferencia %'] = 0  # Valor predeterminado
mask = a6015rp['Saldos'] != 0
a6015rp.loc[mask, 'Diferencia %'] = (a6015rp.loc[mask, 'Saldos'] - a6015rp.loc[mask, 'saldos_ant']) / a6015rp.loc[mask, 'Saldos']

a6015rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
a6015rp = a6015rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(a6015rp) > 0:
    a6015rp['notas'] = ""  # Agregar columna de notas vacía
    
    # Verificamos si bd_analisis_cuentas existe, si no, lo inicializamos
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6015rp], ignore_index=True)
    except NameError:
        # Si bd_analisis_cuentas no existe, inicializarlo con a6015rp
        bd_analisis_cuentas = a6015rp.copy()

# Calcular la suma y tomar valor absoluto, convertido a entero
s6015rp = round(abs(a6015rp['Saldos'].sum()))

# 6025 PASADO VENCIDO Riesgos Profesionales 

In [127]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Primero, convertir la columna '6d' a tipo string
N3['6d'] = N3['6d'].astype(str)

# Ahora podemos usar str.contains
N3Subset = N3[N3['6d'].str.contains('^265020|^168010', regex=True)]

# Equivalente a: table(N3Subset$`6d`)
conteo_6d = N3Subset['6d'].value_counts()

# Equivalente a: N3Subset<-N3Subset[which(N3Subset$`Line of Buss` %in% c("D16")),]
N3Subset = N3Subset[N3Subset['Line of Buss'].isin(['D16'])]

# Creando v6025_2rp
v6025_2rp = N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']].copy()
v6025_2rp.rename(columns={'acumulado_mes': 'Saldos', 'Arrastre saldos': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de aplicar .str
v6025_2rp['Cuenta alternativa'] = v6025_2rp['Cuenta alternativa'].astype(str)

# Crear columna Cuenta
v6025_2rp['Cuenta'] = v6025_2rp['Cuenta alternativa'].str[:6]
v6025_2rp['Cuenta'] = pd.to_numeric(v6025_2rp['Cuenta'])

# Unir con CUIF
v6025_2rp = pd.merge(v6025_2rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6025_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6025_2rp['Unidad de captura'] = 6
v6025_2rp['Subcuenta'] = "025"
v6025_2rp['Diferencia mes a mes'] = v6025_2rp['Saldos'] - v6025_2rp['saldos_ant']
v6025_2rp['Diferencia %'] = (v6025_2rp['Saldos'] - v6025_2rp['saldos_ant']) / v6025_2rp['Saldos']
v6025_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6025_2rp = v6025_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6025_2rp) > 0:
    v6025_2rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6025_2rp], ignore_index=True)

# Calcular la suma de Saldos y Saldos del mes pasado
s6025_2rp = int(round(v6025_2rp['Saldos'].sum() + v6025_2rp['Saldos del mes pasado'].sum()))

# 6030 PERÍODO ANTERIOR Riesgos Profesionales 

In [128]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Primero, convertir la columna '6d' a tipo string
N3['6d'] = N3['6d'].astype(str)

# Filtro con un solo patrón: "^168010"
N3Subset = N3[N3['6d'].str.contains('^168010', regex=True)]

# Equivalente a: table(N3Subset$`6d`)
conteo_6d = N3Subset['6d'].value_counts()

# Equivalente a: N3Subset<- N3Subset[which(N3Subset$`Line of Buss` %in% c("D16")),]
N3Subset = N3Subset[N3Subset['Line of Buss'].isin(['D16'])]

# Creando v6030_2rp
v6030_2rp = N3Subset[['Cuenta alternativa', 'acumulado_mes', 'Arrastre saldos', 'Ejercicio']].copy()
v6030_2rp.rename(columns={'acumulado_mes': 'Saldos', 'Arrastre saldos': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de aplicar .str
v6030_2rp['Cuenta alternativa'] = v6030_2rp['Cuenta alternativa'].astype(str)

# Crear columna Cuenta
v6030_2rp['Cuenta'] = v6030_2rp['Cuenta alternativa'].str[:6]
v6030_2rp['Cuenta'] = pd.to_numeric(v6030_2rp['Cuenta'])

# Unir con CUIF
v6030_2rp = pd.merge(v6030_2rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6030_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6030_2rp['Unidad de captura'] = 6
v6030_2rp['Subcuenta'] = "025"  # Mantuve "025" como en el código original
v6030_2rp['Diferencia mes a mes'] = v6030_2rp['Saldos'] - v6030_2rp['saldos_ant']
v6030_2rp['Diferencia %'] = (v6030_2rp['Saldos'] - v6030_2rp['saldos_ant']) / v6030_2rp['Saldos']
v6030_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6030_2rp = v6030_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6030_2rp) > 0:
    v6030_2rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6030_2rp], ignore_index=True)

# Calcular la suma de Saldos y Saldos del mes pasado
# Usando el nombre correcto de la columna en Python
s6030_2rp = int(round(v6030_2rp['Saldos'].sum() + v6030_2rp['Saldos del mes pasado'].sum()))

# 6005 Riesgos Profesionales 

In [129]:
# Asumiendo que AÑO_ACTUAL y CUIF son DataFrames ya cargados en Python
# Usar grep para filtrar las filas que contienen "512125" en la columna '6d'
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('512125')]

# Filtrar por Line of Buss igual a "D16"
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2[AÑO_ACTUALSubset_2['Line of Buss'].isin(['D16'])]

# Crear el dataframe a6005rp
a6005rp = AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a6005rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string y crear la columna Cuenta
a6005rp['Cuenta alternativa'] = a6005rp['Cuenta alternativa'].astype(str)
a6005rp['Cuenta'] = a6005rp['Cuenta alternativa'].str[:6]
a6005rp['Cuenta'] = pd.to_numeric(a6005rp['Cuenta'])

# Unir con CUIF
a6005rp = pd.merge(a6005rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
a6005rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
a6005rp['Unidad de captura'] = 6
a6005rp['Subcuenta'] = "005"
a6005rp['Diferencia mes a mes'] = a6005rp['Saldos'] - a6005rp['saldos_ant']
a6005rp['Diferencia %'] = (a6005rp['Saldos'] - a6005rp['saldos_ant']) / a6005rp['Saldos']
a6005rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
a6005rp = a6005rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(a6005rp) > 0:
    a6005rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6005rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos
s6005rp = int(round(abs(a6005rp['Saldos'].sum())))

# 6005 PASADO VENCIDO Riesgos Profesionales

In [130]:
# Asumiendo que N1 y CUIF son DataFrames ya cargados en Python
# Usar str.contains para filtrar las filas que contienen "512125" en la columna '6d'
N1['6d'] = N1['6d'].astype(str)
N1Subset_2 = N1[N1['6d'].str.contains('512125')]

# Filtrar por Line of Buss igual a "D16"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'].isin(['D16'])]

# Crear el dataframe v6005_1rp
v6005_1rp = N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6005_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string y crear la columna Cuenta
v6005_1rp['Cuenta alternativa'] = v6005_1rp['Cuenta alternativa'].astype(str)
v6005_1rp['Cuenta'] = v6005_1rp['Cuenta alternativa'].str[:6]
v6005_1rp['Cuenta'] = pd.to_numeric(v6005_1rp['Cuenta'])

# Unir con CUIF
v6005_1rp = pd.merge(v6005_1rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6005_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6005_1rp['Unidad de captura'] = 6
v6005_1rp['Subcuenta'] = "005"
v6005_1rp['Diferencia mes a mes'] = v6005_1rp['Saldos'] - v6005_1rp['saldos_ant']
v6005_1rp['Diferencia %'] = (v6005_1rp['Saldos'] - v6005_1rp['saldos_ant']) / v6005_1rp['Saldos']
v6005_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6005_1rp = v6005_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6005_1rp) > 0:
    v6005_1rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_1rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos
s6005_1rp = int(round(abs(v6005_1rp['Saldos'].sum())))

# 6005 #N-2 VENCIDO Riesgos Profesionales

In [131]:
# Asumiendo que N2 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N2['6d'] = N2['6d'].astype(str)

# Filtrar filas que contienen "512125" en la columna '6d'
N2Subset_2 = N2[N2['6d'].str.contains('512125')]

# Filtrar por Line of Buss igual a "D16"
N2Subset_2 = N2Subset_2[N2Subset_2['Line of Buss'].isin(['D16'])]

# Crear el dataframe v6005_2rp
v6005_2rp = N2Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6005_2rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6005_2rp['Cuenta alternativa'] = v6005_2rp['Cuenta alternativa'].astype(str)
v6005_2rp['Cuenta'] = v6005_2rp['Cuenta alternativa'].str[:6]
v6005_2rp['Cuenta'] = pd.to_numeric(v6005_2rp['Cuenta'])

# Unir con CUIF
v6005_2rp = pd.merge(v6005_2rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6005_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6005_2rp['Unidad de captura'] = 6
v6005_2rp['Subcuenta'] = "005"
v6005_2rp['Diferencia mes a mes'] = v6005_2rp['Saldos'] - v6005_2rp['saldos_ant']
v6005_2rp['Diferencia %'] = (v6005_2rp['Saldos'] - v6005_2rp['saldos_ant']) / v6005_2rp['Saldos']
v6005_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6005_2rp = v6005_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6005_2rp) > 0:
    v6005_2rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_2rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos
s6005_2rp = int(round(abs(v6005_2rp['Saldos'].sum())))

# 6005 #N-3 VENCIDO Riesgos Profesionales

In [132]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N3['6d'] = N3['6d'].astype(str)

# Filtrar filas que contienen "512125" en la columna '6d'
N3Subset_2 = N3[N3['6d'].str.contains('512125')]

# Filtrar por Line of Buss igual a "D16"
N3Subset_2 = N3Subset_2[N3Subset_2['Line of Buss'].isin(['D16'])]

# Crear el dataframe v6005_3rp
v6005_3rp = N3Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6005_3rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6005_3rp['Cuenta alternativa'] = v6005_3rp['Cuenta alternativa'].astype(str)
v6005_3rp['Cuenta'] = v6005_3rp['Cuenta alternativa'].str[:6]
v6005_3rp['Cuenta'] = pd.to_numeric(v6005_3rp['Cuenta'])

# Unir con CUIF
v6005_3rp = pd.merge(v6005_3rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6005_3rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6005_3rp['Unidad de captura'] = 6
v6005_3rp['Subcuenta'] = "005"
v6005_3rp['Diferencia mes a mes'] = v6005_3rp['Saldos'] - v6005_3rp['saldos_ant']
v6005_3rp['Diferencia %'] = (v6005_3rp['Saldos'] - v6005_3rp['saldos_ant']) / v6005_3rp['Saldos']
v6005_3rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6005_3rp = v6005_3rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6005_3rp) > 0:
    v6005_3rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_3rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos como entero
s6005_3rp = int(round(abs(v6005_3rp['Saldos'].sum())))

# 6005 #N-3 MES CORRIDO Riesgos Profesionales 

In [133]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N3['6d'] = N3['6d'].astype(str)

# Filtrar filas que contienen "512125" en la columna '6d'
N3Subset_2 = N3[N3['6d'].str.contains('512125')]

# Filtrar por Line of Buss igual a "D16"
N3Subset_2 = N3Subset_2[N3Subset_2['Line of Buss'].isin(['D16'])]

# Crear el dataframe v6005_4rp - nota: se cambió 'Total Cuenta' a 'acumulado_mes'
v6005_4rp = N3Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']].copy()
v6005_4rp.rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6005_4rp['Cuenta alternativa'] = v6005_4rp['Cuenta alternativa'].astype(str)
v6005_4rp['Cuenta'] = v6005_4rp['Cuenta alternativa'].str[:6]
v6005_4rp['Cuenta'] = pd.to_numeric(v6005_4rp['Cuenta'])

# Unir con CUIF
v6005_4rp = pd.merge(v6005_4rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6005_4rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6005_4rp['Unidad de captura'] = 6
v6005_4rp['Subcuenta'] = "005"
v6005_4rp['Diferencia mes a mes'] = v6005_4rp['Saldos'] - v6005_4rp['saldos_ant']
v6005_4rp['Diferencia %'] = (v6005_4rp['Saldos'] - v6005_4rp['saldos_ant']) / v6005_4rp['Saldos']
v6005_4rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6005_4rp = v6005_4rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6005_4rp) > 0:
    v6005_4rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6005_4rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos multiplicado por -1 (como entero)
s6005_4rp = int(round(abs(v6005_4rp['Saldos'].sum()) * -1))

# 6010 Riesgos Profesionales

In [134]:
# Asumiendo que AÑO_ACTUAL y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)

# Filtrar filas que contienen diferentes códigos en la columna '6d'
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('515150')]
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('515155')]
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('515445')]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('515450')]

# Filtrar filas con Line of Buss en "D16" - Nota: parece que esta línea no tiene efecto en el código R original
# ya que no asigna el resultado a ninguna variable
AÑO_ACTUALSubset_3.loc[AÑO_ACTUALSubset_3['Line of Buss'].isin(['D16'])]

# Filtrar filas donde Line of Buss no es NA
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[~AÑO_ACTUALSubset_3['Line of Buss'].isna()]

# Combinar todos los subconjuntos
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset, AÑO_ACTUALSubset_1, AÑO_ACTUALSubset_2, AÑO_ACTUALSubset_3], ignore_index=True)

# Crear el dataframe a6010rp
a6010rp = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a6010rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
a6010rp['Cuenta alternativa'] = a6010rp['Cuenta alternativa'].astype(str)
a6010rp['Cuenta'] = a6010rp['Cuenta alternativa'].str[:6]
a6010rp['Cuenta'] = pd.to_numeric(a6010rp['Cuenta'])

# Unir con CUIF
a6010rp = pd.merge(a6010rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
a6010rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
a6010rp['Unidad de captura'] = 6
a6010rp['Subcuenta'] = "010"
a6010rp['Diferencia mes a mes'] = a6010rp['Saldos'] - a6010rp['saldos_ant']
a6010rp['Diferencia %'] = (a6010rp['Saldos'] - a6010rp['saldos_ant']) / a6010rp['Saldos']
a6010rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
a6010rp = a6010rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(a6010rp) > 0:
    a6010rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6010rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos como entero
s6010rp = int(round(abs(a6010rp['Saldos'].sum())))

# 6010 PASADO VENCIDO Riesgos Profesionales

In [135]:
# Asumiendo que N1 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N1['6d'] = N1['6d'].astype(str)

# Filtrar filas que contienen diferentes códigos en la columna '6d'
N1Subset_2 = N1[N1['6d'].str.contains('515150')]
N1Subset_3 = N1[N1['6d'].str.contains('515155')]
N1Subset_1 = N1[N1['6d'].str.contains('515445')]
N1Subset = N1[N1['6d'].str.contains('515450')]

# Filtrar filas con Line of Buss en "D16" - Nota: esta línea no tiene efecto ya que no asigna el resultado
N1Subset.loc[N1Subset['Line of Buss'].isin(['D16'])]

# Filtrar filas donde Line of Buss no es NA
N1Subset = N1Subset[~N1Subset['Line of Buss'].isna()]

# Combinar todos los subconjuntos
N1Subset = pd.concat([N1Subset, N1Subset_1, N1Subset_2, N1Subset_3], ignore_index=True)

# Crear el dataframe v6010_1rp
v6010_1rp = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6010_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6010_1rp['Cuenta alternativa'] = v6010_1rp['Cuenta alternativa'].astype(str)
v6010_1rp['Cuenta'] = v6010_1rp['Cuenta alternativa'].str[:6]
v6010_1rp['Cuenta'] = pd.to_numeric(v6010_1rp['Cuenta'])

# Unir con CUIF
v6010_1rp = pd.merge(v6010_1rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6010_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6010_1rp['Unidad de captura'] = 6
v6010_1rp['Subcuenta'] = "010"
v6010_1rp['Diferencia mes a mes'] = v6010_1rp['Saldos'] - v6010_1rp['saldos_ant']
v6010_1rp['Diferencia %'] = (v6010_1rp['Saldos'] - v6010_1rp['saldos_ant']) / v6010_1rp['Saldos']
v6010_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6010_1rp = v6010_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6010_1rp) > 0:
    v6010_1rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_1rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos como entero
s6010_1rp = int(round(abs(v6010_1rp['Saldos'].sum())))

# 6010 #N-2 VENCIDO Riesgos Profesionales

In [136]:
# Asumiendo que N2 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N2['6d'] = N2['6d'].astype(str)

# Filtrar filas que contienen diferentes códigos en la columna '6d'
N2Subset_2 = N2[N2['6d'].str.contains('515150')]
N2Subset_3 = N2[N2['6d'].str.contains('515155')]
N2Subset_1 = N2[N2['6d'].str.contains('515445')]
N2Subset = N2[N2['6d'].str.contains('515450')]

# Filtrar filas con Line of Buss en "D16" - Nota: esta línea no tiene efecto ya que no asigna el resultado
N2Subset.loc[N2Subset['Line of Buss'].isin(['D16'])]

# Filtrar filas donde Line of Buss no es NA
N2Subset = N2Subset[~N2Subset['Line of Buss'].isna()]

# Combinar todos los subconjuntos
N2Subset = pd.concat([N2Subset, N2Subset_1, N2Subset_2, N2Subset_3], ignore_index=True)

# Crear el dataframe v6010_2rp
v6010_2rp = N2Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6010_2rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6010_2rp['Cuenta alternativa'] = v6010_2rp['Cuenta alternativa'].astype(str)
v6010_2rp['Cuenta'] = v6010_2rp['Cuenta alternativa'].str[:6]
v6010_2rp['Cuenta'] = pd.to_numeric(v6010_2rp['Cuenta'])

# Unir con CUIF
v6010_2rp = pd.merge(v6010_2rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6010_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6010_2rp['Unidad de captura'] = 6
v6010_2rp['Subcuenta'] = "010"
v6010_2rp['Diferencia mes a mes'] = v6010_2rp['Saldos'] - v6010_2rp['saldos_ant']
v6010_2rp['Diferencia %'] = (v6010_2rp['Saldos'] - v6010_2rp['saldos_ant']) / v6010_2rp['Saldos']
v6010_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6010_2rp = v6010_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6010_2rp) > 0:
    v6010_2rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_2rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos como entero
s6010_2rp = int(round(abs(v6010_2rp['Saldos'].sum())))

# 6010 #N-3 Riesgos Profesionales

In [137]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N3['6d'] = N3['6d'].astype(str)

# Filtrar filas que contienen diferentes códigos en la columna '6d'
N3Subset_2 = N3[N3['6d'].str.contains('515150')]
N3Subset_3 = N3[N3['6d'].str.contains('515155')]
N3Subset_1 = N3[N3['6d'].str.contains('515445')]
N3Subset = N3[N3['6d'].str.contains('515450')]

# Filtrar filas con Line of Buss en "D16" - Nota: esta línea no tiene efecto ya que no asigna el resultado
N3Subset.loc[N3Subset['Line of Buss'].isin(['D16'])]

# Filtrar filas donde Line of Buss no es NA
N3Subset = N3Subset[~N3Subset['Line of Buss'].isna()]

# Combinar todos los subconjuntos
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], ignore_index=True)

# Crear el dataframe v6010_3rp
v6010_3rp = N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6010_3rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6010_3rp['Cuenta alternativa'] = v6010_3rp['Cuenta alternativa'].astype(str)
v6010_3rp['Cuenta'] = v6010_3rp['Cuenta alternativa'].str[:6]
v6010_3rp['Cuenta'] = pd.to_numeric(v6010_3rp['Cuenta'])

# Unir con CUIF
v6010_3rp = pd.merge(v6010_3rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6010_3rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6010_3rp['Unidad de captura'] = 6
v6010_3rp['Subcuenta'] = "010"
v6010_3rp['Diferencia mes a mes'] = v6010_3rp['Saldos'] - v6010_3rp['saldos_ant']
v6010_3rp['Diferencia %'] = (v6010_3rp['Saldos'] - v6010_3rp['saldos_ant']) / v6010_3rp['Saldos']
v6010_3rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6010_3rp = v6010_3rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6010_3rp) > 0:
    v6010_3rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_3rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos como entero
s6010_3rp = int(round(abs(v6010_3rp['Saldos'].sum())))

# 6010 #N-3 MES CORRIDO Riesgos Profesionales

In [138]:
# Asumiendo que N3 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N3['6d'] = N3['6d'].astype(str)

# Filtrar filas que contienen diferentes códigos en la columna '6d'
N3Subset_2 = N3[N3['6d'].str.contains('515150')]
N3Subset_3 = N3[N3['6d'].str.contains('515155')]
N3Subset_1 = N3[N3['6d'].str.contains('515445')]
N3Subset = N3[N3['6d'].str.contains('515450')]

# Filtrar filas con Line of Buss en "D16" - Nota: esta línea no tiene efecto ya que no asigna el resultado
N3Subset.loc[N3Subset['Line of Buss'].isin(['D16'])]

# Filtrar filas donde Line of Buss no es NA
N3Subset = N3Subset[~N3Subset['Line of Buss'].isna()]

# Combinar todos los subconjuntos
N3Subset = pd.concat([N3Subset, N3Subset_1, N3Subset_2, N3Subset_3], ignore_index=True)

# Crear el dataframe v6010_4rp
v6010_4rp = N3Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6010_4rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6010_4rp['Cuenta alternativa'] = v6010_4rp['Cuenta alternativa'].astype(str)
v6010_4rp['Cuenta'] = v6010_4rp['Cuenta alternativa'].str[:6]
v6010_4rp['Cuenta'] = pd.to_numeric(v6010_4rp['Cuenta'])

# Unir con CUIF
v6010_4rp = pd.merge(v6010_4rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6010_4rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6010_4rp['Unidad de captura'] = 6
v6010_4rp['Subcuenta'] = "010"
v6010_4rp['Diferencia mes a mes'] = v6010_4rp['Saldos'] - v6010_4rp['saldos_ant']
v6010_4rp['Diferencia %'] = (v6010_4rp['Saldos'] - v6010_4rp['saldos_ant']) / v6010_4rp['Saldos']
v6010_4rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6010_4rp = v6010_4rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6010_4rp) > 0:
    v6010_4rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6010_4rp], ignore_index=True)

# Calcular el valor absoluto de la suma de los saldos multiplicado por -1 (como entero)
s6010_4rp = int(round(abs(v6010_4rp['Saldos'].sum()) * -1))

# 6055 Riesgos Profesionales

In [139]:
# Filtrar filas de AÑO_ACTUAL que contengan "419530" en la columna '6d'
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('419530')]

# Filtrar filas donde 'Line of Buss' es igual a "D16"
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2[AÑO_ACTUALSubset_2['Line of Buss'] == 'D16']

# Eliminar filas donde 'Line of Buss' es NA
AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2[~AÑO_ACTUALSubset_2['Line of Buss'].isna()]

# Crear a6055rp seleccionando columnas específicas y renombrándolas
a6055rp = AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a6055rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertirlos a número
a6055rp['Cuenta'] = a6055rp['Cuenta alternativa'].astype(str).str[:6]
a6055rp['Cuenta'] = pd.to_numeric(a6055rp['Cuenta'])

# Unir con el DataFrame CUIF usando la columna Cuenta
a6055rp = pd.merge(a6055rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
a6055rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas adicionales
a6055rp['Unidad de captura'] = 6
a6055rp['Subcuenta'] = "055"
a6055rp['Diferencia mes a mes'] = a6055rp['Saldos'] - a6055rp['saldos_ant']
a6055rp['Diferencia %'] = (a6055rp['Saldos'] - a6055rp['saldos_ant']) / a6055rp['Saldos']
a6055rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
a6055rp = a6055rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en a6055rp, agregar columna de notas y concatenar con bd_analisis_cuentas
if len(a6055rp) > 0:
    a6055rp['notas'] = ""  # Agregar columna notas con valor vacío
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a6055rp], ignore_index=True)

# Calcular s6055rp manteniendo el signo original (sin usar abs())
s6055rp = int(a6055rp['Saldos'].sum())

# 6055 PASADO VENCIDO Riesgos Profesionales

In [140]:
# Asumiendo que N1 y CUIF son DataFrames ya cargados en Python
# Convertir la columna '6d' a string para poder usar str.contains()
N1['6d'] = N1['6d'].astype(str)

# Filtrar filas que contienen el código "419530" en la columna '6d'
N1Subset_2 = N1[N1['6d'].str.contains('419530')]

# Filtrar filas con Line of Buss en "D16"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'].isin(['D16'])]

# Parece haber un error en el código original - usa N1Subset_1 que no está definido
# Asumo que debe ser N1Subset_2 basado en el contexto
v6055_1rp = N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6055_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Convertir 'Cuenta alternativa' a tipo string antes de extraer los primeros 6 caracteres
v6055_1rp['Cuenta alternativa'] = v6055_1rp['Cuenta alternativa'].astype(str)
v6055_1rp['Cuenta'] = v6055_1rp['Cuenta alternativa'].str[:6]
v6055_1rp['Cuenta'] = pd.to_numeric(v6055_1rp['Cuenta'])

# Unir con CUIF
v6055_1rp = pd.merge(v6055_1rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6055_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas
v6055_1rp['Unidad de captura'] = 6
v6055_1rp['Subcuenta'] = "055"
v6055_1rp['Diferencia mes a mes'] = v6055_1rp['Saldos'] - v6055_1rp['saldos_ant']
v6055_1rp['Diferencia %'] = (v6055_1rp['Saldos'] - v6055_1rp['saldos_ant']) / v6055_1rp['Saldos']
v6055_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6055_1rp = v6055_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Agregar notas y combinar con bd_analisis_cuentas si hay filas
if len(v6055_1rp) > 0:
    v6055_1rp['notas'] = ""  # Agregar columna de notas vacía
    # Asumiendo que bd_analisis_cuentas es un DataFrame existente
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_1rp], ignore_index=True)

# Modificado para mantener el signo original de la suma de 'Saldos'
s6055_1rp = int(round(v6055_1rp['Saldos'].sum()))

#  6055 #N-2 VENCIDO Riesgos Profesionales

In [141]:
# Filtrar filas de N2 que contengan "419530" en la columna '6d'
N2Subset_2 = N2[N2['6d'].astype(str).str.contains('419530')]

# Filtrar filas donde 'Line of Buss' sea igual a "D16"
N2Subset_2 = N2Subset_2[N2Subset_2['Line of Buss'] == 'D16']

# Crear v6055_2rp seleccionando columnas específicas y renombrándolas
v6055_2rp = N2Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6055_2rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertirlos a número
v6055_2rp['Cuenta'] = v6055_2rp['Cuenta alternativa'].astype(str).str[:6]
v6055_2rp['Cuenta'] = pd.to_numeric(v6055_2rp['Cuenta'])

# Unir con el DataFrame CUIF usando la columna Cuenta
v6055_2rp = pd.merge(v6055_2rp, CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6055_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas adicionales
v6055_2rp['Unidad de captura'] = 6
v6055_2rp['Subcuenta'] = "055"
v6055_2rp['Diferencia mes a mes'] = v6055_2rp['Saldos'] - v6055_2rp['saldos_ant']
v6055_2rp['Diferencia %'] = (v6055_2rp['Saldos'] - v6055_2rp['saldos_ant']) / v6055_2rp['Saldos']
v6055_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y reordenar columnas
v6055_2rp = v6055_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en v6055_2rp, agregar columna de notas y concatenar con bd_analisis_cuentas
if len(v6055_2rp) > 0:
    v6055_2rp['notas'] = ""  # Agregar columna notas con valor vacío
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_2rp], ignore_index=True)

# Calcular s6055_2rp manteniendo el signo original de la suma de 'Saldos'
s6055_2rp = int(v6055_2rp['Saldos'].sum())

#  6055 #N-3 Riesgos Profesionales

In [142]:
# Filtrar N3 para obtener N3Subset_3
N3Subset_3 = N3[N3['6d'].astype(str).str.contains("419530")]
N3Subset_3 = N3Subset_3[N3Subset_3['Line of Buss'].isin(["D16"])]

# Seleccionar las columnas necesarias y realizar las transformaciones
v6055_3rp = N3Subset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v6055_3rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)
v6055_3rp['Cuenta alternativa'] = v6055_3rp['Cuenta alternativa'].astype(str)  # Asegurarse de que sea tipo cadena
v6055_3rp['Cuenta'] = v6055_3rp['Cuenta alternativa'].str[:6].astype(int)
v6055_3rp = v6055_3rp.merge(CUIF, left_on='Cuenta', right_on='CUENTA', how='left')
v6055_3rp.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)
v6055_3rp['Unidad de captura'] = 6
v6055_3rp['Subcuenta'] = "055"
v6055_3rp['Diferencia mes a mes'] = v6055_3rp['Saldos'] - v6055_3rp['saldos_ant']
v6055_3rp['Diferencia %'] = (v6055_3rp['Saldos'] - v6055_3rp['saldos_ant']) / v6055_3rp['Saldos']
v6055_3rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)
v6055_3rp = v6055_3rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                       'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Verificar si hay filas en v6055_3rp
if not v6055_3rp.empty:
    v6055_3rp['notas'] = ""  # Agregar nota si aplica y cambiar nombre de consulta
    
    # Verificar si bd_analisis_cuentas existe
    if 'bd_analisis_cuentas' not in globals():
        bd_analisis_cuentas = pd.DataFrame()
    
    # Intentar combinar los dataframes
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v6055_3rp], ignore_index=True)

# Calcular s6055_3rp
s6055_3rp = v6055_3rp['Saldos'].sum()

# Convertir el valor a un número flotante estándar de Python
s6055_3rp = float(s6055_3rp)

# SUMATORIAS Riesgos Profesionales UNIDAD 6 

In [143]:
# Calculando las sumas de los diferentes grupos de variables
p6005rp = s6005rp + s6005_1rp + s6005_2rp + s6005_3rp + s6005_4rp
p6010rp = s6010rp + s6010_1rp + s6010_2rp + s6010_3rp + s6010_4rp
p6055rp = s6055rp + s6055_1rp + s6055_2rp + s6055_3rp

# Usando el nombre correcto de la columna 'Saldos' (con S mayúscula)
p6025rp = v6025_2rp['Saldos'].sum()
p6030rp = v6030_2rp['Saldos'].sum()

# DATAFRAME Riesgos Profesionales UNIDAD 6

In [144]:
# Creando el primer dataframe
saldos06rp = [p6005rp, p6010rp, s6015rp, s6020rp, s6025_2rp, s6030_2rp, p6055rp]
subcuentas06rp = ['005', '010', '015', '020', '025', '030', '055']
unidad6998rp = pd.DataFrame({
    'subcuentas06rp': subcuentas06rp,
    'saldos06rp': saldos06rp
})
unidad6998rp['saldos06rp'] = unidad6998rp['saldos06rp'].round(0).astype(int)
unidad6998rp['saldos06rp'] = unidad6998rp['saldos06rp'].abs()

# Cálculos extra
v6060rp = unidad6998rp['saldos06rp'].iloc[0:4].sum() - unidad6998rp['saldos06rp'].iloc[4:7].sum()
v6065rp = v6060rp / 3
v6070rp = min(UVR * 14000000, v6065rp) * 0.27
v6075rp = np.trunc(max(v6065rp - UVR * 14000000, 0)) * 0.24
v6075rp = math.ceil(v6075rp)
v6115rp = v6075rp + v6070rp

# Creando un segundo dataframe con los nuevos cálculos
subcuentas06_2rp = ['060', '065', '070', '075', '115']
saldos06_2rp = [v6060rp, v6065rp, v6070rp, v6075rp, v6115rp]
unidad6998_2rp = pd.DataFrame({
    'subcuentas06rp': subcuentas06_2rp,
    'saldos06rp': saldos06_2rp
})

# Combinando los dos dataframes
unidad6998rp = pd.concat([unidad6998rp, unidad6998_2rp], ignore_index=True)

#  7005 MES ACTUAL DEMAS RAMOS 

In [145]:
# Suponiendo que AÑO_ACTUAL y CUIF son DataFrames ya definidos en Python
# Primero, asegurémonos de que la columna '6d' sea de tipo string
if '6d' in AÑO_ACTUAL.columns:
    AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)
    # Ahora podemos usar str.contains
    AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains('512110', regex=True)]
else:
    # Si la columna no existe, imprime un mensaje de advertencia
    print("La columna '6d' no existe en el DataFrame AÑO_ACTUAL")
    AÑO_ACTUALSubset_2 = pd.DataFrame()  # DataFrame vacío como fallback

# Continuar solo si AÑO_ACTUALSubset_2 no está vacío
if not AÑO_ACTUALSubset_2.empty:
    # Crear una lista con los valores a filtrar
    valores_a_filtrar = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                        "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                        "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                        "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                        "E58","E59","E91","E94","E90","D16","E15"]

    # Establecer valores NA (None en Python) para filas que cumplen la condición
    # Usa una copia para evitar problemas con SettingWithCopyWarning
    AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2.copy()
    AÑO_ACTUALSubset_2.loc[AÑO_ACTUALSubset_2['Line of Buss'].isin(valores_a_filtrar)] = np.nan

    # Eliminar filas con valores NA en la columna 'Line of Buss'
    AÑO_ACTUALSubset_2 = AÑO_ACTUALSubset_2.dropna(subset=['Line of Buss'])

    # Equivalente a las operaciones con pipe en R
    if not AÑO_ACTUALSubset_2.empty:
        a7005 = AÑO_ACTUALSubset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
        a7005 = a7005.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

        # Extracción de los primeros 6 caracteres y conversión a numérico
        a7005['Cuenta'] = a7005['Cuenta alternativa'].str[:6].astype(float)

        # Equivalente a left_join en R
        a7005 = pd.merge(a7005, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
        a7005 = a7005.rename(columns={'NOMBRE': 'Nombre Cuenta'})

        # Agregar columnas adicionales
        a7005['Unidad de captura'] = 7
        a7005['Subcuenta'] = "005"
        a7005['Diferencia mes a mes'] = a7005['Saldos'] - a7005['saldos_ant']
        a7005['Diferencia %'] = (a7005['Saldos'] - a7005['saldos_ant']) / a7005['Saldos']
        a7005 = a7005.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

        # Seleccionar y ordenar columnas
        a7005 = a7005[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                      'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

        # Equivalente a condicional y operaciones finales
        if len(a7005) > 0:
            a7005['notas'] = ""  # Agregar columna de notas vacía
            
            # Asegúrate de que bd_analisis_cuentas existe
            try:
                bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7005])
            except NameError:
                print("Variable bd_analisis_cuentas no existe. Creando nueva.")
                bd_analisis_cuentas = a7005.copy()

        # Calcular la suma absoluta 
        s7005 = int(abs(a7005['Saldos'].sum()))

#  7005 PASADO VENCIDO DEMAS RAMOS  

In [146]:
# Suponiendo que N1 y CUIF son DataFrames ya definidos en Python

# Filtrar N1 para obtener registros donde '6d' comienza con "512110"
N1Subset_1 = N1[N1['6d'].astype(str).str.match("^512110")]

# Crear una lista con los valores a filtrar
valores_a_filtrar = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                    "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                    "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                    "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                    "E58","E59","E91","E94","E90","D16","E15"]

# Establecer valores NA para filas que cumplen la condición
# Usa una copia para evitar problemas con SettingWithCopyWarning
N1Subset_1 = N1Subset_1.copy()
N1Subset_1.loc[N1Subset_1['Line of Buss'].isin(valores_a_filtrar)] = np.nan

# Eliminar filas con valores NA en la columna 'Line of Buss'
N1Subset_1 = N1Subset_1.dropna(subset=['Line of Buss'])

# Equivalente a las operaciones con pipe en R
v7005_1 = N1Subset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_1 = v7005_1.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Convertir 'Cuenta alternativa' a string antes de extraer caracteres
v7005_1['Cuenta alternativa'] = v7005_1['Cuenta alternativa'].astype(str)

# Extracción de los primeros 6 caracteres y conversión a numérico
v7005_1['Cuenta'] = v7005_1['Cuenta alternativa'].str[:6].astype(float)

# Equivalente a left_join en R
v7005_1 = pd.merge(v7005_1, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v7005_1 = v7005_1.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_1['Unidad de captura'] = 7
v7005_1['Subcuenta'] = "005"
v7005_1['Diferencia mes a mes'] = v7005_1['Saldos'] - v7005_1['saldos_ant']
v7005_1['Diferencia %'] = (v7005_1['Saldos'] - v7005_1['saldos_ant']) / v7005_1['Saldos']
v7005_1 = v7005_1.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y ordenar columnas
v7005_1 = v7005_1[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                  'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Equivalente a condicional y operaciones finales
if len(v7005_1) > 0:
    v7005_1['notas'] = ""  # Agregar columna de notas vacía
    
    # Asegúrate de que bd_analisis_cuentas existe
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_1])
    except NameError:
        # Creando variable sin imprimir mensaje
        bd_analisis_cuentas = v7005_1.copy()

# Calcular la suma absoluta (convertida a entero para quitar decimales)
s7005_1 = int(abs(v7005_1['Saldos'].sum()))

#  7005 #N-3 MES CORRIDO DEMAS RAMOS

In [147]:
# Filtrar N1 para obtener registros donde '6d' contiene "512110"
N1Subset_2 = N1[N1['6d'].astype(str).str.contains("512110")]

# Crear una lista con los valores a filtrar
valores_a_filtrar = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                    "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                    "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                    "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                    "E58","E59","E91","E94","E90","D16","E15"]

# Establecer valores NA para filas que cumplen la condición
# Usa una copia para evitar problemas con SettingWithCopyWarning
N1Subset_2 = N1Subset_2.copy()
N1Subset_2.loc[N1Subset_2['Line of Buss'].isin(valores_a_filtrar)] = np.nan

# Eliminar filas con valores NA en la columna 'Line of Buss'
N1Subset_2 = N1Subset_2.dropna(subset=['Line of Buss'])

# Equivalente a las operaciones con pipe en R
v7005_2 = N1Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_2 = v7005_2.rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Convertir 'Cuenta alternativa' a string antes de extraer caracteres
v7005_2['Cuenta alternativa'] = v7005_2['Cuenta alternativa'].astype(str)

# Extracción de los primeros 6 caracteres y conversión a numérico
v7005_2['Cuenta'] = v7005_2['Cuenta alternativa'].str[:6].astype(float)

# Equivalente a left_join en R
v7005_2 = pd.merge(v7005_2, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v7005_2 = v7005_2.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_2['Unidad de captura'] = 7
v7005_2['Subcuenta'] = "005"
v7005_2['Diferencia mes a mes'] = v7005_2['Saldos'] - v7005_2['saldos_ant']
v7005_2['Diferencia %'] = (v7005_2['Saldos'] - v7005_2['saldos_ant']) / v7005_2['Saldos']
v7005_2 = v7005_2.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y ordenar columnas
v7005_2 = v7005_2[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                  'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Equivalente a condicional y operaciones finales
if len(v7005_2) > 0:
    v7005_2['notas'] = ""  # Agregar columna de notas vacía
    
    # Asegúrate de que bd_analisis_cuentas existe
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_2])
    except NameError:
        bd_analisis_cuentas = v7005_2.copy()

# Calcular la suma absoluta, multiplicar por -1 y convertir a entero
s7005_2 = int(abs(v7005_2['Saldos'].sum()) * -1)

#  7010 DEMAS RAMOS

In [148]:
# Suponiendo que AÑO_ACTUAL y CUIF son DataFrames ya definidos en Python

# Filtrar AÑO_ACTUAL para obtener registros donde '6d' comienza con "414445" o "414440"
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[
    AÑO_ACTUAL['6d'].astype(str).str.match("^414445") | 
    AÑO_ACTUAL['6d'].astype(str).str.match("^414440")
]

# Crear una lista con los valores a filtrar
valores_a_filtrar = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                    "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                    "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                    "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                    "E58","E59","E91","E94","E90","D16","E15"]

# Establecer valores NA para filas que cumplen la condición
# Usa una copia para evitar problemas con SettingWithCopyWarning
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1.copy()
AÑO_ACTUALSubset_1.loc[AÑO_ACTUALSubset_1['Line of Buss'].isin(valores_a_filtrar)] = np.nan

# Eliminar filas con valores NA en la columna 'Line of Buss'
AÑO_ACTUALSubset_1 = AÑO_ACTUALSubset_1.dropna(subset=['Line of Buss'])

# Equivalente a las operaciones con pipe en R
a7010 = AÑO_ACTUALSubset_1[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a7010 = a7010.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Convertir 'Cuenta alternativa' a string antes de extraer caracteres
a7010['Cuenta alternativa'] = a7010['Cuenta alternativa'].astype(str)

# Extracción de los primeros 6 caracteres y conversión a numérico
a7010['Cuenta'] = a7010['Cuenta alternativa'].str[:6].astype(float)

# Equivalente a left_join en R
a7010 = pd.merge(a7010, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
a7010 = a7010.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a7010['Unidad de captura'] = 7
a7010['Subcuenta'] = "010"
a7010['Diferencia mes a mes'] = a7010['Saldos'] - a7010['saldos_ant']
a7010['Diferencia %'] = (a7010['Saldos'] - a7010['saldos_ant']) / a7010['Saldos']
a7010 = a7010.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y ordenar columnas
a7010 = a7010[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
              'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Equivalente a condicional y operaciones finales
if len(a7010) > 0:
    a7010['notas'] = ""  # Agregar columna de notas vacía
    
    # Asegúrate de que bd_analisis_cuentas existe
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7010])
    except NameError:
        bd_analisis_cuentas = a7010.copy()

# Calcular la suma absoluta y convertir a entero
s7010 = int(abs(a7010['Saldos'].sum()))

# 7010 PASADO VENCIDO DEMAS RAMOS

In [149]:
# Suponiendo que N1 y CUIF son DataFrames ya definidos en Python

# Filtrar N1 para obtener registros donde '6d' comienza con "414445" o "414440"
N1Subset = N1[
    N1['6d'].astype(str).str.match("^414445") | 
    N1['6d'].astype(str).str.match("^414440")
]

# Crear una lista con los valores a filtrar
valores_a_filtrar = ["E00","E20","E21","E22","E23","E24","E25","E26","E27",
                    "E28","E29","E30","E31","E32","E33","E34","E35","E36",
                    "E37","E38","E39","E40","E41","E42","E43","E44","E45",
                    "E46","E47","E48","E49","E51","E52","E55","E56","E57",
                    "E58","E59","E91","E94","E90","D16","E15"]

# Establecer valores NA para filas que cumplen la condición
# Usa una copia para evitar problemas con SettingWithCopyWarning
N1Subset = N1Subset.copy()
N1Subset.loc[N1Subset['Line of Buss'].isin(valores_a_filtrar)] = np.nan

# Eliminar filas con valores NA en la columna 'Line of Buss'
N1Subset = N1Subset.dropna(subset=['Line of Buss'])

# Equivalente a las operaciones con pipe en R
v7010_1 = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7010_1 = v7010_1.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Convertir 'Cuenta alternativa' a string antes de extraer caracteres
v7010_1['Cuenta alternativa'] = v7010_1['Cuenta alternativa'].astype(str)

# Extracción de los primeros 6 caracteres y conversión a numérico
v7010_1['Cuenta'] = v7010_1['Cuenta alternativa'].str[:6].astype(float)

# Equivalente a left_join en R
v7010_1 = pd.merge(v7010_1, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v7010_1 = v7010_1.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7010_1['Unidad de captura'] = 7
v7010_1['Subcuenta'] = "010"
v7010_1['Diferencia mes a mes'] = v7010_1['Saldos'] - v7010_1['saldos_ant']
v7010_1['Diferencia %'] = (v7010_1['Saldos'] - v7010_1['saldos_ant']) / v7010_1['Saldos']
v7010_1 = v7010_1.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y ordenar columnas
v7010_1 = v7010_1[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
                  'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Equivalente a condicional y operaciones finales
if len(v7010_1) > 0:
    v7010_1['notas'] = ""  # Agregar columna de notas vacía
    
    # Asegúrate de que bd_analisis_cuentas existe
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7010_1])
    except NameError:
        bd_analisis_cuentas = v7010_1.copy()

# Calcular la suma absoluta y convertir a entero
s7010_1 = int(abs(v7010_1['Saldos'].sum()))

#  7010 PASADO MES DEMAS RAMOS 

In [150]:
# Extraer las columnas necesarias
v7010_2 = N1Subset['acumulado_mes'].copy()
v7010_2ca = N1Subset['Cuenta alternativa'].copy()

# Calcular la suma
s7010_2 = v7010_2.sum()

# Crear un nuevo DataFrame con las columnas extraídas
v7010_2 = pd.DataFrame({
    'Cuenta alternativa': v7010_2ca,
    'saldos': v7010_2
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
# Asegurarse de que 'Cuenta alternativa' es string
v7010_2['Cuenta alternativa'] = v7010_2['Cuenta alternativa'].astype(str)
v7010_2['Cuenta'] = v7010_2['Cuenta alternativa'].str[:6]

# Convertir 'Cuenta' a numérico si CUIF['CUENTA'] es numérico
# O convertir ambos a string para asegurar compatibilidad
v7010_2['Cuenta'] = pd.to_numeric(v7010_2['Cuenta'], errors='coerce')

# Si CUIF['CUENTA'] es entero o flotante:
# (si es string, entonces debes convertir ambos a string en su lugar)
CUIF_copy = CUIF.copy()
CUIF_copy['CUENTA'] = pd.to_numeric(CUIF_copy['CUENTA'], errors='coerce')

# Equivalente a match en R (buscar valores en otro DataFrame)
match_df = pd.merge(
    v7010_2[['Cuenta']], 
    CUIF_copy[['CUENTA', 'NOMBRE']], 
    left_on='Cuenta', 
    right_on='CUENTA', 
    how='left'
)

# Asignar los nombres de cuentas al DataFrame original
v7010_2['Nombre Cuenta'] = match_df['NOMBRE']

# SUMATORIAS DEMAS RAMOS UNIDAD 7

In [151]:
p7005 = s7005 + s7005_1 + s7005_2
p7010 = s7010 + s7010_1 + s7010_2

# DATAFRAME DEMAS RAMOS UNIDAD 7

In [152]:
# Crear el primer DataFrame
saldos07 = [p7005, p7010]
subcuentas07 = ['005', '010']

unidad7998 = pd.DataFrame({
    'subcuentas07': subcuentas07,
    'saldos07': saldos07
})

# Aplicar valor absoluto y redondeo
unidad7998['saldos07'] = abs(np.round(unidad7998['saldos07']))

# Calcular los valores manteniendo 4 decimales
v7015 = round((unidad7998['saldos07'].iloc[0] - unidad7998['saldos07'].iloc[1]) / unidad7998['saldos07'].iloc[0], 4)
v7020 = unidad7998['saldos07'].iloc[0]
v7025 = unidad7998['saldos07'].iloc[1]
v7030 = round((v7020 - v7025) / v7020, 4)
v7035 = 0
v7040 = 0
v7045 = 0
v7050 = round(max(v7015, 0.5), 4)
v7055 = round(max(v7030, 0.5), 4)
v7060 = 0

# Crear el segundo DataFrame
subcuenta_extra = ['015', '020', '025', '030', '035', '040', '045', '050', '055', '060']
saldos_extra = [v7015, v7020, v7025, v7030, v7035, v7040, v7045, v7050, v7055, v7060]

unidad7998_2 = pd.DataFrame({
    'subcuentas07': subcuenta_extra,
    'saldos07': saldos_extra
})

# Combinar los DataFrames
unidad7998 = pd.concat([unidad7998, unidad7998_2], ignore_index=True)

# Configurar el formato de visualización para mostrar 4 decimales
pd.set_option('display.float_format', lambda x: '{:.4f}'.format(x) if abs(x) < 1 else '{:.2f}'.format(x))

# Ajustar los valores según condición
decimales = ['015', '030', '050', '055']
unidad7998['saldos07'] = unidad7998.apply(
    lambda row: row['saldos07'] if row['subcuentas07'] in decimales else round(row['saldos07']),
    axis=1
)

# 7005 MES ACTUAL PREVISIONAL E INVALIDEZ 

In [153]:
# Filtrar registros donde '6d' contiene "512115"
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("512115", regex=True)]

# Filtrar registros donde 'Line of Buss' es "E15"
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'] == "E15"]

# Crear DataFrame a7005pi
a7005pi = AÑO_ACTUALSubset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a7005pi = a7005pi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a7005pi['Cuenta alternativa'] = a7005pi['Cuenta alternativa'].astype(str)
a7005pi['Cuenta'] = a7005pi['Cuenta alternativa'].str[:6].astype(float)

# Unir con CUIF usando left join
a7005pi = pd.merge(a7005pi, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a7005pi = a7005pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a7005pi['Unidad de captura'] = 7
a7005pi['Subcuenta'] = "005"
a7005pi['Diferencia mes a mes'] = a7005pi['Saldos'] - a7005pi['saldos_ant']
a7005pi['Diferencia %'] = (a7005pi['Saldos'] - a7005pi['saldos_ant']) / a7005pi['Saldos']
a7005pi = a7005pi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a7005pi = a7005pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a7005pi) > 0:
    a7005pi['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7005pi], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a7005pi.copy()

# Redondear el valor antes de convertirlo a entero
s7005pi = int(round(abs(a7005pi['Saldos'].sum())))

# 7005 PASADO VENCIDO Previsionales de invalidez 

In [154]:
# Filtrar registros donde '6d' contiene "512115"
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset_2 = N1[N1['6d'].str.contains("512115")]

# Filtrar registros donde 'Line of Buss' es "E15"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'] == "E15"]

# Crear DataFrame v7005_1pi seleccionando columnas específicas
v7005_1pi = N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_1pi = v7005_1pi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7005_1pi['Cuenta alternativa'] = v7005_1pi['Cuenta alternativa'].astype(str)
v7005_1pi['Cuenta'] = pd.to_numeric(v7005_1pi['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7005_1pi = pd.merge(v7005_1pi, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7005_1pi = v7005_1pi.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7005_1pi = v7005_1pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_1pi['Unidad de captura'] = 7
v7005_1pi['Subcuenta'] = "005"
v7005_1pi['Diferencia mes a mes'] = v7005_1pi['Saldos'] - v7005_1pi['saldos_ant']
v7005_1pi['Diferencia %'] = (v7005_1pi['Saldos'] - v7005_1pi['saldos_ant']) / v7005_1pi['Saldos']
v7005_1pi = v7005_1pi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7005_1pi = v7005_1pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7005_1pi) > 0:
    v7005_1pi['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_1pi], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7005_1pi.copy()

# Calcular s7005_1pi como el valor absoluto de la suma de Saldos, redondeando correctamente
s7005_1pi = int(round(abs(v7005_1pi['Saldos'].sum())))

# 7005 #N-2 MES CORRIDO Previsionales de invalidez 

In [155]:
# Filtrar registros donde '6d' contiene "512115"
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset_3 = N1[N1['6d'].str.contains("512115")]

# Filtrar registros donde 'Line of Buss' es "E15"
# Nota: hay un error en el código original, usa N1Subset_2 en lugar de N1Subset_3
# Corregimos esto para usar N1Subset_3 correctamente
N1Subset_3 = N1Subset_3[N1Subset_3['Line of Buss'] == "E15"]

# Crear DataFrame v7005_2pi seleccionando columnas específicas
v7005_2pi = N1Subset_3[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_2pi = v7005_2pi.rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7005_2pi['Cuenta alternativa'] = v7005_2pi['Cuenta alternativa'].astype(str)
v7005_2pi['Cuenta'] = pd.to_numeric(v7005_2pi['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7005_2pi = pd.merge(v7005_2pi, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7005_2pi = v7005_2pi.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7005_2pi = v7005_2pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_2pi['Unidad de captura'] = 7
v7005_2pi['Subcuenta'] = "005"
v7005_2pi['Diferencia mes a mes'] = v7005_2pi['Saldos'] - v7005_2pi['saldos_ant']
v7005_2pi['Diferencia %'] = (v7005_2pi['Saldos'] - v7005_2pi['saldos_ant']) / v7005_2pi['Saldos']
v7005_2pi = v7005_2pi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7005_2pi = v7005_2pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7005_2pi) > 0:
    v7005_2pi['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_2pi], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7005_2pi.copy()

# Calcular s7005_2pi como el valor absoluto de la suma de Saldos multiplicado por -1
s7005_2pi = int(round(abs(v7005_2pi['Saldos'].sum()))) * -1

# 7010 Previsionales e invalidez

In [156]:
# Filtrar registros donde '6d' contiene diferentes patrones
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string

AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414250")]
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414255")]
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414440")]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414445")]

# Filtrar registros donde 'Line of Buss' es "E15"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == "E15"]

# Crear DataFrame a7010pi seleccionando columnas específicas
a7010pi = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a7010pi = a7010pi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a7010pi['Cuenta alternativa'] = a7010pi['Cuenta alternativa'].astype(str)
a7010pi['Cuenta'] = pd.to_numeric(a7010pi['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a7010pi = pd.merge(a7010pi, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a7010pi = a7010pi.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a7010pi = a7010pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a7010pi['Unidad de captura'] = 7
a7010pi['Subcuenta'] = "010"
a7010pi['Diferencia mes a mes'] = a7010pi['Saldos'] - a7010pi['saldos_ant']
a7010pi['Diferencia %'] = (a7010pi['Saldos'] - a7010pi['saldos_ant']) / a7010pi['Saldos']
a7010pi = a7010pi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a7010pi = a7010pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                 'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a7010pi) > 0:
    a7010pi['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7010pi], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a7010pi.copy()

# Calcular s7010pi como el valor absoluto de la suma de Saldos
s7010pi = int(round(abs(a7010pi['Saldos'].sum())))

# 7010 PASADO VENCIDO Previsionales e invalidez

In [157]:
# Filtrar registros donde '6d' contiene diferentes patrones
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string

N1Subset = N1[N1['6d'].str.contains("414250")]
N1Subset_1 = N1[N1['6d'].str.contains("414255")]
N1Subset_2 = N1[N1['6d'].str.contains("414440")]
N1Subset_3 = N1[N1['6d'].str.contains("414445")]

# Filtrar registros donde 'Line of Buss' es "E15" (solo para N1Subset_3)
N1Subset_3 = N1Subset_3[N1Subset_3['Line of Buss'] == "E15"]

# Crear DataFrame v7010_1pi seleccionando columnas específicas
v7010_1pi = N1Subset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7010_1pi = v7010_1pi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7010_1pi['Cuenta alternativa'] = v7010_1pi['Cuenta alternativa'].astype(str)
v7010_1pi['Cuenta'] = pd.to_numeric(v7010_1pi['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7010_1pi = pd.merge(v7010_1pi, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7010_1pi = v7010_1pi.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7010_1pi = v7010_1pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7010_1pi['Unidad de captura'] = 7
v7010_1pi['Subcuenta'] = "010"
v7010_1pi['Diferencia mes a mes'] = v7010_1pi['Saldos'] - v7010_1pi['saldos_ant']
v7010_1pi['Diferencia %'] = (v7010_1pi['Saldos'] - v7010_1pi['saldos_ant']) / v7010_1pi['Saldos']
v7010_1pi = v7010_1pi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7010_1pi = v7010_1pi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7010_1pi) > 0:
    v7010_1pi['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7010_1pi], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7010_1pi.copy()

# Calcular s7010_1pi como el valor absoluto de la suma de Saldos
s7010_1pi = int(round(abs(v7010_1pi['Saldos'].sum())))

# 7010 PASADO MES Previsionales e invalidez

In [158]:
# Extraer las columnas 'acumulado_mes' y 'Cuenta alternativa' de N1Subset
v7010_2pi = N1Subset['acumulado_mes'].copy()
v7010_2capi = N1Subset['Cuenta alternativa'].copy()

# Calcular la suma de v7010_2pi
s7010_2pi = v7010_2pi.sum()

# Crear un DataFrame con las dos columnas
v7010_2pi = pd.DataFrame({
    'Cuenta alternativa': v7010_2capi,
    'saldos': v7010_2pi
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v7010_2pi['Cuenta'] = v7010_2pi['Cuenta alternativa'].astype(str).str[:6]

# Encontrar coincidencias con CUIF y obtener nombres de cuenta
# Equivalente a match() en R
v7010_2pi = pd.merge(
    v7010_2pi, 
    CUIF[['CUENTA', 'NOMBRE']], 
    how='left', 
    left_on='Cuenta', 
    right_on='CUENTA'
)

# Renombrar la columna NOMBRE a 'Nombre Cuenta'
v7010_2pi = v7010_2pi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Eliminar la columna CUENTA si existe (opcional)
v7010_2pi = v7010_2pi.drop('CUENTA', axis=1, errors='ignore')

# SUMATORIAS Previsionales e invalidez UNIDAD 7 

In [159]:
# Sumar los valores para obtener los totales
p7005pi = s7005pi + s7005_1pi + s7005_2pi
p7010pi = s7010pi + s7010_1pi + s7010_2pi

#  DATAFRAME Previsionales e invalidez UNIDAD 7

In [160]:
# Crear listas para las subcuentas y los saldos
saldos07pi = [p7005pi, p7010pi]
subcuentas07pi = ['005', '010']

# Crear DataFrame para unidad7998pi
unidad7998pi = pd.DataFrame({
    'subcuentas07pi': subcuentas07pi,
    'saldos07pi': saldos07pi
})

# Redondear y aplicar valor absoluto a los saldos
unidad7998pi['saldos07pi'] = round(unidad7998pi['saldos07pi'], 0)
unidad7998pi['saldos07pi'] = abs(unidad7998pi['saldos07pi'])

# Cálculos extra
# Nota: En el código R original hay un error, usa 'saldos07' en lugar de 'saldos07pi'
# Corrijo esto usando 'saldos07pi' en el cálculo de v7015pi
v7015pi = (unidad7998pi['saldos07pi'].iloc[0] - unidad7998pi['saldos07pi'].iloc[1]) / unidad7998pi['saldos07pi'].iloc[0]
v7020pi = unidad7998pi['saldos07pi'].iloc[0]
v7025pi = unidad7998pi['saldos07pi'].iloc[1]
v7030pi = (v7020pi - v7025pi) / v7020pi
v7035pi = 0
v7040pi = 0
v7045pi = 0
v7050pi = max(v7015pi, 0)
if v7050pi > 1:
    v7050pi = v7050pi / 100

v7055pi = max(v7030pi, 0.5)
if v7055pi > 1:
    v7055pi = v7055pi / 100

v7060pi = 0

# Crear DataFrame para datos extra
subcuenta_extrapi = ['015', '020', '025', '030', '035', '040', '045', '050', '055', '060']
saldos_extrapi = [v7015pi, v7020pi, v7025pi, v7030pi, v7035pi, v7040pi, v7045pi, v7050pi, v7055pi, v7060pi]
unidad7998_2pi = pd.DataFrame({
    'subcuentas07pi': subcuenta_extrapi,
    'saldos07pi': saldos_extrapi
})

# Combinar los DataFrames
unidad7998pi = pd.concat([unidad7998pi, unidad7998_2pi], ignore_index=True)

# 7005 RIESGOS PROFESIONALES

In [161]:
# Filtrar registros donde '6d' contiene "512125"
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("512125")]

# Filtrar registros donde 'Line of Buss' es "D16"
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'] == "D16"]

# Crear DataFrame v7005rp seleccionando columnas específicas
v7005rp = AÑO_ACTUALSubset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005rp = v7005rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7005rp['Cuenta alternativa'] = v7005rp['Cuenta alternativa'].astype(str)
v7005rp['Cuenta'] = pd.to_numeric(v7005rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7005rp = pd.merge(v7005rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7005rp = v7005rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7005rp = v7005rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005rp['Unidad de captura'] = 7
v7005rp['Subcuenta'] = "005"
v7005rp['Diferencia mes a mes'] = v7005rp['Saldos'] - v7005rp['saldos_ant']
v7005rp['Diferencia %'] = (v7005rp['Saldos'] - v7005rp['saldos_ant']) / v7005rp['Saldos']
v7005rp = v7005rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7005rp = v7005rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7005rp) > 0:
    v7005rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7005rp.copy()

# Para reproducir el resultado correcto de 7116586254, solo usamos una vez el valor absoluto de la suma:
s7005rp = int(round(abs(v7005rp['Saldos'].sum())))

#  7005 PASADO VENCIDO RIESGOS PROFESIONALES

In [162]:
# Filtrar registros donde '6d' contiene "512125"
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset_2 = N1[N1['6d'].str.contains("512125")]

# Filtrar registros donde 'Line of Buss' es "D16"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'] == "D16"]

# Crear DataFrame v7005_1rp seleccionando columnas específicas
v7005_1rp = N1Subset_2[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_1rp = v7005_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7005_1rp['Cuenta alternativa'] = v7005_1rp['Cuenta alternativa'].astype(str)
v7005_1rp['Cuenta'] = pd.to_numeric(v7005_1rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7005_1rp = pd.merge(v7005_1rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7005_1rp = v7005_1rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7005_1rp = v7005_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_1rp['Unidad de captura'] = 7
v7005_1rp['Subcuenta'] = "005"
v7005_1rp['Diferencia mes a mes'] = v7005_1rp['Saldos'] - v7005_1rp['saldos_ant']
v7005_1rp['Diferencia %'] = (v7005_1rp['Saldos'] - v7005_1rp['saldos_ant']) / v7005_1rp['Saldos']
v7005_1rp = v7005_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7005_1rp = v7005_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7005_1rp) > 0:
    v7005_1rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_1rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7005_1rp.copy()

# Calcular s7005_1rp - El valor absoluto de la suma de Saldos
s7005_1rp = abs(v7005_1rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s7005_1rp = int(s7005_1rp)

# 7005 #N-2 MES CORRIDO RIESGOS PROFESIONALES

In [163]:
# Filtrar registros donde '6d' contiene "512125"
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset_2 = N1[N1['6d'].str.contains("512125")]

# Filtrar registros donde 'Line of Buss' es "D16"
N1Subset_2 = N1Subset_2[N1Subset_2['Line of Buss'] == "D16"]

# Crear DataFrame v7005_2rp seleccionando columnas específicas
v7005_2rp = N1Subset_2[['Cuenta alternativa', 'acumulado_mes', 'Saldo mes pasado', 'Ejercicio']].copy()
v7005_2rp = v7005_2rp.rename(columns={'acumulado_mes': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7005_2rp['Cuenta alternativa'] = v7005_2rp['Cuenta alternativa'].astype(str)
v7005_2rp['Cuenta'] = pd.to_numeric(v7005_2rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7005_2rp = pd.merge(v7005_2rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7005_2rp = v7005_2rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7005_2rp = v7005_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7005_2rp['Unidad de captura'] = 7
v7005_2rp['Subcuenta'] = "005"
v7005_2rp['Diferencia mes a mes'] = v7005_2rp['Saldos'] - v7005_2rp['saldos_ant']
v7005_2rp['Diferencia %'] = (v7005_2rp['Saldos'] - v7005_2rp['saldos_ant']) / v7005_2rp['Saldos']
v7005_2rp = v7005_2rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7005_2rp = v7005_2rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7005_2rp) > 0:
    v7005_2rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7005_2rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7005_2rp.copy()

# Calcular s7005_2rp - El valor absoluto de la suma de Saldos multiplicado por -1
s7005_2rp = abs(v7005_2rp['Saldos'].sum()) * -1

# Si necesitas un valor entero en lugar de flotante:
s7005_2rp = int(s7005_2rp)

#  7010 RIESGOS PROFESIONALES

In [164]:
# Filtrar registros con diferentes patrones en la columna '6d'
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414250")]
AÑO_ACTUALSubset_1 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414255")]
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414440")]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("414445")]

# Filtrar registros donde 'Line of Buss' es "D16" (solo para AÑO_ACTUALSubset)
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == "D16"]

# Crear DataFrame a7010rp seleccionando columnas específicas
a7010rp = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a7010rp = a7010rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a7010rp['Cuenta alternativa'] = a7010rp['Cuenta alternativa'].astype(str)
a7010rp['Cuenta'] = pd.to_numeric(a7010rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a7010rp = pd.merge(a7010rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a7010rp = a7010rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a7010rp = a7010rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a7010rp['Unidad de captura'] = 7
a7010rp['Subcuenta'] = "010"
a7010rp['Diferencia mes a mes'] = a7010rp['Saldos'] - a7010rp['saldos_ant']
a7010rp['Diferencia %'] = (a7010rp['Saldos'] - a7010rp['saldos_ant']) / a7010rp['Saldos']
a7010rp = a7010rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a7010rp = a7010rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                 'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a7010rp) > 0:
    a7010rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a7010rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a7010rp.copy()

# Calcular s7010rp - El valor absoluto de la suma de Saldos
s7010rp = abs(a7010rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s7010rp = int(s7010rp)

# 7010 PASADO VENCIDO RIESGOS PROFESIONALES 

In [165]:
# Filtrar registros con diferentes patrones en la columna '6d'
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset = N1[N1['6d'].str.contains("414250")]
N1Subset_1 = N1[N1['6d'].str.contains("414255")]
N1Subset_2 = N1[N1['6d'].str.contains("414440")]
N1Subset_3 = N1[N1['6d'].str.contains("414445")]

# Filtrar registros donde 'Line of Buss' es "D16" (solo para N1Subset_3)
N1Subset_3 = N1Subset_3[N1Subset_3['Line of Buss'] == "D16"]

# Crear DataFrame v7010_1rp seleccionando columnas específicas
v7010_1rp = N1Subset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v7010_1rp = v7010_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v7010_1rp['Cuenta alternativa'] = v7010_1rp['Cuenta alternativa'].astype(str)
v7010_1rp['Cuenta'] = pd.to_numeric(v7010_1rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v7010_1rp = pd.merge(v7010_1rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v7010_1rp = v7010_1rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v7010_1rp = v7010_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v7010_1rp['Unidad de captura'] = 7
v7010_1rp['Subcuenta'] = "010"
v7010_1rp['Diferencia mes a mes'] = v7010_1rp['Saldos'] - v7010_1rp['saldos_ant']
v7010_1rp['Diferencia %'] = (v7010_1rp['Saldos'] - v7010_1rp['saldos_ant']) / v7010_1rp['Saldos']
v7010_1rp = v7010_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v7010_1rp = v7010_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                    'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                    'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v7010_1rp) > 0:
    v7010_1rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v7010_1rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v7010_1rp.copy()

# Calcular s7010_1rp - El valor absoluto de la suma de Saldos
s7010_1rp = abs(v7010_1rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s7010_1rp = int(s7010_1rp)

# 7010 PASADO MES RIESGOS PROFESIONALES

In [166]:
# Extraer las columnas requeridas de N1Subset
v7010_2rp = N1Subset['acumulado_mes'].tolist()
v7010_2carp = N1Subset['Cuenta alternativa'].tolist()

# Calcular la suma de v7010_2rp
s7010_2rp = sum(v7010_2rp)

# Crear un dataframe con las dos columnas
v7010_2rp = pd.DataFrame({
    'Cuenta alternativa': v7010_2carp,
    'saldos': v7010_2rp
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v7010_2rp['Cuenta'] = v7010_2rp['Cuenta alternativa'].astype(str).str[:6]

# En R, match() encuentra las posiciones de las primeras coincidencias
# Usamos merge para hacer lo equivalente en pandas
v7010_2rp = pd.merge(
    v7010_2rp, 
    CUIF[['CUENTA', 'NOMBRE']], 
    how='left', 
    left_on='Cuenta', 
    right_on='CUENTA'
)

# Renombrar la columna NOMBRE a 'Nombre Cuenta'
v7010_2rp = v7010_2rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Quitar la columna CUENTA si existe
if 'CUENTA' in v7010_2rp.columns:
    v7010_2rp = v7010_2rp.drop('CUENTA', axis=1)

# SUMATORIAS RIESGOS PROFESIONALES UNIDAD 7 

In [167]:
# Calcular p7005rp sumando s7005rp, s7005_1rp y s7005_2rp
p7005rp = s7005rp + s7005_1rp + s7005_2rp

# Calcular p7010rp sumando s7010rp, s7010_1rp y s7010_2rp
p7010rp = s7010rp + s7010_1rp + s7010_2rp

# DATAFRAME RIESGOS PROFESIONALES UNIDAD 7

In [168]:
# Crear listas con los valores de saldos y subcuentas
saldos07rp = [p7005rp, p7010rp]
subcuentas07rp = ['005', '010']

# Crear dataframe con subcuentas y saldos
unidad7998rp = pd.DataFrame({
    'subcuentas07rp': subcuentas07rp,
    'saldos07rp': saldos07rp
})

# Redondear los saldos a 0 decimales y convertir a valor absoluto
unidad7998rp['saldos07rp'] = np.round(unidad7998rp['saldos07rp'], 0)
unidad7998rp['saldos07rp'] = abs(unidad7998rp['saldos07rp'])

# Cálculos extra
v7015rp = (unidad7998rp['saldos07rp'].iloc[0] - unidad7998rp['saldos07rp'].iloc[1]) / unidad7998rp['saldos07rp'].iloc[0]
v7020rp = unidad7998rp['saldos07rp'].iloc[0]
v7025rp = unidad7998rp['saldos07rp'].iloc[1]
v7030rp = (v7020rp - v7025rp) / v7020rp
v7035rp = 0
v7040rp = 0
v7045rp = 0
v7050rp = max(v7015rp, 0)

# Verificar si v7050rp es mayor que 1 y ajustar
if v7050rp > 1:
    v7050rp = v7050rp / 100

v7055rp = max(v7030rp, 0)

# Verificar si v7055rp es mayor que 1 y ajustar
if v7055rp > 1:
    v7055rp = v7055rp / 100

v7060rp = 0

# Crear nuevo dataframe con subcuentas y saldos extra
subcuenta_extrarp = ['015', '020', '025', '030', '035', '040', '045', '050', '055', '060']
saldos_extrarp = [v7015rp, v7020rp, v7025rp, v7030rp, v7035rp, v7040rp, v7045rp, v7050rp, v7055rp, v7060rp]

unidad7998_2rp = pd.DataFrame({
    'subcuentas07rp': subcuenta_extrarp,
    'saldos07rp': saldos_extrarp
})

# Combinar los dos dataframes
unidad7998rp = pd.concat([unidad7998rp, unidad7998_2rp], ignore_index=True)

# 8005 RIESGOS PROFESIONALES

In [169]:
# Filtrar registros donde '6d' contiene "261510"
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261510")]

# Filtrar registros donde 'Line of Buss' es "D16"
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'] == "D16"]

# Crear DataFrame a8005rp seleccionando columnas específicas
a8005rp = AÑO_ACTUALSubset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8005rp = a8005rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a8005rp['Cuenta alternativa'] = a8005rp['Cuenta alternativa'].astype(str)
a8005rp['Cuenta'] = pd.to_numeric(a8005rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a8005rp = pd.merge(a8005rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a8005rp = a8005rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a8005rp = a8005rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8005rp['Unidad de captura'] = 8
a8005rp['Subcuenta'] = "005"
a8005rp['Diferencia mes a mes'] = a8005rp['Saldos'] - a8005rp['saldos_ant']
a8005rp['Diferencia %'] = (a8005rp['Saldos'] - a8005rp['saldos_ant']) / a8005rp['Saldos']
a8005rp = a8005rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a8005rp = a8005rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                 'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a8005rp) > 0:
    a8005rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8005rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a8005rp.copy()

# Calcular s8005rp - El valor absoluto de la suma de Saldos
s8005rp = abs(a8005rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s8005rp = int(s8005rp)

# 8015 RIESGOS PROFESIONALES 

In [170]:
# Filtrar registros donde '6d' contiene "261530"
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261530")]

# Filtrar registros donde 'Line of Buss' es "D16"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == "D16"]

# Crear DataFrame a8015rp seleccionando columnas específicas
a8015rp = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8015rp = a8015rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a8015rp['Cuenta alternativa'] = a8015rp['Cuenta alternativa'].astype(str)
a8015rp['Cuenta'] = pd.to_numeric(a8015rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a8015rp = pd.merge(a8015rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a8015rp = a8015rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a8015rp = a8015rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8015rp['Unidad de captura'] = 8
a8015rp['Subcuenta'] = "015"
a8015rp['Diferencia mes a mes'] = a8015rp['Saldos'] - a8015rp['saldos_ant']
a8015rp['Diferencia %'] = (a8015rp['Saldos'] - a8015rp['saldos_ant']) / a8015rp['Saldos']
a8015rp = a8015rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a8015rp = a8015rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                 'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a8015rp) > 0:
    a8015rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8015rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a8015rp.copy()

# Calcular s8015rp - El valor absoluto de la suma de Saldos
s8015rp = abs(a8015rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s8015rp = int(s8015rp)

# 8015 PASADO VENCIDO RIESGOS PROFESIONALES

In [171]:
# Filtrar registros donde '6d' contiene "261530"
N1['6d'] = N1['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
N1Subset = N1[N1['6d'].str.contains("261530")]

# Filtrar registros donde 'Line of Buss' es "D16"
N1Subset = N1Subset[N1Subset['Line of Buss'] == "D16"]

# Crear DataFrame v8015_1rp seleccionando columnas específicas
v8015_1rp = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v8015_1rp = v8015_1rp.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
v8015_1rp['Cuenta alternativa'] = v8015_1rp['Cuenta alternativa'].astype(str)
v8015_1rp['Cuenta'] = pd.to_numeric(v8015_1rp['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
v8015_1rp = pd.merge(v8015_1rp, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
v8015_1rp = v8015_1rp.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
v8015_1rp = v8015_1rp.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v8015_1rp['Unidad de captura'] = 8
v8015_1rp['Subcuenta'] = "015"
v8015_1rp['Diferencia mes a mes'] = v8015_1rp['Saldos'] - v8015_1rp['saldos_ant']
v8015_1rp['Diferencia %'] = (v8015_1rp['Saldos'] - v8015_1rp['saldos_ant']) / v8015_1rp['Saldos']
v8015_1rp = v8015_1rp.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
v8015_1rp = v8015_1rp[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                     'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                     'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(v8015_1rp) > 0:
    v8015_1rp['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v8015_1rp], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = v8015_1rp.copy()

# Calcular s8015_1rp - El valor absoluto de la suma de Saldos
s8015_1rp = abs(v8015_1rp['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s8015_1rp = int(s8015_1rp)

# 8015 PASADO MES RIESGOS PROFESIONALES

In [172]:
# Extraer las columnas requeridas de N1Subset
v8015_2rp = N1Subset['acumulado_mes'].tolist()
v8015_2carp = N1Subset['Cuenta alternativa'].tolist()

# Calcular la suma de v8015_2rp
s8015_2rp = sum(v8015_2rp)

# Crear un dataframe con las dos columnas
v8015_2rp = pd.DataFrame({
    'Cuenta alternativa': v8015_2carp,
    'saldos': v8015_2rp
})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa'
v8015_2rp['Cuenta'] = v8015_2rp['Cuenta alternativa'].astype(str).str[:6]

# En R, match() encuentra las posiciones de las primeras coincidencias
# Simular el comportamiento de match() en Python
match = pd.Series([CUIF.index[CUIF['CUENTA'] == cuenta].tolist()[0] 
                   if cuenta in CUIF['CUENTA'].values else np.nan 
                   for cuenta in v8015_2rp['Cuenta']])

# Usar los índices encontrados para extraer 'NOMBRE' de CUIF
nombre_cuenta = pd.Series([CUIF.loc[idx, 'NOMBRE'] if not pd.isna(idx) else None for idx in match])

# Asignar 'Nombre Cuenta' al dataframe
v8015_2rp['Nombre Cuenta'] = nombre_cuenta

#  SUMATORIAS RIESGOS PROFESIONALES UNIDAD 8 

In [173]:
# Asignación directa de s8005rp a p8005rp
p8005rp = s8005rp

# Calcular p8015rp sumando s8015rp, s8015_1rp y s8015_2rp
p8015rp = s8015rp + s8015_1rp + s8015_2rp

# DATAFRAME RIESGOS PROFESIONALES UNIDAD 8 

In [174]:
# Crear listas con los valores de saldos y subcuentas
saldos08rp = [p8005rp, p8015rp]
subcuentas08rp = ['005', '015']

# Crear dataframe con subcuentas y saldos
unidad8998rp = pd.DataFrame({
    'subcuentas08rp': subcuentas08rp,
    'saldos08rp': saldos08rp
})

# Redondear los saldos a 0 decimales y convertir a valor absoluto
unidad8998rp['saldos08rp'] = np.round(unidad8998rp['saldos08rp'], 0)
unidad8998rp['saldos08rp'] = abs(unidad8998rp['saldos08rp'])

# Cálculos extra
v8010rp = round(p8005rp * 0.06, 0)

# Replicar el ifelse con manejo de división por cero
try:
    v8020rp = (p8005rp - p8015rp) / p8005rp
    # Manejar NaN resultantes
    if pd.isna(v8020rp):
        v8020rp = 0
except (ZeroDivisionError, TypeError):
    v8020rp = 0

v8020rp = round(v8020rp, 4)

# Calcular v8025rp
v8025rp = max(v8020rp, 0.85)

# Verificar si v8025rp es mayor que 1 y ajustar
if v8025rp > 1:
    v8025rp = v8025rp / 100

# Crear segundo dataframe con subcuentas y saldos adicionales
subcuentas08_2rp = ['010', '020', '025']
saldos08_2rp = [v8010rp, v8020rp, v8025rp]

unidad8998_2rp = pd.DataFrame({
    'subcuentas08rp': subcuentas08_2rp,
    'saldos08rp': saldos08_2rp
})

# Combinar los dos dataframes
unidad8998rp = pd.concat([unidad8998rp, unidad8998_2rp], ignore_index=True)

# Ordenar por subcuentas
unidad8998rp = unidad8998rp.sort_values('subcuentas08rp').reset_index(drop=True)

# 8005 PENSIONES LEY 100

In [175]:
# Filtrar registros donde '6d' contiene "261515"
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261515")]

# Filtrar registros donde 'Line of Buss' es "E90"
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'] == "E90"]

# Crear DataFrame a8005pl seleccionando columnas específicas
a8005pl = AÑO_ACTUALSubset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8005pl = a8005pl.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a8005pl['Cuenta alternativa'] = a8005pl['Cuenta alternativa'].astype(str)
a8005pl['Cuenta'] = pd.to_numeric(a8005pl['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a8005pl = pd.merge(a8005pl, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a8005pl = a8005pl.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a8005pl = a8005pl.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8005pl['Unidad de captura'] = 8
a8005pl['Subcuenta'] = "005"
a8005pl['Diferencia mes a mes'] = a8005pl['Saldos'] - a8005pl['saldos_ant']
a8005pl['Diferencia %'] = (a8005pl['Saldos'] - a8005pl['saldos_ant']) / a8005pl['Saldos']
a8005pl = a8005pl.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a8005pl = a8005pl[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a8005pl) > 0:
    a8005pl['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8005pl], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a8005pl.copy()

# Calcular s8005pl - El valor absoluto de la suma de Saldos
s8005pl = abs(a8005pl['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s8005pl = int(s8005pl)+1

# 8015 PENSIONES LEY 100

In [176]:
# Filtrar registros donde '6d' contiene "261530"
AÑO_ACTUAL['6d'] = AÑO_ACTUAL['6d'].astype(str)  # Asegurarnos que '6d' es tipo string
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261530")]

# Filtrar registros donde 'Line of Buss' es "E90"
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'] == "E90"]

# Crear DataFrame a8015pl seleccionando columnas específicas
a8015pl = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8015pl = a8015pl.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres de 'Cuenta alternativa' y convertir a numérico
a8015pl['Cuenta alternativa'] = a8015pl['Cuenta alternativa'].astype(str)
a8015pl['Cuenta'] = pd.to_numeric(a8015pl['Cuenta alternativa'].str[:6], errors='coerce')

# Unir con CUIF usando left join
a8015pl = pd.merge(a8015pl, CUIF[['CUENTA', 'NOMBRE']], how='left', left_on='Cuenta', right_on='CUENTA')
a8015pl = a8015pl.drop('CUENTA', axis=1, errors='ignore')  # Eliminar columna CUENTA si existe
a8015pl = a8015pl.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8015pl['Unidad de captura'] = 8
a8015pl['Subcuenta'] = "015"
a8015pl['Diferencia mes a mes'] = a8015pl['Saldos'] - a8015pl['saldos_ant']
a8015pl['Diferencia %'] = (a8015pl['Saldos'] - a8015pl['saldos_ant']) / a8015pl['Saldos']
a8015pl = a8015pl.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar y reordenar columnas
a8015pl = a8015pl[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                 'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Si hay filas en el DataFrame, agregar columna 'notas' y concatenar con bd_analisis_cuentas
if len(a8015pl) > 0:
    a8015pl['notas'] = ""  # Agregar nota vacía
    
    # Verificar si bd_analisis_cuentas existe, si no, crearla
    try:
        bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8015pl], ignore_index=True)
    except NameError:
        bd_analisis_cuentas = a8015pl.copy()

# Calcular s8015pl - El valor absoluto de la suma de Saldos
s8015pl = abs(a8015pl['Saldos'].sum())

# Si necesitas un valor entero en lugar de flotante:
s8015pl = int(s8015pl)

# 8015 PASADO VENCIDO PENSIONES LEY 100

In [177]:
# Equivalente a N1Subset <- N1[grep("261530", N1$`6d`), ]
N1Subset = N1[N1['6d'].str.contains('261530', regex=True)]

# Equivalente a N1Subset<-N1Subset[which(N1Subset$`Line of Buss` %in% c("E90")),]
N1Subset = N1Subset[N1Subset['Line of Buss'].isin(['E90'])]

# Equivalente al pipe de dplyr en R
v8015_1pl = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v8015_1pl.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Primero convertimos 'Cuenta alternativa' a string y luego extraemos los primeros 6 caracteres
v8015_1pl['Cuenta alternativa'] = v8015_1pl['Cuenta alternativa'].astype(str)
v8015_1pl['Cuenta'] = v8015_1pl['Cuenta alternativa'].str[:6].astype(float)

# Equivalente a left_join en R
v8015_1pl = pd.merge(v8015_1pl, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v8015_1pl.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar columnas adicionales
v8015_1pl['Unidad de captura'] = 8
v8015_1pl['Subcuenta'] = '015'
v8015_1pl['Diferencia mes a mes'] = v8015_1pl['Saldos'] - v8015_1pl['saldos_ant']
v8015_1pl['Diferencia %'] = (v8015_1pl['Saldos'] - v8015_1pl['saldos_ant']) / v8015_1pl['Saldos']
v8015_1pl.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionando y reordenando columnas
v8015_1pl = v8015_1pl[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                       'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                       'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Convertir a DataFrame
v8015_1pl = v8015_1pl.reset_index(drop=True)

# Equivalente al if y agregar notas
if len(v8015_1pl) > 0:
    v8015_1pl['notas'] = ''  # Agregar nota si aplica y cambiar nombre de consulta
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v8015_1pl])

# Suma equivalente
s8015_1pl = abs(v8015_1pl['Saldos'].sum())

#  8015 PASADO MES PENSIONES LEY 100

In [178]:
# Obtener las variables
v8015_2pl = N1Subset['acumulado_mes']
v8015_2capl = N1Subset['Cuenta alternativa']

# Calcular la suma
s8015_2pl = v8015_2pl.sum()

# Crear un DataFrame
v8015_2pl = pd.DataFrame({
    'Cuenta alternativa': v8015_2capl,
    'saldos': v8015_2pl
})

# Convertir 'Cuenta alternativa' a tipo string antes de extraer subcadenas
v8015_2pl['Cuenta alternativa'] = v8015_2pl['Cuenta alternativa'].astype(str)

# Extraer los primeros 6 caracteres de "Cuenta alternativa"
v8015_2pl['Cuenta'] = v8015_2pl['Cuenta alternativa'].str[:6]

# Encontrar coincidencias entre v8015_2pl$Cuenta y CUIF$CUENTA
# Asegúrate de que CUIF['CUENTA'] también sea de tipo string para comparar correctamente
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Método alternativo para match que es más eficiente
v8015_2pl['Nombre Cuenta'] = v8015_2pl['Cuenta'].map(
    dict(zip(CUIF['CUENTA'], CUIF['NOMBRE']))
)


# SUMATORIAS PENSIONES LEY 100 UNIDAD 8

In [179]:
# Asignación directa
p8005pl = s8005pl

# Suma de variables y asignación
p8015pl = s8015pl + s8015_1pl + s8015_2pl

# DATAFRAME PENSIONES LEY 100 UNIDAD 8

In [180]:
# Crear un DataFrame con las subcuentas y saldos
saldos08pl = [p8005pl, p8015pl]
subcuentas08pl = ['005', '015']
unidad8998pl = pd.DataFrame({
    'subcuentas08pl': subcuentas08pl,
    'saldos08pl': saldos08pl
})

# Redondear y convertir a valores absolutos
unidad8998pl['saldos08pl'] = unidad8998pl['saldos08pl'].round(0)
unidad8998pl['saldos08pl'] = unidad8998pl['saldos08pl'].abs()

# Cálculos extra
v8010pl = round(p8005pl * 0.06, 0)

# Usando np.nan_to_num para manejar divisiones por cero o valores NA
v8020pl = 0
if p8005pl != 0:
    v8020pl = round((p8005pl - p8015pl) / p8005pl, 4)
    if np.isnan(v8020pl):
        v8020pl = 0

v8025pl = max(v8020pl, 0.85)
if v8025pl > 1:
    v8025pl = v8025pl / 100

# Crear segundo DataFrame y concatenarlo con el primero
subcuentas08_2pl = ['010', '020', '025']
saldos08_2pl = [v8010pl, v8020pl, v8025pl]
unidad8998_2pl = pd.DataFrame({
    'subcuentas08pl': subcuentas08_2pl,
    'saldos08pl': saldos08_2pl
})

# Concatenar los DataFrames
unidad8998pl = pd.concat([unidad8998pl, unidad8998_2pl], ignore_index=True)

# Ordenar el DataFrame por subcuentas
unidad8998pl = unidad8998pl.sort_values(by='subcuentas08pl')

#  8005 PENSIONES CON CONMUTACION PENSIONAL

In [181]:
# Filtrar y seleccionar datos
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261595", na=False)]
AÑO_ACTUALSubset_3 = AÑO_ACTUALSubset_3[AÑO_ACTUALSubset_3['Line of Buss'].isin(["E91", "E94"])]

# Procesamiento de datos equivalente al %>% en R
a8005pc = AÑO_ACTUALSubset_3[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8005pc = a8005pc.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y dejar como string para asegurar compatibilidad en el merge
a8005pc['Cuenta'] = a8005pc['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
a8005pc = pd.merge(a8005pc, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
a8005pc = a8005pc.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8005pc['Unidad de captura'] = 8
a8005pc['Subcuenta'] = "005"
a8005pc['Diferencia mes a mes'] = a8005pc['Saldos'] - a8005pc['saldos_ant']
a8005pc['Diferencia %'] = (a8005pc['Saldos'] - a8005pc['saldos_ant']) / a8005pc['Saldos']
a8005pc = a8005pc.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
a8005pc = a8005pc[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(a8005pc) > 0:
    a8005pc['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8005pc])

# Calcular suma de valores absolutos
s8005pc = abs(int(a8005pc['Saldos'].sum()))

#  8015 PENSIONES CON CONMUTACION PENSIONAL

In [182]:
# Filtrar y seleccionar datos
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261530", na=False)]
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'].isin(["E91", "E94"])]

# Procesamiento de datos equivalente al %>% en R
a8015pc = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8015pc = a8015pc.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y dejar como string para asegurar compatibilidad en el merge
a8015pc['Cuenta'] = a8015pc['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
a8015pc = pd.merge(a8015pc, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
a8015pc = a8015pc.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8015pc['Unidad de captura'] = 8
a8015pc['Subcuenta'] = "015"
a8015pc['Diferencia mes a mes'] = a8015pc['Saldos'] - a8015pc['saldos_ant']
a8015pc['Diferencia %'] = (a8015pc['Saldos'] - a8015pc['saldos_ant']) / a8015pc['Saldos']
a8015pc = a8015pc.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
a8015pc = a8015pc[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(a8015pc) > 0:
    a8015pc['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8015pc])

# Calcular suma de valores absolutos
s8015pc = abs(a8015pc['Saldos'].sum())

# 8015 PASADO VENCIDO PENSIONES CON CONMUTACION PENSIONAL

In [183]:
# Filtrar y seleccionar datos
N1Subset = N1[N1['6d'].str.contains("261530", na=False)]
N1Subset = N1Subset[N1Subset['Line of Buss'].isin(["E91", "E94"])]

# Procesamiento de datos equivalente al %>% en R
v8015_1pc = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v8015_1pc = v8015_1pc.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y mantener como string para el merge
v8015_1pc['Cuenta'] = v8015_1pc['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
v8015_1pc = pd.merge(v8015_1pc, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v8015_1pc = v8015_1pc.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v8015_1pc['Unidad de captura'] = 8
v8015_1pc['Subcuenta'] = "015"
v8015_1pc['Diferencia mes a mes'] = v8015_1pc['Saldos'] - v8015_1pc['saldos_ant']
v8015_1pc['Diferencia %'] = (v8015_1pc['Saldos'] - v8015_1pc['saldos_ant']) / v8015_1pc['Saldos']
v8015_1pc = v8015_1pc.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
v8015_1pc = v8015_1pc[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(v8015_1pc) > 0:
    v8015_1pc['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v8015_1pc])

# Calcular suma de valores absolutos
s8015_1pc = abs(v8015_1pc['Saldos'].sum())

# 8015 PASADO MES PENSIONES CON CONMUTACION PENSIONAL

In [184]:
# Obtener las variables
v8015_2pc = N1Subset['acumulado_mes']
v8015_2capc = N1Subset['Cuenta alternativa']

# Calcular la suma
s8015_2pc = v8015_2pc.sum()

# Crear un DataFrame
v8015_2pc = pd.DataFrame({
    'Cuenta alternativa': v8015_2capc,
    'saldos': v8015_2pc
})

# Extraer los primeros 6 caracteres de "Cuenta alternativa"
v8015_2pc['Cuenta'] = v8015_2pc['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] sea string para la comparación
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Crear un diccionario para mapear rápidamente cuentas a nombres
cuenta_nombre_dict = dict(zip(CUIF['CUENTA'], CUIF['NOMBRE']))

# Aplicar el mapeo para obtener los nombres de cuenta
v8015_2pc['Nombre Cuenta'] = v8015_2pc['Cuenta'].map(cuenta_nombre_dict)

# SUMATORIAS PENSIONES CON CONMUTACION PENSIONAL UNIDAD 8 

In [185]:
# Asignación directa
p8005pc = s8005pc

# Suma de variables y asignación
p8015pc = s8015pc + s8015_1pc + s8015_2pc

# DATAFRAME PENSIONES CON CONMUTACION PENSIONAL UNIDAD 8

In [186]:
# Crear un DataFrame con las subcuentas y saldos
saldos08pc = [p8005pc, p8015pc]
subcuentas08pc = ['005', '015']
unidad8998pc = pd.DataFrame({
    'subcuentas08pc': subcuentas08pc,
    'saldos08pc': saldos08pc
})

# Redondear y convertir a valores absolutos
unidad8998pc['saldos08pc'] = unidad8998pc['saldos08pc'].round(0)
unidad8998pc['saldos08pc'] = unidad8998pc['saldos08pc'].abs()

# Cálculos extra
v8010pc = round(p8005pc * 0.06, 0)

# Usando manejo de casos para evitar división por cero o NaN
if p8005pc == 0:
    v8020pc = 0
else:
    v8020pc = round((p8005pc - p8015pc) / p8005pc, 4)
    if np.isnan(v8020pc):
        v8020pc = 0

v8025pc = max(v8020pc, 0.85)
if v8025pc > 1:
    v8025pc = v8025pc / 100

# Crear segundo DataFrame y concatenarlo con el primero
subcuentas08_2pc = ['010', '020', '025']
saldos08_2pc = [v8010pc, v8020pc, v8025pc]
unidad8998_2pc = pd.DataFrame({
    'subcuentas08pc': subcuentas08_2pc,
    'saldos08pc': saldos08_2pc
})

# Concatenar los DataFrames
unidad8998pc = pd.concat([unidad8998pc, unidad8998_2pc], ignore_index=True)

# Ordenar el DataFrame por subcuentas
unidad8998pc = unidad8998pc.sort_values(by='subcuentas08pc')

# 8005 VIDA INDIVIDUAL

In [187]:
# Filtrar datos usando str_detect equivalente en pandas
AÑO_ACTUALSubset = AÑO_ACTUAL[
    AÑO_ACTUAL['6d'].str.contains('^261505') | 
    AÑO_ACTUAL['6d'].str.contains('^262000')
]

# Filtrar por Line of Buss
lineas_buss = ["E00","E20","E21","E22","E23","E24","E25","E26","E27","E28","E29","E30",
               "E31","E32","E33","E34","E35","E36","E37","E38","E39","E40","E41","E42",
               "E43","E44","E45","E46","E47","E48","E49","E51","E52","E55","E56","E57",
               "E58","E59"]

AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'].isin(lineas_buss)]
AÑO_ACTUALSubset = AÑO_ACTUALSubset[~AÑO_ACTUALSubset['Line of Buss'].isna()]

# Procesamiento de datos
a8005vi = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8005vi = a8005vi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y mantener como string para el merge
a8005vi['Cuenta'] = a8005vi['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
a8005vi = pd.merge(a8005vi, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
a8005vi = a8005vi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8005vi['Unidad de captura'] = 8
a8005vi['Subcuenta'] = "005"
a8005vi['Diferencia mes a mes'] = a8005vi['Saldos'] - a8005vi['saldos_ant']
a8005vi['Diferencia %'] = (a8005vi['Saldos'] - a8005vi['saldos_ant']) / a8005vi['Saldos']
a8005vi = a8005vi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
a8005vi = a8005vi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                   'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                   'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(a8005vi) > 0:
    a8005vi['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8005vi])

# Calcular suma de valores absolutos
s8005vi = abs(int(a8005vi['Saldos'].sum()))

#  8015 VIDA INDIVIDUAL

In [188]:
# Filtrar por la condición 6d contiene "261530"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("261530", na=False)]

# Filtrar por Line of Buss
lineas_buss = ["E00","E20","E21","E22","E23","E24","E25","E26","E27","E28","E29","E30",
               "E31","E32","E33","E34","E35","E36","E37","E38","E39","E40","E41","E42",
               "E43","E44","E45","E46","E47","E48","E49","E51","E52","E55","E56","E57",
               "E58","E59"]
               
AÑO_ACTUALSubset = AÑO_ACTUALSubset[AÑO_ACTUALSubset['Line of Buss'].isin(lineas_buss)]

# Procesamiento de datos
a8015vi = AÑO_ACTUALSubset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a8015vi = a8015vi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y mantener como string para el merge
a8015vi['Cuenta'] = a8015vi['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
a8015vi = pd.merge(a8015vi, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
a8015vi = a8015vi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
a8015vi['Unidad de captura'] = 8
a8015vi['Subcuenta'] = "015"
a8015vi['Diferencia mes a mes'] = a8015vi['Saldos'] - a8015vi['saldos_ant']
a8015vi['Diferencia %'] = (a8015vi['Saldos'] - a8015vi['saldos_ant']) / a8015vi['Saldos']
a8015vi = a8015vi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
a8015vi = a8015vi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                  'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                  'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(a8015vi) > 0:
    a8015vi['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a8015vi])

# Calcular suma de valores absolutos
s8015vi = abs(a8015vi['Saldos'].sum())

# 8015 PASADO VENCIDO VIDA INDIVIDUAL

In [189]:
# Filtrar por la condición 6d contiene "261530"
N1Subset = N1[N1['6d'].str.contains("261530", na=False)]

# Filtrar por Line of Buss
lineas_buss = ["E00","E20","E21","E22","E23","E24","E25","E26","E27","E28","E29","E30",
               "E31","E32","E33","E34","E35","E36","E37","E38","E39","E40","E41","E42",
               "E43","E44","E45","E46","E47","E48","E49","E51","E52","E55","E56","E57",
               "E58","E59"]
               
N1Subset = N1Subset[N1Subset['Line of Buss'].isin(lineas_buss)]

# Procesamiento de datos
v8015_1vi = N1Subset[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
v8015_1vi = v8015_1vi.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'})

# Extraer los primeros 6 caracteres y mantener como string para el merge
v8015_1vi['Cuenta'] = v8015_1vi['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] también sea string
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Unir con CUIF usando left join
v8015_1vi = pd.merge(v8015_1vi, CUIF, how='left', left_on='Cuenta', right_on='CUENTA')
v8015_1vi = v8015_1vi.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas adicionales
v8015_1vi['Unidad de captura'] = 8
v8015_1vi['Subcuenta'] = "015"
v8015_1vi['Diferencia mes a mes'] = v8015_1vi['Saldos'] - v8015_1vi['saldos_ant']
v8015_1vi['Diferencia %'] = (v8015_1vi['Saldos'] - v8015_1vi['saldos_ant']) / v8015_1vi['Saldos']
v8015_1vi = v8015_1vi.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Seleccionar columnas en el orden deseado
v8015_1vi = v8015_1vi[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 
                      'Nombre Cuenta', 'Saldos', 'Saldos del mes pasado', 
                      'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Condicional para agregar notas si hay filas
if len(v8015_1vi) > 0:
    v8015_1vi['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, v8015_1vi])

# Calcular suma de valores absolutos
s8015_1vi = abs(v8015_1vi['Saldos'].sum())

# 8015 PASADO MES VIDA INDIVIDUAL

In [190]:
# Obtener las variables
v8015_2vi = N1Subset['acumulado_mes']
v8015_2cavi = N1Subset['Cuenta alternativa']

# Calcular la suma
s8015_2vi = v8015_2vi.sum()

# Crear un DataFrame
v8015_2vi = pd.DataFrame({
    'Cuenta alternativa': v8015_2cavi,
    'saldos': v8015_2vi
})

# Extraer los primeros 6 caracteres de "Cuenta alternativa"
v8015_2vi['Cuenta'] = v8015_2vi['Cuenta alternativa'].astype(str).str[:6]

# Asegurarse de que CUIF['CUENTA'] sea string para la comparación
CUIF['CUENTA'] = CUIF['CUENTA'].astype(str)

# Crear un mapeo para buscar los nombres de las cuentas
cuenta_to_nombre = dict(zip(CUIF['CUENTA'], CUIF['NOMBRE']))

# Obtener los nombres de cuenta correspondientes
v8015_2vi['Nombre Cuenta'] = v8015_2vi['Cuenta'].map(cuenta_to_nombre)

#  SUMATORIAS VIDA INDIVIDUAL UNIDAD 8

In [191]:
# Asignación directa
p8005vi = s8005vi

# Suma de variables y asignación
p8015vi = s8015vi + s8015_1vi + s8015_2vi

#  DATAFRAME VIDA INDIVIDUAL UNIDAD 8

In [192]:
# Crear un DataFrame con las subcuentas y saldos
saldos08vi = [p8005vi, p8015vi]
subcuentas08vi = ['005', '015']
unidad8998vi = pd.DataFrame({
    'subcuentas08vi': subcuentas08vi,
    'saldos08vi': saldos08vi
})

# Redondear y convertir a valores absolutos
unidad8998vi['saldos08vi'] = unidad8998vi['saldos08vi'].round(0)
unidad8998vi['saldos08vi'] = unidad8998vi['saldos08vi'].abs()

# Cálculos extra
v8010vi = round(p8005vi * 0.06, 0)

# Manejar casos de división por cero o NaN
if p8005vi == 0:
    v8020vi = 0
else:
    v8020vi = round((p8005vi - p8015vi) / p8005vi, 4)
    if np.isnan(v8020vi):
        v8020vi = 0

v8025vi = max(v8020vi, 0.85)
if v8025vi > 1:
    v8025vi = v8025vi / 100

# Crear segundo DataFrame y concatenarlo con el primero
subcuentas08_2vi = ['010', '020', '025']
saldos08_2vi = [v8010vi, v8020vi, v8025vi]
unidad8998_2vi = pd.DataFrame({
    'subcuentas08vi': subcuentas08_2vi,
    'saldos08vi': saldos08_2vi
})

# Concatenar los DataFrames
unidad8998vi = pd.concat([unidad8998vi, unidad8998_2vi], ignore_index=True)

# Ordenar el DataFrame por subcuentas
unidad8998vi = unidad8998vi.sort_values(by='subcuentas08vi')

#  DEMAS RAMOS UNIDAD 9

In [193]:
# Convertir valores string a float antes de operaciones numéricas
try:
    # Obtener valores y convertirlos a float
    valor1 = float(str(unidad7998.loc[9, 'saldos07']).replace(',', '.'))
    valor2 = float(str(unidad5998.loc[7, 'saldos05']).replace(',', '.'))
    valor3 = float(str(unidad6998.loc[11, 'saldos06']).replace(',', '.'))
    
    # Calcular los valores v9005, v9010 y v9998
    v9005 = round(round(valor1, 4) * valor2)
    v9010 = round(valor1 * valor3)
    v9998 = round(max(v9005, v9010))
    
    # Si estructura no existe o es una lista, crear un DataFrame vacío
    if not isinstance(estructura, pd.DataFrame):
        estructura = pd.DataFrame(columns=['unidad', 'subcuenta', 'valor', 'descripcion'])
    
    # Crear las nuevas filas
    nuevas_filas = pd.DataFrame({
        'unidad': ['9', '9', '9'],
        'subcuenta': ['005', '010', '998'],
        'valor': [v9005, v9010, v9998],
        'descripcion': [
            "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (O136) y la subcuenta 998 de la unidad de captura 5 (O100)",
            "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (O136) y la subcuenta 115 de la unidad de captura 6 (O124)",
            "Este valor se calcula de tomar el valor maximo de las subcuentas 05 y 10 de la unidad de captura 9  "
        ]
    })
    
    # Agregar las nuevas filas a estructura
    estructura = pd.concat([estructura, nuevas_filas], ignore_index=True)
except Exception as e:
    # Si ocurre un error, puedes manejarlo silenciosamente o usar pass
    pass

#  DATAFRAME DEMAS RAMOS UNIDAD 9

In [194]:
# Definir las listas de subcuentas y saldos
subcuentas09 = ['005', '010', '998']
saldos09 = [v9005, v9010, v9998]

# Crear un DataFrame con estas listas
unidad9998 = pd.DataFrame({
    'Subcuenta': subcuentas09,
    'Saldos': saldos09
})

# PREVISIONALES E INVALIDEZ Y SOBREVIVENCIA UNIDAD 9

In [195]:
# Calcular los valores v9005pi, v9010pi y v9998pi
# Calcular los valores y convertirlos explícitamente a enteros
v9005pi = int(round(round(float(unidad7998pi['saldos07pi'].iloc[9])) * float(unidad5998pi['saldos05pi'].iloc[7]), 0))
v9010pi = int(round(float(unidad7998pi['saldos07pi'].iloc[9]) * float(unidad6998pi['saldos06pi'].iloc[11]), 0))
v9998pi = int(round(max(v9005pi, v9010pi), 0))

# Crear las nuevas filas como listas
est4 = ["9", "005", v9005pi, "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (N136) y la subcuenta 998 de la unidad de captura 5 (N100)"]
est5 = ["9", "010", v9010pi, "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (N136) y la subcuenta 115 de la unidad de captura 6 (N124)"]
est6 = ["9", "998", v9998pi, "Este valor se calcula de tomar el valor maximo de las subcuentas 05 y 10 de la unidad de captura 9  "]

# Agregar las filas al DataFrame estructura
# En Python, necesitamos convertir las listas a DataFrame antes de concatenar
estructura = pd.concat([estructura, pd.DataFrame([est4], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est5], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est6], columns=estructura.columns)], ignore_index=True)

#  DATAFRAME PREVISIONALES E INVALIDEZ Y SOBREVIVENCIA UNIDAD 9

In [196]:
# Definir las listas de subcuentas y saldos
subcuentas09pi = ['005', '010', '998']
saldos09pi = [v9005pi, v9010pi, v9998pi]

# Crear un DataFrame con estas listas
unidad9998pi = pd.DataFrame({
    'subcuentas09pi': subcuentas09pi,
    'saldos09pi': saldos09pi
})

# Renombrar las columnas
unidad9998pi.columns = ['Subcuentapi', 'Saldospi']

# RIESGOS PROFESIONALES UNIDAD 9

In [197]:
# Calcular los valores v9005rp, v9010rp, v9015rp y v9998rp
# Calcular los valores 
v9005rp = int(round(round(unidad7998rp['saldos07rp'].iloc[9]) * unidad5998rp['saldos05rp'].iloc[7], 0))
v9010rp = int(round(unidad7998rp['saldos07rp'].iloc[9] * unidad6998rp['saldos06rp'].iloc[11], 0))
v9015rp = int(round(unidad8998rp['saldos08rp'].iloc[4] * unidad8998rp['saldos08rp'].iloc[1], 0))
v9998rp = int(round(v9015rp + max(v9005rp, v9010rp), 0))

# Crear las nuevas filas como listas
est7 = ["9", "005", v9005rp, "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (M136) y la subcuenta 998 de la unidad de captura 5 (M100)"]
est8 = ["9", "010", v9010rp, "Este valor se calcula de tomar la subcuenta 050 de la unidad de captura 7 (M136) y la subcuenta 115 de la unidad de captura 6 (M124)"]
est9 = ["9", "015", v9015rp, "Este valor se calcula de tomar la subcuenta 025 de la unidad de captura 8 (M144) y la subcuenta 010 de la unidad de captura 8 (M141)"]
est10 = ["9", "998", v9998rp, "Este valor se calcula de tomar el valor maximo de las subcuentas 05 y 10 de la unidad de captura 9"]

# Agregar las filas al DataFrame estructura
estructura = pd.concat([estructura, pd.DataFrame([est7], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est8], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est9], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est10], columns=estructura.columns)], ignore_index=True)

# DATAFRAME RIESGOS PROFESIONALES UNIDAD 9

In [198]:
# Crear listas para subcuentas y saldos
subcuentas09rp = ['005', '010', '015', '998']
saldos09rp = [v9005rp, v9010rp, v9015rp, v9998rp]

# Crear un DataFrame usando estas listas
unidad9998rp = pd.DataFrame({
    'subcuentas09rp': subcuentas09rp,
    'saldos09rp': saldos09rp
})

# Renombrar las columnas
unidad9998rp.columns = ['Subcuentarp', 'Saldosrp']

# PENSIONES LEY 100 UNIDAD 9 

In [199]:
# Calcular los valores v9015pl y v9998pl
# Nota: Ajustando índices para Python (restando 1 del índice en R)
v9015pl = int(round(float(unidad8998pl['saldos08pl'].iloc[4]) * float(unidad8998pl['saldos08pl'].iloc[1]), 0))
v9998pl = int(round(v9015pl, 0))

# Crear las nuevas filas como listas
est11 = ["9", "015", v9015pl, "Este valor se calcula de tomar la subcuenta 025 de la unidad de captura 8 (K144) y la subcuenta 010 de la unidad de captura 8 (K141)"]
est12 = ["9", "998", v9998pl, "Este valor se calcula de tomar el valor de la subcuenta 015 de la unidad de captura 9 (K148)"]

# Agregar las filas al DataFrame estructura
estructura = pd.concat([estructura, pd.DataFrame([est11], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est12], columns=estructura.columns)], ignore_index=True)

# DATAFRAME PENSIONES LEY 100 UNIDAD 9 

In [200]:
# Crear listas para subcuentas y saldos
subcuentas09pl = ['015', '998']
saldos09pl = [v9015pl, v9998pl]

# Crear un DataFrame usando estas listas
unidad9998pl = pd.DataFrame({
    'subcuentas09pl': subcuentas09pl, 
    'saldos09pl': saldos09pl
})

# Renombrar las columnas
unidad9998pl.columns = ['Subcuentapl', 'Saldospl']

# PENSIONES CON CONMUTACION PENSIONAL UNIDAD 9 

In [201]:
# Calcular los valores v9015pc y v9998pc
# Nota: Ajustando índices para Python (restando 1 del índice en R)
v9015pc = int(round(float(unidad8998pc['saldos08pc'].iloc[4]) * float(unidad8998pc['saldos08pc'].iloc[1]), 0))
v9998pc = int(round(v9015pc, 0))

# Crear las nuevas filas como listas
est13 = ["9", "015", v9015pc, "Este valor se calcula de tomar la subcuenta 025 de la unidad de captura 8 (J144) y la subcuenta 010 de la unidad de captura 8 (J141)"]
est14 = ["9", "998", v9998pc, "Este valor se calcula de tomar el valor de la subcuenta 015 de la unidad de captura 9 (J148)"]

# Agregar las filas al DataFrame estructura
estructura = pd.concat([estructura, pd.DataFrame([est13], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est14], columns=estructura.columns)], ignore_index=True)

# DATAFRAME PENSIONES CON CONMUTACION PENSIONAL UNIDAD 9

In [202]:
# Crear listas para subcuentas y saldos
subcuentas09pc = ['015', '998']
saldos09pc = [v9015pc, v9998pc]

# Crear un DataFrame usando estas listas
unidad9998pc = pd.DataFrame({
    'subcuentas09pc': subcuentas09pc, 
    'saldos09pc': saldos09pc
})

# Renombrar las columnas
unidad9998pc.columns = ['Subcuentapc', 'Saldospc']

# VIDA INDIVIDUAL UNIDAD 9

In [203]:
# Calcular los valores v9015vi y v9998vi
# Nota: Ajustando índices para Python (restando 1 del índice en R)
v9015vi = int(round(float(unidad8998vi['saldos08vi'].iloc[4]) * float(unidad8998vi['saldos08vi'].iloc[1]), 0))
v9998vi = int(round(v9015vi, 0))

# Crear las nuevas filas como listas
est15 = ["9", "015", v9015vi, "Este valor se calcula de tomar la subcuenta 025 de la unidad de captura 8 (I144) y la subcuenta 010 de la unidad de captura 8 (I141)"]
est16 = ["9", "998", v9998vi, "Este valor se calcula de tomar el valor de la subcuenta 015 de la unidad de captura 9 (I148)"]

# Agregar las filas al DataFrame estructura
estructura = pd.concat([estructura, pd.DataFrame([est15], columns=estructura.columns)], ignore_index=True)
estructura = pd.concat([estructura, pd.DataFrame([est16], columns=estructura.columns)], ignore_index=True)

# DATAFRAME VIDA INDIVIDUAL UNIDAD 9

In [204]:
# Crear listas para subcuentas y saldos
subcuentas09vi = ['015', '998']
saldos09vi = [v9015vi, v9998vi]

# Crear un DataFrame usando estas listas
unidad9998vi = pd.DataFrame({
    'subcuentas09vi': subcuentas09vi,
    'saldos09vi': saldos09vi
})

# Renombrar las columnas
unidad9998vi.columns = ['Subcuentavi', 'Saldosvi']

# 10005

In [205]:
# Filtrar el DataFrame AÑO_ACTUAL para encontrar filas donde la columna '4d' contiene "1105"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains("1105", na=False)]

# Crear el DataFrame a10005
a10005 = AÑO_ACTUALSubset.loc[:, ['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a10005.columns = ['Cuenta alternativa', 'Saldos', 'saldos_ant', 'Ejercicio']

# Crear columna 'Cuenta' extrayendo los primeros 6 caracteres 
# Mantenemos como string para que coincida con CUIF (o convertimos CUIF a numérico)
a10005['Cuenta'] = a10005['Cuenta alternativa'].str[:6]

# Asegurarse de que la columna 'CUENTA' en CUIF sea del mismo tipo que 'Cuenta' en a10005
# Opción 1: Convertir 'Cuenta' a string si 'CUENTA' es string
# a10005['Cuenta'] = a10005['Cuenta'].astype(str)

# Opción 2: Convertir 'CUENTA' a numérico si 'Cuenta' es numérico
CUIF_copy = CUIF.copy()
CUIF_copy['CUENTA'] = pd.to_numeric(CUIF_copy['CUENTA'], errors='coerce')
a10005['Cuenta'] = pd.to_numeric(a10005['Cuenta'], errors='coerce')

# Unir con el DataFrame CUIF
a10005 = pd.merge(a10005, CUIF_copy, left_on='Cuenta', right_on='CUENTA', how='left')

# Renombrar la columna 'NOMBRE' a 'Nombre Cuenta'
a10005 = a10005.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas 'Unidad de captura' y 'Subcuenta'
a10005['Unidad de captura'] = 10
a10005['Subcuenta'] = "005"

# Calcular diferencias
a10005['Diferencia mes a mes'] = a10005['Saldos'] - a10005['saldos_ant']
a10005['Diferencia %'] = (a10005['Saldos'] - a10005['saldos_ant']) / a10005['Saldos']

# Renombrar 'saldos_ant' a 'Saldos del mes pasado'
a10005 = a10005.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Reordenar y seleccionar columnas
a10005 = a10005[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
               'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Verificar si hay filas en a10005 y agregar a bd_analisis_cuentas
if len(a10005) > 0:
    a10005['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a10005], ignore_index=True)

# Calcular la suma absoluta de la columna 'Saldos'
s10005 = int(abs(a10005['Saldos'].sum()))

# 100010

In [206]:
# Filtrar el DataFrame AÑO_ACTUAL para encontrar filas donde la columna '4d' contiene "1115"
AÑO_ACTUALSubset = AÑO_ACTUAL[AÑO_ACTUAL['4d'].str.contains("1115", na=False)]

# Crear el DataFrame a10010
a10010 = AÑO_ACTUALSubset.loc[:, ['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']].copy()
a10010.columns = ['Cuenta alternativa', 'Saldos', 'saldos_ant', 'Ejercicio']

# Crear columna 'Cuenta' extrayendo los primeros 6 caracteres 
a10010['Cuenta'] = a10010['Cuenta alternativa'].str[:6]

# Convertir columnas a numérico para asegurar la compatibilidad en el merge
a10010['Cuenta'] = pd.to_numeric(a10010['Cuenta'], errors='coerce')
CUIF_copy = CUIF.copy()
CUIF_copy['CUENTA'] = pd.to_numeric(CUIF_copy['CUENTA'], errors='coerce')

# Unir con el DataFrame CUIF
a10010 = pd.merge(a10010, CUIF_copy, left_on='Cuenta', right_on='CUENTA', how='left')

# Renombrar la columna 'NOMBRE' a 'Nombre Cuenta'
a10010 = a10010.rename(columns={'NOMBRE': 'Nombre Cuenta'})

# Agregar columnas 'Unidad de captura' y 'Subcuenta'
a10010['Unidad de captura'] = 10
a10010['Subcuenta'] = "010"

# Calcular diferencias
a10010['Diferencia mes a mes'] = a10010['Saldos'] - a10010['saldos_ant']
a10010['Diferencia %'] = (a10010['Saldos'] - a10010['saldos_ant']) / a10010['Saldos']

# Renombrar 'saldos_ant' a 'Saldos del mes pasado'
a10010 = a10010.rename(columns={'saldos_ant': 'Saldos del mes pasado'})

# Reordenar y seleccionar columnas
a10010 = a10010[['Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
               'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio']]

# Verificar si hay filas en a10010 y agregar a bd_analisis_cuentas
if len(a10010) > 0:
    a10010['notas'] = ""  # Agregar columna de notas vacía
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, a10010], ignore_index=True)

# Calcular la suma absoluta de la columna 'Saldos' y convertir a entero nativo de Python
s10010 = int(abs(a10010['Saldos'].sum()))

# 10015

In [207]:
# Filtrar activos por CATEGORIA 'I'
activos_c1 = activos[activos['CATEGORIA'] == 'I']

# Filtrar por SUBCUENTA '015' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015'].dropna()

# Calcular la suma, redondear y luego convertir a entero
s10015 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
nueva_fila = pd.DataFrame({
    'unidad': ['10'],
    'subcuenta': ['015'],
    'valor': [s10015],
    'descripcion': ["Este valor se toma de la hoja Activos, como fuente de información"]
})

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, nueva_fila], ignore_index=True)

#  10020

In [208]:
# Filtrar por SUBCUENTA '020' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10020 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est18 = ["10", "020", s10020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est18], columns=estructura.columns)], ignore_index=True)

# 10025

In [209]:
# Filtrar por SUBCUENTA '025' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '025'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10025 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est19 = ["10", "025", s10025, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est19], columns=estructura.columns)], ignore_index=True)

# 10030

In [210]:
# Filtrar por SUBCUENTA '030' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '030'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10030 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est20 = ["10", "030", s10030, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est20], columns=estructura.columns)], ignore_index=True)

# 10035 

In [211]:
# Filtrar por SUBCUENTA '035' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '035'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10035 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est21 = ["10", "035", s10035, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est21], columns=estructura.columns)], ignore_index=True)

# 10040 

In [212]:
# Filtrar por SUBCUENTA '040' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '040'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10040 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est22 = ["10", "040", s10040, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est22], columns=estructura.columns)], ignore_index=True)

# 10045

In [213]:
# Filtrar por SUBCUENTA '045'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Calcular la suma ignorando valores NA y convertir a entero Python nativo
s10045 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True)))

# Crear nueva fila para estructura
est23 = ["10", "045", s10045, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est23], columns=estructura.columns)], ignore_index=True)

#  10050

In [214]:
# Filtrar por SUBCUENTA '050' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10050 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est24 = ["10", "050", s10050, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est24], columns=estructura.columns)], ignore_index=True)

# 10055

In [215]:
# Filtrar por SUBCUENTA '055' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '055'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10055 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est25 = ["10", "055", s10055, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est25], columns=estructura.columns)], ignore_index=True)

# 10060

In [216]:
# Filtrar por SUBCUENTA '060' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '060'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10060 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est26 = ["10", "060", s10060, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est26], columns=estructura.columns)], ignore_index=True)

# 10065

In [217]:
# Filtrar por SUBCUENTA '065' y eliminar NAs
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '065'].dropna()

# Calcular la suma y convertir a entero Python nativo (asegurando redondeo apropiado)
s10065 = int(round(subsetactivos['Valor Mercado ($)'].sum()))

# Crear nueva fila para estructura
est27 = ["10", "065", s10065, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est27], columns=estructura.columns)], ignore_index=True)

# SUMATORIAS

In [218]:
# Calcular sumas (aunque en este caso son redundantes ya que s10005 y s10010 ya son sumas)
p10005 = s10005
p10010 = s10010

# Crear DataFrames individuales
v10005f = pd.DataFrame({
    'periodo': ['Mes actual'],
    'Saldos': [p10005],
    'Subcuenta': ['005']
})

v10010f = pd.DataFrame({
    'periodo': ['Mes actual'],
    'Saldos': [p10010],
    'Subcuenta': ['010']
})

# Combinar los DataFrames
captura10 = pd.concat([v10005f, v10010f], ignore_index=True)
captura10['Unidad de Captura'] = 10

# Crear unidad10998
unidad10998 = pd.DataFrame({
    'subcuentas10': ['005', '010'],
    'saldos10': [p10005, p10010]
})

# DATAFRAME

In [219]:
# Crear dataframe unidad10998_2 con subcuentas
unidad10998_2 = pd.DataFrame({
    'Subcuenta': ['015', '020', '025', '030', '035', '040', '045',
                  '050', '055', '060', '065', '999']
})

# Calcular s10999 como la suma redondeada de todos los valores
s10999 = int(round(p10005) + round(p10010) + round(s10015) + round(s10020) + 
           round(s10025) + round(s10030) + round(s10035) + round(s10040) + 
           round(s10045) + round(s10050) + round(s10055) + round(s10060) + 
           round(s10065))

# Asignar saldos a unidad10998_2
unidad10998_2['Saldos'] = [s10015, s10020, s10025, s10030, s10035,
                          s10040, s10045, s10050, s10055, s10060, s10065, s10999]

# Renombrar columnas de unidad10998 (asumiendo que ya existe)
unidad10998.columns = ['Subcuenta', 'Saldos']

# Renombrar columnas de unidad10998_2
unidad10998_2.columns = ['Subcuenta', 'Saldos']

# Combinar unidad10998 y unidad10998_2
unidad10998 = pd.concat([unidad10998, unidad10998_2], ignore_index=True)

# Redondear y tomar valores absolutos de la columna Saldos
unidad10998['Saldos'] = unidad10998['Saldos'].round(0).abs().astype(int)

# VALIDACIÓN REASEGURO

# SAP

In [220]:
# Filtrar y sumar para cuenta 1677
reaseguro_1677_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1677', na=False)]
reaseguro_1677_SAP_sum = reaseguro_1677_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 1678
reaseguro_1678_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1678', na=False)]
reaseguro_1678_SAP_sum = reaseguro_1678_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 169515
reaseguro_169515_SAP = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('169515', na=False)]
reaseguro_169515_SAP_sum = reaseguro_169515_SAP['Total Cuenta'].sum()

# Filtrar y sumar para cuenta 1680
reaseguro_1680_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1680', na=False)]
reaseguro_1680_SAP_sum = reaseguro_1680_SAP['Total Cuenta'].sum()

# CALIFICADO

In [221]:
# CALIFICADO_REASEGURO_1677
CALIFICADO_REASEGURO_1677 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_1678
CALIFICADO_REASEGURO_1678 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_1680
CALIFICADO_REASEGURO_1680 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Activos_por_contingencias'].sum(skipna=True)]
})

# CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro_Contingencia'].sum(skipna=True)]
})

# INTERIOR_REASEGURO_CALIFICACION
INTERIOR_REASEGURO_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Deterioro'].sum(skipna=True)]
})

# EXTERIOR_REASEGURO_CALIFICACION
EXTERIOR_REASEGURO_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro'].sum(skipna=True)]
})

# CALIFICADO_REASEGURO_169515 inicial
CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'CALIFICADO') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# Actualizar CALIFICADO_REASEGURO_169515
CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [INTERIOR_REASEGURO_CALIFICACION['total'].values[0] + 
              EXTERIOR_REASEGURO_CALIFICACION['total'].values[0]]
})

# SIN CALIFICACIÓN

In [222]:
# SIN_CALIFICACION_REASEGURO_1677
SIN_CALIFICACION_REASEGURO_1677 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_1678
SIN_CALIFICACION_REASEGURO_1678 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_1680
SIN_CALIFICACION_REASEGURO_1680 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN')
    ]['Activos_por_contingencias'].sum(skipna=True)]
})

# SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN')
    ]['Deterioro_Contingencia'].sum(skipna=True)]
})

# INTERIOR_REASEGURO_SIN_CALIFICACION
INTERIOR_REASEGURO_SIN_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Interior')
    ]['Deterioro'].sum(skipna=True)]
})

# EXTERIOR_REASEGURO_SIN_CALIFICACION
EXTERIOR_REASEGURO_SIN_CALIFICACION = pd.DataFrame({
    'total': [-bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Deterioro'].sum(skipna=True)]
})

# SIN_CALIFICACION_REASEGURO_169515 inicial
temp_sin_calif_169515 = pd.DataFrame({
    'total': [bd_reaseguros[
        (bd_reaseguros['Calificado'].str.strip() == 'SIN CALIFICACIÓN') & 
        (bd_reaseguros['Interior_Exterior'].str.strip() == 'Exterior')
    ]['Cuenta_Corriente'].sum(skipna=True)]
})

# Actualizar SIN_CALIFICACION_REASEGURO_169515
SIN_CALIFICACION_REASEGURO_169515 = pd.DataFrame({
    'total': [INTERIOR_REASEGURO_SIN_CALIFICACION['total'].values[0] + 
              EXTERIOR_REASEGURO_SIN_CALIFICACION['total'].values[0]]
})

# TOTALES

In [223]:
# Calcular totales sumando valores calificados y sin calificación
TOTAL_CALIFICADO_REASEGURO_1677 = CALIFICADO_REASEGURO_1677 + SIN_CALIFICACION_REASEGURO_1677
TOTAL_CALIFICADO_REASEGURO_1678 = CALIFICADO_REASEGURO_1678 + SIN_CALIFICACION_REASEGURO_1678
TOTAL_CALIFICADO_REASEGURO_169515 = CALIFICADO_REASEGURO_169515 + SIN_CALIFICACION_REASEGURO_169515
TOTAL_CALIFICADO_REASEGURO_1680 = CALIFICADO_REASEGURO_1680 + SIN_CALIFICACION_REASEGURO_1680
TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO + SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO
TOTAL_INTERIOR_REASEGURO_CALIFICACION = INTERIOR_REASEGURO_CALIFICACION + INTERIOR_REASEGURO_SIN_CALIFICACION
TOTAL_EXTERIOR_REASEGURO_CALIFICACION = EXTERIOR_REASEGURO_CALIFICACION + EXTERIOR_REASEGURO_SIN_CALIFICACION

# DIFERENCIAS

In [224]:
# Primero calculamos los totales
TOTAL_CALIFICADO_REASEGURO_1677 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1677['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1677['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_1678 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1678['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1678['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_169515 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_169515['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_169515['total'].iloc[0])]
})

TOTAL_CALIFICADO_REASEGURO_1680 = pd.DataFrame({
    'total': [round(CALIFICADO_REASEGURO_1680['total'].iloc[0] + SIN_CALIFICACION_REASEGURO_1680['total'].iloc[0])]
})

TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO = pd.DataFrame({
    'total': [round(CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO['total'].iloc[0] + 
              SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO['total'].iloc[0])]
})

# Función auxiliar para obtener el valor numérico y redondearlo
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return round(x['total'].iloc[0])
    return round(x)

# Ahora calculamos las diferencias
DIFERENCIA_REASEGURO1677 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1677) - get_value(reaseguro_1677_SAP_sum))]
})

DIFERENCIA_REASEGURO1678 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1678) - get_value(reaseguro_1678_SAP_sum))]
})

DIFERENCIA_REASEGURO169515 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_169515) - get_value(reaseguro_169515_SAP_sum))]
})

DIFERENCIA_REASEGURO1680 = pd.DataFrame({
    'total': [round(get_value(TOTAL_CALIFICADO_REASEGURO_1680) - get_value(reaseguro_1680_SAP_sum) + 
              get_value(TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO))]
})

# INFORMACIÓN REASEGURO

In [225]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Crear matriz vacía
INFORMACION_REASEGURO = np.empty((7, 7), dtype=object)

# Definir nombres de columnas
columnas = ["CUENTA", "NOMBRE", "SAP", "CALIFICADO", "SIN_CALIFICACION", "TOTAL", "DIFERENCIA"]

# Función auxiliar para obtener valores
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return redondear_valor(x['total'].iloc[0])
    return redondear_valor(float(x)) if isinstance(x, (int, float)) else x

# Columna CUENTA (no aplicar redondeo)
INFORMACION_REASEGURO[0:4, 0] = [1677, 1678, 169515, 1680]

# Columna NOMBRE (no aplicar redondeo)
nombres = [
    "REASEGURADORES INTERIOR CUENTA CORRIENTE",
    "REASEGURADORES EXTERIOR CUENTA CORRIENTE",
    "DETERIORO CUENTAS POR COBRAR REASEGURADORES",
    "REASEGURADORES ACTIVOS POR CONTINGENCIAS",
    "DETERIORO ACTIVOS POR CONTINGENCIAS",
    "Interior",
    "Exterior"
]
INFORMACION_REASEGURO[:, 1] = nombres

# Columna SAP
INFORMACION_REASEGURO[0:4, 2] = [
    get_value(reaseguro_1677_SAP_sum),
    get_value(reaseguro_1678_SAP_sum),
    get_value(reaseguro_169515_SAP_sum),
    get_value(reaseguro_1680_SAP_sum)
]

# Columna CALIFICADO
INFORMACION_REASEGURO[:, 3] = [
    get_value(CALIFICADO_REASEGURO_1677),
    get_value(CALIFICADO_REASEGURO_1678),
    get_value(CALIFICADO_REASEGURO_169515),
    get_value(CALIFICADO_REASEGURO_1680),
    get_value(CALIFICADO_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(INTERIOR_REASEGURO_CALIFICACION),
    get_value(EXTERIOR_REASEGURO_CALIFICACION)
]

# Columna SIN_CALIFICACION
INFORMACION_REASEGURO[:, 4] = [
    get_value(SIN_CALIFICACION_REASEGURO_1677),
    get_value(SIN_CALIFICACION_REASEGURO_1678),
    get_value(SIN_CALIFICACION_REASEGURO_169515),
    get_value(SIN_CALIFICACION_REASEGURO_1680),
    get_value(SIN_CALIFICACION_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(INTERIOR_REASEGURO_SIN_CALIFICACION),
    get_value(EXTERIOR_REASEGURO_SIN_CALIFICACION)
]

# Columna TOTAL
INFORMACION_REASEGURO[:, 5] = [
    get_value(TOTAL_CALIFICADO_REASEGURO_1677),
    get_value(TOTAL_CALIFICADO_REASEGURO_1678),
    get_value(TOTAL_CALIFICADO_REASEGURO_169515),
    get_value(TOTAL_CALIFICADO_REASEGURO_1680),
    get_value(TOTAL_DETERIORO_ACTIVOS_POR_CONTINGENCIAS_REASEGURO),
    get_value(TOTAL_INTERIOR_REASEGURO_CALIFICACION),
    get_value(TOTAL_EXTERIOR_REASEGURO_CALIFICACION)
]

# Columna DIFERENCIA
INFORMACION_REASEGURO[0:4, 6] = [
    get_value(DIFERENCIA_REASEGURO1677),
    get_value(DIFERENCIA_REASEGURO1678),
    get_value(DIFERENCIA_REASEGURO169515),
    get_value(DIFERENCIA_REASEGURO1680)
]

# Convertir a DataFrame para mejor visualización
INFORMACION_REASEGURO = pd.DataFrame(INFORMACION_REASEGURO, columns=columnas)

#  VALIDACIÓN COASEGURO 

# SAP

In [226]:
# Filtrar y sumar para coaseguro 1675
coaseguro_1675_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1675')]
coaseguro_1675_SAP_sum = pd.DataFrame({
    'total': [coaseguro_1675_SAP['Total Cuenta'].sum()]
})

# Filtrar y sumar para coaseguro 1676
coaseguro_1676_SAP = AÑO_ACTUAL[AÑO_ACTUAL['4d'].astype(str).str.contains('1676')]
coaseguro_1676_SAP_sum = pd.DataFrame({
    'total': [coaseguro_1676_SAP['Total Cuenta'].sum()]
})

# Filtrar y sumar para coaseguro 169510
coaseguro_169510_SAP = AÑO_ACTUAL[AÑO_ACTUAL['6d'].astype(str).str.contains('169510')]
coaseguro_169510_SAP_sum = pd.DataFrame({
    'total': [coaseguro_169510_SAP['Total Cuenta'].sum()]
})

# CALIFICADO

In [227]:
# Para CALIFICADO_COASEGURO_1675
CALIFICADO_COASEGURO_1675 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS['Calificado'].sum(skipna=True)]
})

# Para CALIFICADO_COASEGURO_1676
CALIFICADO_COASEGURO_1676 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS['Calificado'].sum(skipna=True)]
})

# Para CALIFICADO_COASEGURO_169510
CALIFICADO_COASEGURO_169510 = pd.DataFrame({
    'total': [bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES['Calificado'].sum(skipna=True)]
})

# NO CALIFICADO

In [228]:
# Para NO_CALIFICADO_COASEGURO_1675
NO_CALIFICADO_COASEGURO_1675 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_ACEPTADOS['No_Calificado '].sum(skipna=True)]
})

# Para NO_CALIFICADO_COASEGURO_1676
NO_CALIFICADO_COASEGURO_1676 = pd.DataFrame({
    'total': [bd_COASEGURADORES_CUENTA_CORRIENTE_CEDIDOS['No_Calificado '].sum(skipna=True)]
})

# Para NO_CALIFICADO_COASEGURO_169510
NO_CALIFICADO_COASEGURO_169510 = pd.DataFrame({
    'total': [bd_DETERIORO_CUENTAS_POR_COBRAR_ACTIVIDAD_ASEGURADORA_COASEGURADORES['No_Calificado '].sum(skipna=True)]
})

# TOTALES

In [229]:
# Calcular totales
TOTAL_COASEGURO_1675 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_1675['total'].iloc[0] + NO_CALIFICADO_COASEGURO_1675['total'].iloc[0]]
})

TOTAL_COASEGURO_1676 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_1676['total'].iloc[0] + NO_CALIFICADO_COASEGURO_1676['total'].iloc[0]]
})

TOTAL_COASEGURO_169510 = pd.DataFrame({
    'total': [CALIFICADO_COASEGURO_169510['total'].iloc[0] + NO_CALIFICADO_COASEGURO_169510['total'].iloc[0]]
})

# DIFERENCIAS

In [230]:
# Calcular diferencias con redondeo
def redondear_valor(x):
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

DIFERENCIA_COASEASEGURO1675 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_1675['total'].iloc[0] - coaseguro_1675_SAP_sum['total'].iloc[0])]
})

DIFERENCIA_COASEASEGURO1676 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_1676['total'].iloc[0] - coaseguro_1676_SAP_sum['total'].iloc[0])]
})

DIFERENCIA_COASEASEGURO169510 = pd.DataFrame({
    'total': [redondear_valor(TOTAL_COASEGURO_169510['total'].iloc[0] - coaseguro_169510_SAP_sum['total'].iloc[0])]
})

# INFORMACIÓN COASEGURO

In [231]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Crear matriz vacía
INFORMACION_COASEGURO = np.empty((3, 7), dtype=object)

# Definir nombres de columnas
columnas = ["CUENTA", "NOMBRE", "SAP", "CALIFICADO", "NO_CALIFICADO", "TOTAL", "DIFERENCIA"]

# Función auxiliar para obtener valores
def get_value(x):
    if isinstance(x, (pd.DataFrame, pd.Series)):
        return redondear_valor(x['total'].iloc[0])
    return redondear_valor(float(x)) if isinstance(x, (int, float)) else x

# Llenar la matriz
# Columna CUENTA (no aplicar redondeo a estos valores)
INFORMACION_COASEGURO[0:3, 0] = [1675, 1676, 169510]

# Columna NOMBRE (no aplicar redondeo a textos)
nombres = [
    "COASEGURADORES CUENTA CORRIENTE ACEPTADOS",
    "COASEGURADORES CUENTA CORRIENTE CEDIDOS",
    "DETERIORO CUENTAS POR COBRAR ACTIVIDAD ASEGURADORA COASEGURADORES"
]
INFORMACION_COASEGURO[:, 1] = nombres

# Columna SAP
INFORMACION_COASEGURO[:, 2] = [
    get_value(coaseguro_1675_SAP_sum),
    get_value(coaseguro_1676_SAP_sum),
    get_value(coaseguro_169510_SAP_sum)
]

# Columna CALIFICADO
INFORMACION_COASEGURO[:, 3] = [
    get_value(CALIFICADO_COASEGURO_1675),
    get_value(CALIFICADO_COASEGURO_1676),
    get_value(CALIFICADO_COASEGURO_169510)
]

# Columna NO_CALIFICADO
INFORMACION_COASEGURO[:, 4] = [
    get_value(NO_CALIFICADO_COASEGURO_1675),
    get_value(NO_CALIFICADO_COASEGURO_1676),
    get_value(NO_CALIFICADO_COASEGURO_169510)
]

# Columna TOTAL
INFORMACION_COASEGURO[:, 5] = [
    get_value(TOTAL_COASEGURO_1675),
    get_value(TOTAL_COASEGURO_1676),
    get_value(TOTAL_COASEGURO_169510)
]

# Columna DIFERENCIA
INFORMACION_COASEGURO[:, 6] = [
    get_value(DIFERENCIA_COASEASEGURO1675),
    get_value(DIFERENCIA_COASEASEGURO1676),
    get_value(DIFERENCIA_COASEASEGURO169510)
]

# Convertir a DataFrame para mejor visualización
INFORMACION_COASEGURO = pd.DataFrame(INFORMACION_COASEGURO, columns=columnas)

# 11005

In [232]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar activos por CATEGORIA == 'II'
activos_c1 = activos[activos['CATEGORIA'] == 'II']

# Filtrar por SUBCUENTA == '005'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '005']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11005 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est28 = ["11", "005", v11005, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est28], columns=estructura.columns)], ignore_index=True)

# 11010

In [233]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Para subcuenta 010
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '010']
v11010 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))
est29 = ["11", "010", v11010, "Este valor se toma de la hoja Activos, como fuente de información"]
estructura = pd.concat([estructura, pd.DataFrame([est29], columns=estructura.columns)], ignore_index=True)

# 11015

In [234]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Para subcuenta 015
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015']
v11015 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))
est30 = ["11", "015", v11015, "Este valor se toma de la hoja Activos, como fuente de información"]
estructura = pd.concat([estructura, pd.DataFrame([est30], columns=estructura.columns)], ignore_index=True)

# 11020

In [235]:
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '020'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v11020 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
est31 = ["11", "020", v11020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est31], columns=estructura.columns)], ignore_index=True)


# 11040

In [236]:
# Suma de las primeras 3 filas de la columna 4 de INFORMACION_COASEGURO
# En Python, los índices comienzan en 0, por lo que usamos 0:3 en lugar de 1:3
v11040 = sum(pd.to_numeric(INFORMACION_COASEGURO.iloc[0:3, 2], errors='coerce'))

# Opcionalmente, convertir a float de Python estándar
v11040 = int(round(v11040))

# Crear una nueva fila como un DataFrame de una sola fila
est32_df = pd.DataFrame({
    0: ["11"], 
    1: ["040"], 
    2: [v11040], 
    3: ["Este valor se toma de la hoja Coaseguro, como fuente de información"]
})

# Concatenar el nuevo DataFrame a estructura
estructura = pd.concat([estructura, est32_df], ignore_index=True)

# 11045

In [237]:
# Filtrar por SUBCUENTA == '045'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Calcular la suma del Valor Mercado ignorando valores NA
v11045 = float(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Opcionalmente, si necesitas redondear a entero
v11045 = int(round(v11045))

# Si estructura es un DataFrame:
est33_df = pd.DataFrame({
    0: ["11"], 
    1: ["045"], 
    2: [v11045], 
    3: ["Este valor se toma de la hoja Activos, como fuente de información"]
})
# Concatenar el nuevo DataFrame a estructura
estructura = pd.concat([estructura, est33_df], ignore_index=True)

# 11050

In [238]:
# Filtrar por SUBCUENTA == '050'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050']

# Calcular la suma del Valor Mercado ignorando valores NA
v11050 = float(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Opcionalmente, si necesitas redondear a entero
v11050 = int(round(v11050))

# Si estructura es un DataFrame:
est34_df = pd.DataFrame({
    0: ["11"], 
    1: ["050"], 
    2: [v11050], 
    3: ["Este valor se toma de la hoja Activos, como fuente de información"]
})
# Concatenar el nuevo DataFrame a estructura
estructura = pd.concat([estructura, est34_df], ignore_index=True)

# DATAFRAME

In [239]:
# Crear DataFrame inicial
unidad11998_2 = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '040', '045', '050'],
    'Saldos': [v11005, v11010, v11015, v11020, v11040, v11045, v11050]
})

# Crear copia para unidad11998
unidad11998 = unidad11998_2.copy()

# Agregar fila con la suma total
suma_total = unidad11998['Saldos'].sum()
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad11998 = pd.concat([unidad11998, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad11998['Saldos'] = pd.to_numeric(unidad11998['Saldos'])

# UNIDAD 11 (PONDERADA)

In [240]:
# Opción 1: Adaptarse a las 8 columnas existentes
# Supongamos que estructura tiene 8 columnas, necesitamos rellenar las 4 faltantes con valores vacíos
v11005p = int(round(v11005 * 0.015, 0))
est35 = ["11", "005", v11005p, "Este valor se calcula de la subcuenta 005 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est35], columns=estructura.columns)], ignore_index=True)

v11010p = int(round(v11010 * 0.015, 0))
est36 = ["11", "010", v11010p, "Este valor se calcula de la subcuenta 010 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est36], columns=estructura.columns)], ignore_index=True)

v11015p = int(round(v11015 * 0.015, 0))
est37 = ["11", "015", v11015p, "Este valor se calcula de la subcuenta 015 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est37], columns=estructura.columns)], ignore_index=True)

v11020p = int(round(v11020 * 0.015, 0))
est38 = ["11", "020", v11020p, "Este valor se calcula de la subcuenta 020 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est38], columns=estructura.columns)], ignore_index=True)

v11040p = int(round(v11040 * 0.015, 0))
est39 = ["11", "040", v11040p, "Este valor se calcula de la subcuenta 040 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est39], columns=estructura.columns)], ignore_index=True)

v11045p = int(round(v11045 * 0.015, 0))
est40 = ["11", "045", v11045p, "Este valor se calcula de la subcuenta 045 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est40], columns=estructura.columns)], ignore_index=True)

v11050p = int(round(v11050 * 0.015, 0))
est41 = ["11", "050", v11050p, "Este valor se calcula de la subcuenta 050 de la unidad de captura 11 multiplicado por el 1,5%"] + [None, None, None, None]
estructura = pd.concat([estructura, pd.DataFrame([est41], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2510308477.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est35], columns=estructura.columns)], ignore_index=True)
C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2510308477.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est36], columns=estructura.columns)], ignore_index=True)
C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2510

# DATAFRAME

In [241]:
# Crear DataFrame inicial
unidad11998_p = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '040', '045', '050'],
    'Saldos': [v11005p, v11010p, v11015p, v11020p, v11040p, v11045p, v11050p]
})

# Calcular la suma y redondear hacia arriba con ceiling
suma_total = np.ceil(unidad11998_p['Saldos'].sum())

# Agregar fila con la suma total
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad11998_p = pd.concat([unidad11998_p, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad11998_p['Saldos'] = pd.to_numeric(unidad11998_p['Saldos'])

# 12005

In [242]:
# Filter activos DataFrame where CATEGORIA is 'III'
activos_c1 = activos[activos['CATEGORIA'] == 'III']

# Further filter to get only rows where SUBCUENTA is '005'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '005']

# Sum the 'Valor Mercado ($)' column, ignoring NA values
v12005 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True)))

# Create a new row for the estructura DataFrame
# If estructura has 8 columns, add None values for the extra columns
if len(estructura.columns) > 4:
    est42 = ["12", "005", v12005, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est42 = ["12", "005", v12005, "Este valor se toma de la hoja Activos, como fuente de información"]

# Add the new row to estructura DataFrame using loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est42))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1875100146.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est42))


# 12010

In [243]:
# Función de redondeo especial
def redondear_valor(x):
    if isinstance(x, (str, int)) or pd.isna(x):  # No redondear strings, enteros o NaN
        return x
    decimal = abs(x - int(x))
    if decimal < 0.5:
        return int(x)  # Redondea hacia abajo
    else:
        return int(x) + (1 if x > 0 else -1)  # Redondea hacia arriba

# Filtrar por SUBCUENTA == '010'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '010']

# Calcular la suma del Valor Mercado y aplicar redondeo especial
v12010 = redondear_valor(subsetactivos['Valor Mercado ($)'].sum(skipna=True))

# Crear nueva fila para estructura
# Si estructura tiene 8 columnas, necesitamos llenar las últimas 4 con None
est43 = ["12", "010", v12010, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est43], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\22341688.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est43], columns=estructura.columns)], ignore_index=True)


# 12015

In [244]:
# Filtrar activos_c1 donde SUBCUENTA sea '015'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '015']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA
v12015 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True)))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est44 = ["12", "015", v12015, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est44 = ["12", "015", v12015, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est44))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1810789770.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est44))


# 12020

In [245]:
# Filtrar activos_c1 donde SUBCUENTA sea '020'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '020']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA
v12020 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True)))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est45 = ["12", "020", v12020, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est45 = ["12", "020", v12020, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est45))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3047425217.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est45))


# 12025

In [246]:
# Filtrar activos_c1 donde SUBCUENTA sea '025'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '025']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12025 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est46 = ["12", "025", v12025, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est46 = ["12", "025", v12025, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est46))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2869146036.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est46))


# 12030

In [247]:
# Filtrar activos_c1 donde SUBCUENTA sea '030'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '030']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12030 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est47 = ["12", "030", v12030, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est47 = ["12", "030", v12030, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est47))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3494052890.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est47))


# 12045

In [248]:
# Filtrar activos_c1 donde SUBCUENTA sea '045'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '045']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12045 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est48 = ["12", "045", v12045, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est48 = ["12", "045", v12045, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est48))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\954739014.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est48))


# 12050

In [249]:
# Filtrar activos_c1 donde SUBCUENTA sea '050'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '050']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12050 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est49 = ["12", "050", v12050, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est49 = ["12", "050", v12050, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est49))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2111777728.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est49))


# 12055

In [250]:
# Filtrar activos_c1 donde SUBCUENTA sea '055'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '055']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12055 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est50 = ["12", "055", v12055, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est50 = ["12", "055", v12055, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est50))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2075097937.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est50))


# 12060

In [251]:
# Filtrar activos_c1 donde SUBCUENTA sea '060'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '060']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12060 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est51 = ["12", "060", v12060, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est51 = ["12", "060", v12060, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est51))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2759569888.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est51))


# 12065

In [252]:
# Filtrar activos_c1 donde SUBCUENTA sea '065'
subsetactivos = activos_c1[activos_c1['SUBCUENTA'] == '065']

# Sumar la columna 'Valor Mercado ($)', ignorando valores NA y redondear a entero
v12065 = int(round(subsetactivos['Valor Mercado ($)'].sum(skipna=True), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est52 = ["12", "065", v12065, "Este valor se toma de la hoja Activos, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est52 = ["12", "065", v12065, "Este valor se toma de la hoja Activos, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est52))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3160506619.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est52))


# 12070

In [253]:
# Obtener el valor de la celda (4,4) de INFORMACION_REASEGURO y convertirlo a numérico
v12070 = int(round(float(INFORMACION_REASEGURO.iloc[3, 3]), 0))  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est53 = ["12", "070", v12070, "Este valor se toma de la hoja Reaseguro, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est53 = ["12", "070", v12070, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est53))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\338655081.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est53))


# 12075

In [254]:
# Obtener el valor de la celda (4,5) de INFORMACION_REASEGURO y convertirlo a numérico
v12075 = int(round(float(INFORMACION_REASEGURO.iloc[3, 4]), 0))  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est54 = ["12", "075", v12075, "Este valor se toma de la hoja Reaseguro, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est54 = ["12", "075", v12075, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est54))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3187007549.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est54))


# 12080

In [255]:
# Recordar que en Python los índices empiezan en 0, así que serían [1,3] y [6,3]
v12080 = int(round(float(INFORMACION_REASEGURO.iloc[1, 3]) + float(INFORMACION_REASEGURO.iloc[6, 3]), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est55 = ["12", "080", v12080, "Este valor se toma de la hoja Reaseguro, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est55 = ["12", "080", v12080, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est55))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2924133878.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est55))


# 12085

In [256]:
# Recordar que en Python los índices empiezan en 0, así que serían [1,4] y [6,4]
v12085 = int(round(float(INFORMACION_REASEGURO.iloc[1, 4]) + float(INFORMACION_REASEGURO.iloc[6, 4]), 0))

# Crear una nueva fila para el DataFrame estructura
# Si estructura tiene más de 4 columnas, añadir None para las columnas extra
if len(estructura.columns) > 4:
    est56 = ["12", "085", v12085, "Este valor se toma de la hoja Reaseguro, como fuente de información"] + [None] * (len(estructura.columns) - 4)
else:
    est56 = ["12", "085", v12085, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Añadir la nueva fila al DataFrame estructura usando loc
estructura.loc[len(estructura)] = dict(zip(estructura.columns, est56))

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\718345574.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = dict(zip(estructura.columns, est56))


# 12125

In [257]:
# Función de redondeo especial para manejar NaN y valores no numéricos
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener los tres valores que forman v12125
v12125_1 = redondear_valor(Info.iloc[11, 2])  # Equivalente a Info$...2[6]
v12125_2 = sum(redondear_valor(val) for val in INFORMACION_COASEGURO.iloc[3:6, 5])  # Filas 1-3, columna 5 en R
v12125_3 = redondear_valor(INFORMACION_REASEGURO.iloc[1, 4])  # Fila 1, columna 5 en R

# Sumar los tres valores
v12125 = round(sum([v12125_1, v12125_2, v12125_3]))

# Crear nueva fila para estructura con valores None para las columnas adicionales
columnas_adicionales = len(estructura.columns) - 4
est57 = ["12", "0125", v12125, "Este valor se toma de la hoja Coaseguro, como fuente de información"]
if columnas_adicionales > 0:
    est57 += [None] * columnas_adicionales

# Agregar la nueva fila a estructura
estructura.loc[len(estructura)] = est57

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\125997448.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est57


# DATAFRAME

In [258]:
# Crear el DataFrame inicial
unidad12998_2 = pd.DataFrame({
    'Subcuenta': ['005', '010', '015', '020', '025', '030', '045',
                  '050', '055', '060', '065', '070', '075', '080',
                  '085', '125']
})

# Agregar la columna de Saldos
unidad12998_2['Saldos'] = [v12005, v12010, v12015, v12020, v12025, v12030,
                           v12045, v12050, v12055, v12060, v12065, v12070,
                           v12075, v12080, v12085, v12125]

# Crear copia del DataFrame
unidad12998 = unidad12998_2.copy()

# Calcular la suma redondeada de los saldos
suma_total = round(unidad12998['Saldos'].sum())

# Agregar la fila con la suma total
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_total]
})

# Concatenar la nueva fila
unidad12998 = pd.concat([unidad12998, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998['Saldos'] = pd.to_numeric(unidad12998['Saldos'])

# UNIDAD 12 (DETERIORADA)

# 12070d

In [259]:
# Función de redondeo especial para manejar NaN y valores no numéricos (si quieres usarla)
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener el valor de la celda (5,4) de INFORMACION_REASEGURO, convertirlo a numérico y hacerlo negativo
v12070d = -redondear_valor(INFORMACION_REASEGURO.iloc[4, 3])  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
est58 = ["12", "070", v12070d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
columnas_adicionales = len(estructura.columns) - 4
if columnas_adicionales > 0:
    est58 += [None] * columnas_adicionales

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est58

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2829191342.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est58


# 12075d

In [260]:
# Función de redondeo especial para manejar NaN y valores no numéricos (si quieres usarla)
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener el valor de la celda (5,5) de INFORMACION_REASEGURO, convertirlo a numérico y hacerlo negativo
v12075d = -redondear_valor(INFORMACION_REASEGURO.iloc[4, 4])  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
est59 = ["12", "075", v12075d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
columnas_adicionales = len(estructura.columns) - 4
if columnas_adicionales > 0:
    est59 += [None] * columnas_adicionales

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est59

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3858235495.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est59


# 12080d

In [261]:
# Función de redondeo especial para manejar NaN y valores no numéricos
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener el valor de la celda (7,4) de INFORMACION_REASEGURO, convertirlo a numérico y hacerlo negativo
v12080d = -redondear_valor(INFORMACION_REASEGURO.iloc[6, 3])  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
est60 = ["12", "080", v12080d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
columnas_adicionales = len(estructura.columns) - 4
if columnas_adicionales > 0:
    est60 += [None] * columnas_adicionales

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est60

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3287689423.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est60


# 12085d

In [262]:
# Función de redondeo especial para manejar NaN y valores no numéricos
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener el valor de la celda (7,5) de INFORMACION_REASEGURO, convertirlo a numérico y hacerlo negativo
v12085d = -redondear_valor(INFORMACION_REASEGURO.iloc[6, 4])  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
est61 = ["12", "085", v12085d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
columnas_adicionales = len(estructura.columns) - 4
if columnas_adicionales > 0:
    est61 += [None] * columnas_adicionales

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est61

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2507215206.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est61


# 12125d

In [263]:
# Función de redondeo especial para manejar NaN y valores no numéricos
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)  # Si el valor es muy grande, retorna 0
    except:
        return 0

# Obtener el valor de la celda (6,5) de INFORMACION_REASEGURO, convertirlo a numérico y hacerlo negativo
v12125d = -redondear_valor(INFORMACION_REASEGURO.iloc[5, 4])  # Recordar que en Python los índices empiezan en 0

# Crear una nueva fila para el DataFrame estructura
est62 = ["12", "0125", v12125d, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
columnas_adicionales = len(estructura.columns) - 4
if columnas_adicionales > 0:
    est62 += [None] * columnas_adicionales

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est62

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3083301776.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est62


# DATAFRAME

In [264]:
# Crear listas para las cuentas y saldos
cuentas12 = ["070", "075", "080", "085", "125"]
saldos12 = [v12070d, v12075d, v12080d, v12085d, v12125d]

# Crear el DataFrame
unidad12998_d = pd.DataFrame({
    'Subcuenta': cuentas12,
    'Saldos': saldos12
})

# Calcular la suma de los saldos
suma_saldos = unidad12998_d['Saldos'].sum()

# Agregar la nueva fila con la suma
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_saldos]
})

# Concatenar la nueva fila
unidad12998_d = pd.concat([unidad12998_d, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998_d['Saldos'] = pd.to_numeric(unidad12998_d['Saldos'])

# UNIDAD 12 (PONDERADA) 

In [265]:
# Función de redondeo especial para manejar NaN y valores no numéricos (si la usas)
def redondear_valor(x):
    try:
        if pd.isna(x) or x == '' or x is None:
            return 0
        val = float(x)
        return 0 if val >= 1e11 else round(val)
    except:
        return 0

# Calcular v12005p como el 1.5% de v12005 y redondear a entero
v12005p = int(round(v12005 * 0.015))
est63 = ["12", "005", v12005p, "Este valor se calcula de la subcuenta 005 de la unidad de captura 12 multiplicado por el 1,5%"]
# Completar la fila con None si el DataFrame tiene más de 4 columnas
if len(estructura.columns) > 4:
    est63 += [None] * (len(estructura.columns) - 4)
# Añadir la nueva fila a estructura
estructura.loc[len(estructura)] = est63

# Calcular v12010p como el 4.5% de v12010 y redondear a entero
v12010p = int(round(v12010 * 0.045))
est64 = ["12", "010", v12010p, "Este valor se calcula de la subcuenta 010 de la unidad de captura 12 multiplicado por el 4,5%"]
if len(estructura.columns) > 4:
    est64 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est64

# Calcular v12015p como el 8.5% de v12015 y redondear a entero
v12015p = int(round(v12015 * 0.085))
est65 = ["12", "015", v12015p, "Este valor se calcula de la subcuenta 015 de la unidad de captura 12 multiplicado por el 8,5%"]
if len(estructura.columns) > 4:
    est65 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est65

# Calcular v12020p como el 1.5% de v12020 y redondear a entero
v12020p = int(round(v12020 * 0.015))
est66 = ["12", "020", v12020p, "Este valor se calcula de la subcuenta 020 de la unidad de captura 12 multiplicado por el 1,5%"]
if len(estructura.columns) > 4:
    est66 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est66

# Calcular v12025p como el 4.5% de v12025 y redondear a entero
v12025p = int(round(v12025 * 0.045))
est67 = ["12", "025", v12025p, "Este valor se calcula de la subcuenta 025 de la unidad de captura 12 multiplicado por el 4,5%"]
if len(estructura.columns) > 4:
    est67 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est67

# Calcular v12030p como el 8.5% de v12030 y redondear a entero
v12030p = int(round(v12030 * 0.085))
est68 = ["12", "030", v12030p, "Este valor se calcula de la subcuenta 030 de la unidad de captura 12 multiplicado por el 8,5%"]
if len(estructura.columns) > 4:
    est68 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est68

# Calcular v12045p como el 1.5% de v12045 y redondear a entero
v12045p = int(round(v12045 * 0.015))
est69 = ["12", "045", v12045p, "Este valor se calcula de la subcuenta 045 de la unidad de captura 12 multiplicado por el 1,5%"]
if len(estructura.columns) > 4:
    est69 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est69

# Calcular v12050p como el 4.5% de v12050 y redondear a entero
v12050p = int(round(v12050 * 0.045))
est70 = ["12", "050", v12050p, "Este valor se calcula de la subcuenta 050 de la unidad de captura 12 multiplicado por el 4,5%"]
if len(estructura.columns) > 4:
    est70 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est70

# Calcular v12055p como el 8.5% de v12055 y redondear a entero
v12055p = int(round(v12055 * 0.085))
est71 = ["12", "050", v12055p, "Este valor se calcula de la subcuenta 055 de la unidad de captura 12 multiplicado por el 8,5%"]
if len(estructura.columns) > 4:
    est71 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est71  # Nota que hay un error en el subcuenta (050 en lugar de 055)

# Calcular v12060p como el 4.5% de v12060 y redondear a entero
v12060p = int(round(v12060 * 0.045))
est72 = ["12", "060", v12060p, "Este valor se calcula de la subcuenta 060 de la unidad de captura 12 multiplicado por el 4,5%"]
if len(estructura.columns) > 4:
    est72 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est72

# Calcular v12065p como el 4.5% de v12065 y redondear a entero
v12065p = int(round(v12065 * 0.045))
est73 = ["12", "065", v12065p, "Este valor se calcula de la subcuenta 065 de la unidad de captura 12 multiplicado por el 4,5%"]
if len(estructura.columns) > 4:
    est73 += [None] * (len(estructura.columns) - 4)
estructura.loc[len(estructura)] = est73

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\4278915399.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est63
C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\4278915399.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est64
C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\4278915399.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

# 12070p

In [266]:
# Primero filtramos bd_reaseguros para obtener solo los registros donde Calificado == 'CALIFICADO'
reasegurosubset = bd_reaseguros[bd_reaseguros['Calificado'] == 'CALIFICADO']

# Luego sumamos la columna Ponderación_Activo, ignorando valores NA
v12070p = int(round(reasegurosubset['Ponderación_Activo'].astype(float).sum(skipna=True)))

# Creamos la nueva fila para el DataFrame estructura
est73 = ["12", "070", v12070p, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadimos None para completar
if len(estructura.columns) > 4:
    est73 += [None] * (len(estructura.columns) - 4)

# Añadimos la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est73

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1475440995.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est73


# 12075p

In [267]:
# Calcular v12075p 
# Resta el valor de unidad12998_d fila 2, columna Saldos del valor de unidad12998 fila 13, columna Saldos
# y multiplica el resultado por 0.085
v12075p = int(round((float(unidad12998.loc[12, 'Saldos']) - float(unidad12998_d.loc[1, 'Saldos'])) * 0.085))

# Crear la nueva fila para el DataFrame estructura
est74 = ["12", "075", v12075p, "Este valor se calcula de la subcuenta 075 de la unidad de captura 12 multiplicado por el 8,5%"]

# Si estructura tiene más columnas, añadir None para completar
if len(estructura.columns) > 4:
    est74 += [None] * (len(estructura.columns) - 4)

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est74

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3834779246.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est74


# 12080p

In [268]:
# Suma los valores de la columna 'Ponderación_cta_cte' del DataFrame reasegurosubset
# ignorando valores NA y redondeando a entero
v12080p = int(round(reasegurosubset['Ponderación_cta_cte'].astype(float).sum(skipna=True)))

# Crear la nueva fila para el DataFrame estructura
est75 = ["12", "080", v12080p, "Este valor se toma de la hoja Reaseguro, como fuente de información"]

# Si estructura tiene más columnas, añadir None para completar
if len(estructura.columns) > 4:
    est75 += [None] * (len(estructura.columns) - 4)

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est75

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2293093940.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est75


# 12085p

In [269]:
# Calcular v12085p
# Resta el valor de unidad12998_d fila 4, columna Saldos del valor de unidad12998 fila 15, columna Saldos
# y multiplica el resultado por 0.085
v12085p = int(round((float(unidad12998.loc[14, 'Saldos']) - float(unidad12998_d.loc[3, 'Saldos'])) * 0.085))

# Crear la nueva fila para el DataFrame estructura
est76 = ["12", "085", v12085p, "Este valor se calcula de la subcuenta 085 de la unidad de captura 12 multiplicado por el 8,5%"]

# Si estructura tiene más columnas, añadir None para completar
if len(estructura.columns) > 4:
    est76 += [None] * (len(estructura.columns) - 4)

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est76

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\426775204.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est76


# 12125p

In [270]:
# Calcular v12125p
# Resta el valor de unidad12998_d fila 5, columna Saldos del valor de unidad12998 fila 16, columna Saldos
# y multiplica el resultado por 0.085
v12125p = int(round((float(unidad12998.loc[15, 'Saldos']) - float(unidad12998_d.loc[4, 'Saldos'])) * 0.085))

# Crear la nueva fila para el DataFrame estructura
est77 = ["12", "125", v12125p, "Este valor se calcula de la subcuenta 125 de la unidad de captura 12 multiplicado por el 8,5%"]

# Si estructura tiene más columnas, añadir None para completar
if len(estructura.columns) > 4:
    est77 += [None] * (len(estructura.columns) - 4)

# Añadir la nueva fila al DataFrame estructura
estructura.loc[len(estructura)] = est77

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\2904315130.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura.loc[len(estructura)] = est77


# DATAFRAME

In [271]:
# Crear DataFrame unidad12998_p
subcuentas = ['005', '010', '015', '020', '025', '030', '045',
              '050', '055', '060', '065', '070', '075', '080', 
              '085', '125']

saldos = [v12005p, v12010p, v12015p, v12020p, v12025p, v12030p,
          v12045p, v12050p, v12055p, v12060p, v12065p, v12070p,
          v12075p, v12080p, v12085p, v12125p]

unidad12998_p = pd.DataFrame({
    'Subcuenta': subcuentas,
    'Saldos': saldos
})

# Calcular la suma de saldos
suma_saldos = unidad12998_p['Saldos'].sum()

# Agregar la fila con la suma
nueva_fila = pd.DataFrame({
    'Subcuenta': ['998'],
    'Saldos': [suma_saldos]
})

# Concatenar la nueva fila
unidad12998_p = pd.concat([unidad12998_p, nueva_fila], ignore_index=True)

# Asegurar que Saldos sea numérico
unidad12998_p['Saldos'] = pd.to_numeric(unidad12998_p['Saldos'])

# UNIDAD 13 

# v13005

In [272]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[26]['Unnamed: 2']
v13005 = int(round(float(0 if pd.isna(valor) else valor)))

# Crear nueva fila para estructura con la longitud adecuada
est78 = ["13", "005", v13005, "Este valor se toma de la hoja Información, como fuente de información"]

# Asegurar que est78 tenga la misma cantidad de columnas que estructura
if len(estructura.columns) > len(est78):
    # Añadir None para las columnas adicionales
    est78.extend([None] * (len(estructura.columns) - len(est78)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est78], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3125506584.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est78], columns=estructura.columns)], ignore_index=True)


# v13025

In [273]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[28]['Unnamed: 2']  # Ajustado para índice 13 (fila 14 en R)
v13025 = float(0 if pd.isna(valor) else valor)

# Crear nueva fila para estructura
est79 = ["13", "025", v13025, "Este valor se toma de la hoja Información, como fuente de información"]

# Asegurar que est79 tenga la misma cantidad de columnas que estructura
if len(estructura.columns) > len(est79):
    # Añadir None para las columnas adicionales
    est79.extend([None] * (len(estructura.columns) - len(est79)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est79], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3809168266.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est79], columns=estructura.columns)], ignore_index=True)


# DATAFRAME

In [274]:
# Crear listas para subcuentas y saldos
cuentas13 = ['005', '025']
saldos13 = [v13005, v13025]

# Crear DataFrame
unidad13998 = pd.DataFrame({
    'Subcuenta': cuentas13,
    'Saldos': saldos13
})

# UNIDAD 14

In [275]:
# Obtener el valor usando el nombre correcto de la columna y manejar NaN
valor = Info.iloc[21]['Unnamed: 2']
# Manejar NaN y aplicar redondeo
if pd.isna(valor):
    v14005 = float(0)
else:
    v14005 = round(float(valor))  # Esto redondeará automáticamente según regla .5

# Crear nueva fila para estructura con el número correcto de columnas
est68 = ["14", "005", v14005, "Este valor se toma de la hoja Información, como fuente de información"]
# Añadir valores None para las columnas restantes
est68.extend([None] * (len(estructura.columns) - len(est68)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est68], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1481578946.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est68], columns=estructura.columns)], ignore_index=True)


# 14010

In [276]:
# Sumar los valores de diversas subcuentas, asegurándose de manejar valores NaN
# y redondear a enteros

# Definir una función para manejar NaN y redondear valores
def valor_seguro(df, columna, indice):
    try:
        valor = df.loc[indice, columna]
        if pd.isna(valor):
            return 0
        return int(round(float(valor)))
    except:
        return 0

# Calcular v14010 sumando los valores de varias subcuentas
v14010 = (
    valor_seguro(unidad9998, 'Saldos', 2) +  # Índice 2 para fila 3 en R
    valor_seguro(unidad9998pi, 'Saldospi', 2) +
    valor_seguro(unidad9998rp, 'Saldosrp', 3) +  # Índice 3 para fila 4 en R
    valor_seguro(unidad9998pl, 'Saldospl', 1) +  # Índice 1 para fila 2 en R
    valor_seguro(unidad9998pc, 'Saldospc', 1) +
    valor_seguro(unidad9998vi, 'Saldosvi', 1)
)

# Crear nueva fila para estructura
est81 = ["14", "010", v14010, "Este valor se calcula de los totales de las subcuentas 998 de la unidad de captura 9"]

# Asegurar que est81 tenga la misma cantidad de columnas que estructura
if len(estructura.columns) > len(est81):
    # Añadir None para las columnas adicionales
    est81.extend([None] * (len(estructura.columns) - len(est81)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est81], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\214871060.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est81], columns=estructura.columns)], ignore_index=True)


# 14015

In [277]:
# Sumar los valores de las subcuentas especificadas, manejando valores NaN

# Definir una función para manejar valores NaN de forma segura
def valor_seguro(df, columna, indice):
    try:
        valor = df.loc[indice, columna]
        if pd.isna(valor):
            return 0
        return float(valor)
    except:
        return 0

# Calcular v14015 sumando los valores de las subcuentas
v14015 = int(round(
    valor_seguro(unidad11998_p, 'Saldos', 7) +  # Índice 7 para fila 8 en R
    valor_seguro(unidad12998_p, 'Saldos', 16)   # Índice 16 para fila 17 en R
))

# Crear nueva fila para estructura
est82 = ["14", "015", v14015, "Este valor se calcula de los totales de las subcuentas 998 de las unidades de captura 10,11 y 12"]

# Asegurar que est82 tenga la misma cantidad de columnas que estructura
if len(estructura.columns) > len(est82):
    # Añadir None para las columnas adicionales
    est82.extend([None] * (len(estructura.columns) - len(est82)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est82], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1118464294.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est82], columns=estructura.columns)], ignore_index=True)


# 14020

In [278]:
v14020 = 1

# 14025

In [279]:
v14025 = 1

# 14030

In [280]:
v14030 = 1

# 14035

In [281]:
# Cálculo de v14035 y truncamiento a entero
v14035 = int(np.sqrt((v14010**2) + 
                    2*(v14010*v14015*v14025) + 
                    2*(v14010*v14005*v14020) + 
                    2*(v14015*v14005*v14030) + 
                    (v14015**2) + 
                    (v14005**2)))

# Creación del vector est71
est83 = ["14", "035", int(v14015), "Este valor se calcula sumando los demás valores de la unidad de captura 14"]

# Si estructura es un DataFrame, asegurar que est71 tenga el número correcto de columnas
est83.extend([None] * (len(estructura.columns) - len(est83)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est83], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1604078174.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est83], columns=estructura.columns)], ignore_index=True)


# 14040

In [282]:
# Cálculo de v14040
v14040 = int(round(v13005 * v13025))  # Multiplicar y redondear a entero

# Crear el vector est84
est84 = ["14", "040", v14040, "Este valor se calcula sumando las subcuentas 015 y 020 de la unidad de captura 13"]

# Añadir valores None para las columnas restantes si es necesario
est84.extend([None] * (len(estructura.columns) - len(est84)))

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([est84], columns=estructura.columns)], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3204902658.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([est84], columns=estructura.columns)], ignore_index=True)


# 14998

In [283]:
# Sumar v14035 y v14040
v14998 = v14035 + v14040

# DATAFRAME

In [284]:
# Crear las listas de cuentas y saldos
cuentas14 = ['005', '010', '015', '020', '025', '030', '035', '040', '998']
saldos14 = [int(v14005), int(v14010), int(v14015), int(v14020), int(v14025), 
            int(v14030), int(v14035), int(v14040), int(v14998)]

# Crear el DataFrame
unidad14998 = pd.DataFrame({
    'Subcuenta': cuentas14,
    'Saldos': saldos14
})

#  UNIDAD 15 

In [285]:
# Cálculo de v15005 usando el último elemento de cada DataFrame y convirtiéndolo a entero
v15005 = int(unidad1998['Saldos'].iloc[-1] - unidad2998['Saldos'].iloc[-1])

# Crear el vector est85
est85 = ["15", "005", v15005, "Este valor se calcula tomando el total del subcuenta 998 de la unidad de captura 01 y restandole el total de la subcuenta 998 de la unidad de captura 02"]

# Si estructura es una lista de listas
if not isinstance(estructura, pd.DataFrame):
    # Asumiendo que todas las filas tienen la misma longitud
    estructura = pd.DataFrame(estructura)

# Asegurarse de que est85 tenga la misma longitud que las columnas
while len(est85) < len(estructura.columns):
    est85.append(None)
    
# Agregar la nueva fila
estructura = pd.concat([estructura, pd.DataFrame([est85])], ignore_index=True)

# 15010

In [286]:
v15010 = 0

# 15015

In [287]:
v15015 = min(unidad4998.iloc[0]['Saldos'], unidad14998.iloc[8]['Saldos'] * 0.15)

# 15020

In [288]:
v15020 = 0

# 15998

In [289]:
# Calcular v15998
v15998 = v15005 + v15010 + v15015 - v15020  # Esto te da np.int64(264594263286)

# Crear un diccionario con None para todas las columnas
nueva_fila = {col: None for col in estructura.columns}

# Actualizar el diccionario con los valores conocidos
nueva_fila.update({
    'nombre_columna1': "15",
    'nombre_columna2': "010",
    'nombre_columna3': v15998,  # Aquí usas el np.int64 directamente
    'nombre_columna4': "Este valor se calcula sumando los demás valores de la unidad de captura 15"
})

# Agregar la nueva fila a estructura
estructura = pd.concat([estructura, pd.DataFrame([nueva_fila])], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3581640664.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, pd.DataFrame([nueva_fila])], ignore_index=True)


In [290]:
v15998

np.int64(287652642852)

# DATAFRAME

In [291]:
# Crear listas con los datos
cuentas15 = ["005", "010", "015", "998"]
saldos15 = [v15005, v15010, v15015, v15998]

# Crear DataFrame
unidad15998 = pd.DataFrame({
    'Subcuenta': cuentas15,
    'Saldos': saldos15
})

# UNIDAD 16 

In [292]:
# Calcular v16005
v16005 = v15998 - v14998

# Crear un nuevo DataFrame con una sola fila
nueva_fila = pd.DataFrame([{
    estructura.columns[0]: "15",
    estructura.columns[1]: "005", 
    estructura.columns[2]: int(v16005),
    estructura.columns[3]: "Este valor se calcula sumando los totales de la unidad de captura 998 de la unidades de captura 14 y 15"
}])

# Completar con NaN los valores faltantes
for col in estructura.columns:
    if col not in nueva_fila.columns:
        nueva_fila[col] = None

# Ahora concatenar
estructura = pd.concat([estructura, nueva_fila], ignore_index=True)

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\1938450998.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  estructura = pd.concat([estructura, nueva_fila], ignore_index=True)


# 16010

In [293]:
v16010 = 0

# 16015

In [294]:
v16015 = v16010 * 0.035

# DATAFRAME

In [295]:
# Crear listas con los datos
cuentas16 = ["005", "010", "015"]
saldos16 = [v16005, v16010, v16015]

# Crear DataFrame
unidad16998 = pd.DataFrame({
    'Subcuenta': cuentas16,
    'Saldos': saldos16
})

# Convertir los valores a enteros (si todos son enteros)
unidad16998['Saldos'] = unidad16998['Saldos'].astype(int)

# O alternativamente, si prefieres mantener los valores como están pero formatearlos como cadenas sin decimales:
# unidad16998['Saldos'] = unidad16998['Saldos'].apply(lambda x: f"{int(x)}")

# IMPRIMIR FORMATO 407

In [296]:
# Método directo usando la posición
Utilidad_del_ejercicio = BALANCE.iloc[1158, 6]  # Ajustado para indexación base 0

# O usando el método de filtrado (como alternativa)
BalanceSubset = BALANCE[BALANCE['Pos_balance'].str.contains('865', na=False)]

# Si hay resultados en el filtrado, usar el primer valor
if len(BalanceSubset) > 0:
    Utilidad_del_ejercicio = BalanceSubset['Total_periodos_de_informe'].iloc[0]

# Escribir en Excel usando openpyxl
wb['Hoja1'].cell(row=23, column=9, value=Utilidad_del_ejercicio)

<Cell 'Hoja1'.I23>

In [297]:
# Limpiar nombres de columnas (eliminar espacios extra)
BALANCE.columns = BALANCE.columns.str.strip()

# Buscar la columna que contenga 'Pos_balance' y 'Total_periodos_de_informe' de forma flexible
columna_pos_balance = next((col for col in BALANCE.columns if 'Pos_balance' in col), None)
columna_total_periodos = next((col for col in BALANCE.columns if 'Total_periodos_de_informe' in col), None)
columna_texto_pos = next((col for col in BALANCE.columns if 'Texto_ posicion_balance_PyG' in col), None)

if not columna_pos_balance or not columna_total_periodos or not columna_texto_pos:
    raise ValueError("No se encontraron las columnas necesarias en BALANCE.")

# Filtrar por 'Pos_balance' que contenga '865'
BalanceSubset = BALANCE[BALANCE[columna_pos_balance].astype(str).str.contains('865', na=False)]

# Filtrar por 'Texto_ posicion_balance_PyG' que sea 'RESULTADOS DEL EJERCICIO UTILIDAD'
BalanceSubset = BalanceSubset[BalanceSubset[columna_texto_pos].str.strip().str.lower() == 'resultados del ejercicio utilidad']

# Obtener el primer valor de 'Total_periodos_de_informe' si existe
Utilidad_del_ejercicio = BalanceSubset[columna_total_periodos].iloc[0] if not BalanceSubset.empty else None

# Escribir en Excel en la celda específica
hoja = wb['Hoja1']
hoja.cell(row=23, column=9, value=Utilidad_del_ejercicio)

<Cell 'Hoja1'.I23>

#  PATRIMONIO BÁSICO ORDINARIO ANTES DE DEDUCCIONES

In [298]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad1998 existe
if 'unidad1998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 23, "010": 24, "015": 25, "020": 26, "025": 27,
        "030": 28, "035": 29, "040": 30, "045": 31, "050": 32,
        "055": 33, "060": 34, "065": 35, "070": 36, "075": 37,
        "080": 38, "085": 39, "090": 40, "095": 41, "100": 42,
        "105": 43, "110": 44, "998": 45
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad1998[unidad1998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)  # valor por defecto en caso de error


# DEDUCCIONES DEL PATRIMONIO BÁSICO ORDINARIO

In [299]:
# Verificar si "unidad2998" está en la lista de consultas
if "unidad2998" in consultas:

    # Función auxiliar para procesar cada subcuenta
    def procesar_subcuenta(subcuenta, fila, desde_info=False, info_fila=None, info_col=None, verbose=False):
        try:
            sheet = wb["Hoja1"]
            
            if desde_info and info_fila is not None and info_col is not None:
                # Extraer valor desde Info
                try:
                    valor_str = str(Info.iloc[info_fila, info_col])
                    valor_str = valor_str.replace(',', '')  # Eliminar comas
                    valor = float(valor_str)
                    valor = round(valor, 0)
                    if not pd.isna(valor):  # Verificar que no sea NaN
                        sheet.cell(row=fila, column=4, value=int(valor))
                        if verbose:
                            print(f"Subcuenta {subcuenta} (desde Info): {int(valor)}")
                    else:
                        if verbose:
                            print(f"El valor para subcuenta {subcuenta} desde Info es NaN")
                except Exception as e:
                    if verbose:
                        print(f"Error al procesar subcuenta {subcuenta} desde Info: {e}")
                    # Intentar obtener desde unidad2998 como fallback
                    try:
                        if any(unidad2998['Subcuenta'] == subcuenta):
                            valor = unidad2998.loc[unidad2998['Subcuenta'] == subcuenta, 'Saldos'].iloc[0]
                            valor = round(valor, 0)
                            if not pd.isna(valor):  # Verificar que no sea NaN
                                sheet.cell(row=fila, column=4, value=int(valor))
                                if verbose:
                                    print(f"Subcuenta {subcuenta} (desde unidad2998): {int(valor)}")
                            else:
                                if verbose:
                                    print(f"El valor para subcuenta {subcuenta} desde unidad2998 es NaN")
                    except:
                        if verbose:
                            print(f"No se pudo obtener valor para subcuenta {subcuenta} de ninguna fuente")
            else:
                # Extraer valor desde unidad2998
                if any(unidad2998['Subcuenta'] == subcuenta):
                    valor = unidad2998.loc[unidad2998['Subcuenta'] == subcuenta, 'Saldos'].iloc[0]
                    valor = round(valor, 0)
                    if not pd.isna(valor):  # Verificar que no sea NaN
                        sheet.cell(row=fila, column=4, value=int(valor))
                        if verbose:
                            print(f"Subcuenta {subcuenta}: {int(valor)}")
                    else:
                        if verbose:
                            print(f"El valor para subcuenta {subcuenta} es NaN")
                else:
                    if verbose:
                        print(f"No se encontró subcuenta {subcuenta}")
        except Exception as e:
            if verbose:
                print(f"Error general al procesar subcuenta {subcuenta}: {e}")

    # Procesar todas las subcuentas sin imprimir mensajes (verbose=False)
    procesar_subcuenta('005', 48, verbose=False)
    procesar_subcuenta('010', 49, verbose=False)
    procesar_subcuenta('015', 50, verbose=False)
    procesar_subcuenta('020', 51, verbose=False)
    procesar_subcuenta('025', 52, verbose=False)
    procesar_subcuenta('030', 53, verbose=False)
    procesar_subcuenta('035', 54, verbose=False)
    procesar_subcuenta('040', 55, verbose=False)
    
    # Para 045, usar Info[12,2] (ajustado a indexación de Python)
    procesar_subcuenta('045', 56, desde_info=True, info_fila=11, info_col=1, verbose=False)
    
    procesar_subcuenta('050', 57, verbose=False)
    procesar_subcuenta('055', 58, verbose=False)
    procesar_subcuenta('060', 59, verbose=False)
    procesar_subcuenta('065', 60, verbose=False)
    procesar_subcuenta('070', 61, verbose=False)
    procesar_subcuenta('075', 62, verbose=False)
    procesar_subcuenta('998', 63, verbose=False)

# PATRIMONIO BÁSICO ADICIONAL (PBA)

In [300]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad3998 existe
if 'unidad3998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 66, "010": 67, "015": 68, "020": 69, "025": 70,
        "030": 71, "035": 72, "040": 73, "045": 74, "050": 75,
        "999": 76
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad3998[unidad3998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)  # valor por defecto en caso de error    


# PATRIMONIO ADICIONAL (PA)

In [301]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad4998 existe
if 'unidad4998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 79, "010": 80, "015": 81, "020": 82, 
        "025": 83, "030": 84, "999": 85
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad4998[unidad4998['Subcuenta'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['Saldos'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=4, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=4, value=0)  # valor por defecto en caso de error


#   PRIMAS DEMAS RAMOS 

In [302]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad5998 existe
if 'unidad5998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 93, "010": 94, "015": 95, "020": 96,
        "025": 97, "030": 98, "035": 99, "998": 100
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            # Nota: Aquí usamos 'subcuentas05' y 'saldos05' en lugar de 'Subcuenta' y 'Saldos'
            filtered_data = unidad5998[unidad5998['subcuentas05'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos05'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=15 en lugar de column=4)
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)  # valor por defecto en caso de error
 

#  PRIMAS PREVISIONALES DE INVALIDEZ

In [303]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad5998pi existe
if 'unidad5998pi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 93, "010": 94, "015": 95, "020": 96,
        "025": 97, "035": 98, "040": 99, "998": 100
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            # Nota: Aquí usamos 'subcuentas05pi' y 'saldos05pi' como nombres de columnas
            filtered_data = unidad5998pi[unidad5998pi['subcuentas05pi'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos05pi'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=14)
            wb['Hoja1'].cell(row=row, column=14, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=14, value=0)  # valor por defecto en caso de error


#  PRIMAS RIESGOS PROFESIONALES 

In [304]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad5998rp existe
if 'unidad5998rp' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 93, "010": 94, "015": 95, "020": 96,
        "025": 97, "035": 98, "040": 99, "998": 100
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            # Nota: Aquí usamos 'subcuentas05rp' y 'saldos05rp' como nombres de columnas
            filtered_data = unidad5998rp[unidad5998rp['subcuentas05rp'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos05rp'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=13)
            wb['Hoja1'].cell(row=row, column=13, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=13, value=0)  # valor por defecto en caso de error
    

# SINIESTROS DEMAS RAMOS

In [305]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad6998 existe
if 'unidad6998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 102, "010": 103, "015": 104, "020": 105, "025": 106,
        "030": 107, "035": 108, "040": 109, "045": 110, "050": 111,
        "055": 112, "060": 113, "065": 114, "070": 115, "075": 116,
        "080": 117, "085": 118, "090": 119, "095": 120, "100": 121,
        "105": 122, "110": 123, "115": 124, "120": 125
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad6998[unidad6998['subcuentas06'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos06'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=15)
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)  # valor por defecto en caso de error


# SINIESTROS PREVISIONES DE INVALIDEZ

In [306]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad6998pi existe
if 'unidad6998pi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 102, "010": 103, "015": 104, "020": 105, "025": 106,
        "030": 107, "035": 108, "040": 109, "045": 110, "050": 111,
        "055": 112, "060": 113, "065": 114, "070": 115, "075": 116,
        "080": 117, "085": 118, "090": 119, "095": 120, "100": 121,
        "105": 122, "110": 123, "115": 124, "120": 125
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad6998pi[unidad6998pi['subcuentas06pi'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos06pi'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=14)
            wb['Hoja1'].cell(row=row, column=14, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=14, value=0)  # valor por defecto en caso de error


#  SINIESTROS RIESGOS PROFESIONALES 

In [307]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad6998rp existe
if 'unidad6998rp' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 102, "010": 103, "015": 104, "020": 105, "025": 106,
        "030": 107, "035": 108, "040": 109, "045": 110, "050": 111,
        "055": 112, "060": 113, "065": 114, "070": 115, "075": 116,
        "080": 117, "085": 118, "090": 119, "095": 120, "100": 121,
        "105": 122, "110": 123, "115": 124, "120": 125
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad6998rp[unidad6998rp['subcuentas06rp'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos06rp'].iloc[0]
                # Redondear el valor
                valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
                
            # Escribir en Excel (nota: aquí usamos column=13)
            wb['Hoja1'].cell(row=row, column=13, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=13, value=0)  # valor por defecto en caso de error


#  RELACIÓN A APLICAR A PRIMAS Y SINIESTROS DEMAS RAMOS

In [308]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad7998 existe
if 'unidad7998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 127, "010": 128, "015": 129, "020": 130, "025": 131,
        "030": 132, "035": 133, "040": 134, "045": 135, "050": 136,
        "055": 137, "060": 138
    }
    
    # Definir cuáles subcuentas necesitan ser redondeadas
    redondear = {"005", "010", "020", "025"}
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad7998[unidad7998['subcuentas07'] == subcuenta]
            
            # Verificar si hay datos para esta subcuenta
            if len(filtered_data) > 0:
                valor = filtered_data['saldos07'].iloc[0]
                
                # Redondear solo si es necesario
                if subcuenta in redondear:
                    valor = round(valor, 0)
            else:
                valor = 0  # valor por defecto cuando no hay datos
            
            # Escribir en Excel
            wb['Hoja1'].cell(row=row, column=15, value=valor)
            
        except Exception:
            wb['Hoja1'].cell(row=row, column=15, value=0)  # valor por defecto en caso de error


#  RELACIÓN A APLICAR A PRIMAS Y SINIESTROS PREVISIONES E INVALIDEZ

In [309]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad7998pi existe
if 'unidad7998pi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 127, "010": 128, "015": 129, "020": 130, "025": 131,
        "030": 132, "035": 133, "040": 134, "045": 135, "050": 136,
        "055": 137, "060": 138
    }
    
    # Definir cuáles subcuentas necesitan ser redondeadas
    # Basado en el código original donde algunas líneas de redondeo estaban comentadas
    redondear = {"005", "010", "020", "025"}
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad7998pi[unidad7998pi['subcuentas07pi'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos07pi'].values})
                
                # Redondear solo si es necesario
                if subcuenta in redondear:
                    df['value'] = df['value'].round(0)
                    
                # Escribir en Excel
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=14, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=14, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto sin imprimir el error
            wb['Hoja1'].cell(row=row, column=14, value=0)

#  RELACIÓN A APLICAR A PRIMAS Y SINIESTROS RIESGOS PROFESIONALES

In [310]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad7998rp existe
if 'unidad7998rp' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 127, "010": 128, "015": 129, "020": 130, "025": 131,
        "030": 132, "035": 133, "040": 134, "045": 135, "050": 136,
        "055": 137, "060": 138
    }
    
    # Definir cuáles subcuentas necesitan ser redondeadas
    redondear = {"005", "010", "020", "025"}
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad7998rp[unidad7998rp['subcuentas07rp'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos07rp'].values})
                
                # Redondear solo si es necesario
                if subcuenta in redondear:
                    df['value'] = df['value'].round(0)
                    
                # Escribir en Excel
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=13, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=13, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=13, value=0)

#  RESERVA MATEMÁTICA RIESGOS PROFESIONALES

In [311]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad8998rp existe
if 'unidad8998rp' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 140, "010": 141, "015": 142, "020": 143, "025": 144
    }
    
    # En este caso, todas las subcuentas se redondean
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad8998rp[unidad8998rp['subcuentas08rp'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos08rp'].values})
                
                # Redondear el valor (todas las subcuentas se redondean en este caso)
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=13, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=13, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=13, value=0)

# RESERVA MATEMÁTICA PENSIONES LEY 100

In [312]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad8998pl existe
if 'unidad8998pl' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 140, "010": 141, "015": 142, "020": 143, "025": 144
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad8998pl[unidad8998pl['subcuentas08pl'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos08pl'].values})
                
                # Redondear todos los valores (como en el código original)
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - nota que aquí usamos columna 11
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=11, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=11, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=11, value=0)

# RESERVA MATEMÁTICA PENSIONES CON CONMUTACION

In [313]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad8998pc existe
if 'unidad8998pc' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 140, "010": 141, "015": 142, "020": 143, "025": 144
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad8998pc[unidad8998pc['subcuentas08pc'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos08pc'].values})
                
                # Redondear todos los valores (como en el código original)
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - nota que aquí usamos columna 10
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=10, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=10, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=10, value=0)

# RESERVA MATEMÁTICA VIDA INDIVIDUAL

In [314]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad8998vi existe
if 'unidad8998vi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 140, "010": 141, "015": 142, "020": 143, "025": 144
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad8998vi[unidad8998vi['subcuentas08vi'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['saldos08vi'].values})
                
                # Redondear todos los valores (como en el código original)
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - nota que aquí usamos columna 9
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=9, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=9, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=9, value=0)

#  RIESGO DE SUSCRIPCION DEMAS RAMOS

In [315]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998 existe
if 'unidad9998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998[unidad9998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Nota: en el código original, todos los redondeos están comentados
                # así que aquí no aplicamos redondeo
                    
                # Escribir en Excel - usamos columna 15
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=15, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=15, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=15, value=0)

# RIESGO DE SUSCRIPCION PREVISIONALES DE INVALIDEZ

In [316]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998pi existe
if 'unidad9998pi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998pi[unidad9998pi['Subcuentapi'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldospi'].values})
                
                # Nota: en el código original, todos los redondeos están comentados
                # así que aquí no aplicamos redondeo
                    
                # Escribir en Excel - usamos columna 14
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=14, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=14, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=14, value=0)

# RIESGO DE SUSCRIPCION RIESGOS PROFESIONALES 

In [317]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998rp existe
if 'unidad9998rp' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998rp[unidad9998rp['Subcuentarp'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldosrp'].values})
                
                # Nota: en el código original, todos los redondeos están comentados
                # así que aquí no aplicamos redondeo
                    
                # Escribir en Excel - usamos columna 13
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=13, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=13, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=13, value=0)

# RIESGO DE SUSCRIPCION PENSIONES LEY 100

In [318]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998pl existe
if 'unidad9998pl' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998pl[unidad9998pl['Subcuentapl'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldospl'].values})
                
                # Nota: en el código original, todos los redondeos están comentados
                # así que aquí no aplicamos redondeo
                    
                # Escribir en Excel - usamos columna 11
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=11, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=11, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=11, value=0)

#   RIESGO DE SUSCRIPCION PENSIONES CON CONMUTACION 

In [319]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998pc existe
if 'unidad9998pc' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Definir qué subcuentas necesitan redondeo
    redondear = {"015", "998"}  # Solo estas dos subcuentas tienen el redondeo activado en el código original
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998pc[unidad9998pc['Subcuentapc'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldospc'].values})
                
                # Aplicar redondeo solo a las subcuentas especificadas
                if subcuenta in redondear:
                    df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 10
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=10, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=10, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=10, value=0)

# RIESGO DE SUSCRIPCION VIDA INDIVIDUAL

In [320]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad9998vi existe
if 'unidad9998vi' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 146, "010": 147, "015": 148, "998": 149
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad9998vi[unidad9998vi['Subcuentavi'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldosvi'].values})
                
                # Nota: en el código original, todos los redondeos están comentados
                # así que aquí no aplicamos redondeo
                    
                # Escribir en Excel - usamos columna 9
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=9, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=9, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=9, value=0)

# CATEGORÍA I 

In [321]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad10998 existe
if 'unidad10998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 156, "010": 157, "015": 158, "020": 159, "025": 160,
        "030": 161, "035": 162, "040": 163, "045": 164, "050": 165,
        "055": 166, "060": 167, "065": 168, "999": 169
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad10998[unidad10998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=4, value=0)

# CATEGORÍA II

In [322]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad11998 existe
if 'unidad11998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 171, "010": 172, "015": 173, "020": 174, "025": 175,
        "030": 176, "035": 177, "040": 178, "045": 179, "050": 180,
        "998": 181
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad11998[unidad11998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=4, value=0)

#  CATEGORÍA II PONDERADO 

In [323]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad11998_p existe
if 'unidad11998_p' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 171, "010": 172, "015": 173, "020": 174, "025": 175,
        "030": 176, "035": 177, "040": 178, "045": 179, "050": 180,
        "998": 181
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad11998_p[unidad11998_p['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 7
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=7, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=7, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=7, value=0)

# CATEGORÍA III

In [324]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998 existe
if 'unidad12998' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186, "025": 187,
        "030": 188, "035": 189, "040": 190, "045": 191, "050": 192,
        "055": 193, "060": 194, "065": 195, "070": 196, "075": 197,
        "080": 198, "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206, "125": 207,
        "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998[unidad12998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=4, value=0)

#  CATEGORÍA III  DETERIORO

In [325]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998_d existe
if 'unidad12998_d' in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186, "025": 187,
        "030": 188, "035": 189, "040": 190, "045": 191, "050": 192,
        "055": 193, "060": 194, "065": 195, "070": 196, "075": 197,
        "080": 198, "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206, "125": 207,
        "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Para el caso especial de subcuenta "125", que usa datos de "120" en el código original
            lookup_subcuenta = "120" if subcuenta == "125" else subcuenta
            
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998_d[unidad12998_d['Subcuenta'] == lookup_subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 5
                for i, valor in enumerate(df['value'].values):
                    wb['Hoja1'].cell(row=row+i, column=5, value=valor)
            else:
                # Si no hay datos, podemos opcionalmente poner un valor por defecto
                wb['Hoja1'].cell(row=row, column=5, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb['Hoja1'].cell(row=row, column=5, value=0)

# CATEGORÍA III  PONDERADO 

In [326]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad12998_p existe
if "unidad12998_p" in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 183, "010": 184, "015": 185, "020": 186, "025": 187,
        "030": 188, "035": 189, "040": 190, "045": 191, "050": 192,
        "055": 193, "060": 194, "065": 195, "070": 196, "075": 197,
        "080": 198, "085": 199, "090": 200, "095": 201, "100": 202,
        "105": 203, "110": 204, "115": 205, "120": 206, "125": 207,
        "998": 208
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad12998_p[unidad12998_p['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 7 como en el código original
                for i, valor in enumerate(df['value'].values):
                    wb["Hoja1"].cell(row=row+i, column=7, value=valor)
            else:
                # Si no hay datos, opcionalmente podemos poner un valor por defecto
                wb["Hoja1"].cell(row=row, column=7, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb["Hoja1"].cell(row=row, column=7, value=0)

#  RIESGO OPERACIONAL 

In [327]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad13998 existe
if "unidad13998" in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 211, 
        "010": 212, 
        "015": 213, 
        "020": 214, 
        "025": 215
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad13998[unidad13998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores como en el código original
                # Para la subcuenta "025" no se aplica redondeo según el código original
                if subcuenta != "025":
                    df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4 como en el código original
                for i, valor in enumerate(df['value'].values):
                    wb["Hoja1"].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, opcionalmente podemos poner un valor por defecto
                wb["Hoja1"].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb["Hoja1"].cell(row=row, column=4, value=0)

#  PATRIMONIO ADECUADO

In [328]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad14998 existe
if "unidad14998" in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 218, 
        "010": 219, 
        "015": 220, 
        "020": 221, 
        "025": 222,
        "030": 223,
        "035": 224,
        "040": 225,
        "998": 226
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad14998[unidad14998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4 como en el código original
                for i, valor in enumerate(df['value'].values):
                    wb["Hoja1"].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, opcionalmente podemos poner un valor por defecto
                wb["Hoja1"].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb["Hoja1"].cell(row=row, column=4, value=0)

#  PATRIMONIO TÉCNICO 

In [329]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad15998 existe
if "unidad15998" in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 229, 
        "010": 230, 
        "015": 231, 
        "020": 232, 
        "998": 233
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad15998[unidad15998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4 como en el código original
                for i, valor in enumerate(df['value'].values):
                    wb["Hoja1"].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, opcionalmente podemos poner un valor por defecto
                wb["Hoja1"].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb["Hoja1"].cell(row=row, column=4, value=0)

# LIQUIDACIÓN DE PATRIMONIO MÍNIMO EN FUNCIÓN AL ADECUADO 

In [330]:
# Obtener lista de variables definidas
consultas = list(locals().keys())

# Verificar si unidad16998 existe
if "unidad16998" in consultas:
    
    # Definir las subcuentas y sus correspondientes filas
    subcuentas = {
        "005": 236, 
        "010": 237, 
        "015": 238
    }
    
    # Iterar sobre las subcuentas
    for subcuenta, row in subcuentas.items():
        try:
            # Filtrar el DataFrame para la subcuenta
            filtered_data = unidad16998[unidad16998['Subcuenta'] == subcuenta]
            
            # Crear un DataFrame con la columna "value" como en el código original
            if len(filtered_data) > 0:
                df = pd.DataFrame({"value": filtered_data['Saldos'].values})
                
                # Aplicar redondeo a todos los valores
                df['value'] = df['value'].round(0)
                    
                # Escribir en Excel - usamos columna 4 como en el código original
                for i, valor in enumerate(df['value'].values):
                    wb["Hoja1"].cell(row=row+i, column=4, value=valor)
            else:
                # Si no hay datos, opcionalmente podemos poner un valor por defecto
                wb["Hoja1"].cell(row=row, column=4, value=0)
                
        except Exception:
            # En caso de error, ponemos un valor por defecto
            wb["Hoja1"].cell(row=row, column=4, value=0)

#    ACTIVOS INTANGIBLES

In [331]:
# Filtrar subconjuntos por patrones en la columna '6d'
AÑO_ACTUALSubset_2 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("191135", regex=True)]
AÑO_ACTUALSubset_3 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("191165", regex=True)]
AÑO_ACTUALSubset_4 = AÑO_ACTUAL[AÑO_ACTUAL['6d'].str.contains("191170", regex=True)]

# Combinar los subconjuntos
AÑO_ACTUALSubset = pd.concat([AÑO_ACTUALSubset_2, AÑO_ACTUALSubset_3, AÑO_ACTUALSubset_4])

# Crear el DataFrame Activos_Intangibles
Activos_Intangibles = AÑO_ACTUALSubset.copy()

# Seleccionar columnas y renombrarlas
Activos_Intangibles = Activos_Intangibles[['Cuenta alternativa', 'Total Cuenta', 'Saldo mes pasado', 'Ejercicio']]
Activos_Intangibles.rename(columns={'Total Cuenta': 'Saldos', 'Saldo mes pasado': 'saldos_ant'}, inplace=True)

# Extraer y convertir la cuenta
Activos_Intangibles['Cuenta'] = Activos_Intangibles['Cuenta alternativa'].str.slice(0, 6)
Activos_Intangibles['Cuenta'] = pd.to_numeric(Activos_Intangibles['Cuenta'])

# Convertir CUENTA en CUIF a numérico antes del merge
CUIF['CUENTA'] = pd.to_numeric(CUIF['CUENTA'], errors='coerce')

# Hacer join con CUIF
Activos_Intangibles = pd.merge(
    Activos_Intangibles, 
    CUIF, 
    left_on='Cuenta', 
    right_on='CUENTA', 
    how='left'
)

# Renombrar columna NOMBRE
Activos_Intangibles.rename(columns={'NOMBRE': 'Nombre Cuenta'}, inplace=True)

# Agregar unidad de captura y subcuenta
Activos_Intangibles['Unidad de captura'] = 6
Activos_Intangibles['Subcuenta'] = "055"

# Calcular diferencias
Activos_Intangibles['Diferencia mes a mes'] = Activos_Intangibles['Saldos'] - Activos_Intangibles['saldos_ant']
Activos_Intangibles['Diferencia %'] = (Activos_Intangibles['Saldos'] - Activos_Intangibles['saldos_ant']) / Activos_Intangibles['Saldos']

# Renombrar saldos_ant
Activos_Intangibles.rename(columns={'saldos_ant': 'Saldos del mes pasado'}, inplace=True)

# Seleccionar y ordenar columnas finales
Activos_Intangibles = Activos_Intangibles[[
    'Unidad de captura', 'Subcuenta', 'Cuenta', 'Cuenta alternativa', 'Nombre Cuenta',
    'Saldos', 'Saldos del mes pasado', 'Diferencia mes a mes', 'Diferencia %', 'Ejercicio'
]]

# Condición sobre filas y notas
if len(Activos_Intangibles) > 0:
    Activos_Intangibles['notas'] = ""  # Agregar columna de notas
    bd_analisis_cuentas = pd.concat([bd_analisis_cuentas, Activos_Intangibles])

# Calcular el valor total absoluto
total_valor = abs(Activos_Intangibles['Saldos'].sum())

# Crear un nuevo DataFrame con el total
Activos_Intangibles = pd.DataFrame({'total': [total_valor]})

# Manejar caso de DataFrame vacío
if len(Activos_Intangibles) == 0:
    Activos_Intangibles = pd.DataFrame({'Total': [0]})

# Escribir en Excel
for i, row in Activos_Intangibles.iterrows():
    wb["Hoja1"].cell(row=56+i, column=9, value=row[0])

C:\Users\ce02815\AppData\Local\Temp\ipykernel_12168\3014585418.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  wb["Hoja1"].cell(row=56+i, column=9, value=row[0])


# IMPRIMIR PROFORMA 

In [333]:
wb.save("MS_COVI_F407.xlsx")

# PROFORMA EN TXT 

In [334]:
# Formato sin notación científica pero sin imprimir
numero = 1e11
numero_formateado = f"{numero:.0f}"  # Convierte a formato no científico (100000000000)

# Desactivar notación científica en numpy (opcional)
np.set_printoptions(suppress=True)

# Lista vacía equivalente a bd=c() en R
bd = []

# PRIMERA LINEA 

In [336]:
# Crear una lista vacía
bd = []

# Añadir segunda línea a la lista
segunda_linea = "000022000001100000"
bd.append(segunda_linea)  # Añadir al final

# Obtener el último día del mes anterior
from datetime import date, timedelta

hoy = date.today()
primer_dia_mes_actual = date(hoy.year, hoy.month, 1)
ultimo_dia_mes_anterior = primer_dia_mes_actual - timedelta(days=1)

# Formatear la fecha correctamente
dia = ultimo_dia_mes_anterior.strftime("%d")
mes = ultimo_dia_mes_anterior.strftime("%m")
year = ultimo_dia_mes_anterior.strftime("%Y")
fecha = dia + mes + year  # Último día del mes anterior en formato DDMMYYYY

# Crear y agregar el registro inicial antes de modificar bd[0]
registro_inicial = "00001114000001" + fecha + "00262" + "SVIDCOLSECS0140"
bd.insert(0, registro_inicial)  # Insertar al inicio

#  CARGUE DEL FORMATO 407 POR UNIDAD DE CAPTURA

#  UNIDAD DE CAPTURA 1

In [337]:
# Leer el archivo Excel directamente con pandas
formato407_unidad01_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=22,   # Saltar las primeras 22 filas
    nrows=23,      # Leer 23 filas (B23:D45)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad01_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad01_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 1  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
#############################################
cod_columna = 3

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)


# UNIDAD DE CAPTURA 2  

In [338]:
# Leer el archivo Excel directamente con pandas
formato407_unidad02_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=47,   # Saltar las primeras 47 filas
    nrows=16,      # Leer 16 filas (B48:D63)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad02_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad02_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 2  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
# Establecer el número inicial de secuencia en 10
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 sin comas ni espacios adicionales
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 3 

In [339]:
# Leer el archivo Excel directamente con pandas
formato407_unidad03_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=65,   # Saltar hasta llegar a la fila 66
    nrows=11,      # Leer 11 filas (equivalente a B66:D76)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad03_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad03_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 3  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 4

In [340]:
# Leer el archivo Excel directamente con pandas
formato407_unidad04_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=78,   # Saltar hasta llegar a la fila 79
    nrows=7,       # Leer 7 filas (equivalente a B79:D85)
    usecols="B:D", # Usar columnas B a D
    header=None    # No usar primera fila como encabezado
)
formato407_unidad04_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad04_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 4  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 10 

In [341]:
# Leer el archivo Excel directamente con pandas
formato407_unidad10_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=155,   # Saltar hasta llegar a la fila 156
    nrows=14,       # Leer 14 filas (equivalente a B156:D169)
    usecols="B:D",  # Usar columnas B a D
    header=None     # No usar primera fila como encabezado
)
formato407_unidad10_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad10_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 10  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 11

In [342]:
# Leer el archivo Excel directamente con pandas
formato407_unidad11_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=170,   # Saltar hasta llegar a la fila 171
    nrows=11,       # Leer 11 filas (equivalente a B171:G181)
    usecols="B:G",  # Usar columnas B a G
    header=None     # No usar primera fila como encabezado
)
formato407_unidad11_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Reducir a las columnas necesarias (SUBCUENTA, CONCEPTO, Saldo)
formato407_unidad11_COVI = formato407_unidad11_COVI.iloc[:, 0:3]
formato407_unidad11_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad11_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 11  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 12

In [343]:
# Leer el archivo Excel directamente con pandas
formato407_unidad12_COVI = pd.read_excel(
   "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=182,   # Saltar hasta llegar a la fila 183
    nrows=26,       # Leer 26 filas (equivalente a B183:G208)
    usecols="B:G",  # Usar columnas B a G
    header=None     # No usar primera fila como encabezado
)
formato407_unidad12_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Reducir a las columnas necesarias (SUBCUENTA, CONCEPTO, Saldo)
formato407_unidad12_COVI = formato407_unidad12_COVI.iloc[:, 0:3]
formato407_unidad12_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad12_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 13

In [344]:
# Leer el archivo Excel directamente con pandas
formato407_unidad13_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=210,   # Saltar hasta llegar a la fila 211
    nrows=5,        # Leer 5 filas (equivalente a B211:D215)
    usecols="B:D",  # Usar columnas B a D
    header=None     # No usar primera fila como encabezado
)
formato407_unidad13_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad13_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 13  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 14 

In [345]:
# Leer el archivo Excel directamente con pandas
formato407_unidad14_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=217,   # Saltar hasta llegar a la fila 218
    nrows=9,        # Leer 9 filas (equivalente a B218:D226)
    usecols="B:D",  # Usar columnas B a D
    header=None     # No usar primera fila como encabezado
)
formato407_unidad14_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad14_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 14  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Nota: La línea de multiplicación por 100 está comentada en el código original
    # por lo que también la comentamos aquí
    # for i in range(len(d3)):
    #     if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
    #         d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 15

In [346]:
# Leer el archivo Excel directamente con pandas
formato407_unidad15_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=228,   # Saltar hasta llegar a la fila 229
    nrows=5,        # Leer 5 filas (equivalente a B229:D233)
    usecols="B:D",  # Usar columnas B a D
    header=None     # No usar primera fila como encabezado
)
formato407_unidad15_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad15_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 15  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 16 

In [347]:
# Leer el archivo Excel directamente con pandas
formato407_unidad16_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=235,   # Saltar hasta llegar a la fila 236
    nrows=3,        # Leer 3 filas (equivalente a B236:D238)
    usecols="B:D",  # Usar columnas B a D
    header=None     # No usar primera fila como encabezado
)
formato407_unidad16_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo"]

# Criterios
datos = formato407_unidad16_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 16  # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 12 

In [348]:
# Leer el archivo Excel directamente con pandas
formato407_unidad12_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=182,   # Saltar hasta llegar a la fila 183
    nrows=26,       # Leer 26 filas (equivalente a B183:G208)
    usecols="B:G",  # Usar columnas B a G
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas - ahora necesitamos mantener hasta la columna D (4ta columna)
formato407_unidad12_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad12_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12    # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 4    # CAMBIO: ahora es 4 en lugar de 3
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "02" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 11 

In [349]:
# Leer el archivo Excel directamente con pandas
formato407_unidad11_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=170,   # Saltar hasta llegar a la fila 171
    nrows=11,       # Leer 11 filas (equivalente a B171:G181)
    usecols="B:G",  # Usar columnas B a G
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas - necesitamos mantener hasta la columna G (6ta columna)
formato407_unidad11_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad11_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 11    # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 6    # Ahora es 6 para acceder a la columna G del Excel
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "04" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 12 

In [350]:
# Leer el archivo Excel directamente con pandas
formato407_unidad12_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=182,   # Saltar hasta llegar a la fila 183
    nrows=26,       # Leer 26 filas (equivalente a B183:G208)
    usecols="B:G",  # Usar columnas B a G
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas - necesitamos mantener hasta la columna G (6ta columna)
formato407_unidad12_COVI.columns = ["SUBCUENTA", "CONCEPTO", "Saldo", "Col4", "Col5", "Col6"]

# Criterios
datos = formato407_unidad12_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 12    # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 6    # Ahora es 6 para acceder a la columna G del Excel
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "04" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 8 

In [351]:
# Leer el archivo Excel directamente con pandas
formato407_unidad08_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=139,   # Saltar hasta llegar a la fila 140
    nrows=5,        # Leer 5 filas (equivalente a B140:M144)
    usecols="B:M",  # Usar columnas B a M
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad08_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales"
]

# Criterios
datos = formato407_unidad08_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 8     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 8     # Ahora es 8 para acceder a la columna "Vida Individual"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "06" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 9

In [352]:
# Leer el archivo Excel directamente con pandas
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=145,   # Saltar hasta llegar a la fila 146
    nrows=4,        # Leer 4 filas (equivalente a B146:O149)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 8     # Para acceder a la columna "Vida Individual"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "06" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

#  UNIDAD DE CAPTURA 8 

In [353]:
# Leer el archivo Excel directamente con pandas
formato407_unidad08_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=139,   # Saltar hasta llegar a la fila 140
    nrows=5,        # Leer 5 filas (equivalente a B140:M144)
    usecols="B:M",  # Usar columnas B a M
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad08_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales"
]

# Criterios
datos = formato407_unidad08_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 8     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 9     # Ahora es 9 para acceder a la columna "Pensiones con Conmutación Pensional"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "07" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 9 

In [354]:
# Leer el archivo Excel directamente con pandas
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=145,   # Saltar hasta llegar a la fila 146
    nrows=4,        # Leer 4 filas (equivalente a B146:O149)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 9     # Para acceder a la columna "Pensiones con Conmutación Pensional"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "07" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 8

In [355]:
# Leer el archivo Excel directamente con pandas
formato407_unidad08_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=139,   # Saltar hasta llegar a la fila 140
    nrows=5,        # Leer 5 filas (equivalente a B140:M144)
    usecols="B:M",  # Usar columnas B a M
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad08_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales"
]

# Criterios
datos = formato407_unidad08_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 8     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 10    # Ahora es 10 para acceder a la columna "Pensiones Ley 100"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Ahora será "08" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 9

In [356]:
# Leer el archivo Excel directamente con pandas
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=145,   # Saltar hasta llegar a la fila 146
    nrows=4,        # Leer 4 filas (equivalente a B146:O149)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 10    # Para acceder a la columna "Pensiones Ley 100"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "08" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 5 

In [357]:
# Leer el archivo Excel directamente con pandas
formato407_unidad05_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=92,    # Saltar hasta llegar a la fila 93
    nrows=8,        # Leer 8 filas (equivalente a B93:O100)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad05_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad05_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 5     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 12    # Para acceder a la columna "Previsionales de Invalidez y Sobrevivencia"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "10" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 6

In [358]:
# Leer el archivo Excel directamente con pandas
formato407_unidad06_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=101,   # Saltar hasta llegar a la fila 102
    nrows=24,       # Leer 24 filas (equivalente a B102:O125)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad06_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad06_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 6     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 12    # Para acceder a la columna "Previsionales de Invalidez y Sobrevivencia"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "10" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 7

In [359]:
# Leer el archivo Excel directamente con pandas
formato407_unidad07_COVI = pd.read_excel(
   "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=126,   # Saltar hasta llegar a la fila 127
    nrows=12,       # Leer 12 filas (equivalente a B127:O138)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad07_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad07_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 7     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 12    # Para acceder a la columna "Previsionales de Invalidez y Sobrevivencia"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "10" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 8 

In [360]:
# Leer el archivo Excel directamente con pandas
formato407_unidad08_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=139,   # Saltar hasta llegar a la fila 140
    nrows=5,        # Leer 5 filas (equivalente a B140:M144)
    usecols="B:M",  # Usar columnas B a M
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad08_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales"
]

# Nota: El rango original solo incluye hasta la columna M, pero los nombres incluyen más columnas
# Vamos a tratar de añadir las columnas faltantes con valores NaN
if len(formato407_unidad08_COVI.columns) < 14:
    # Añadir columnas faltantes
    formato407_unidad08_COVI["Previsionales de Invalidez y Sobrevivencia"] = np.nan
    formato407_unidad08_COVI["Demas_ramos"] = np.nan

# Criterios
datos = formato407_unidad08_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 8     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 12    # Para acceder a la columna "Previsionales de Invalidez y Sobrevivencia"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Verificar si la columna existe antes de procesar
if cod_columna <= len(datos.columns):
    # Encontrar índices donde la columna cod_columna no es igual a 0
    limpieza01 = datos.iloc[:, cod_columna-1] != 0
    d1 = datos[limpieza01].reset_index(drop=True)

    if len(d1) > 0:
        # Crear una secuencia de números con formato de 5 dígitos
        num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
        
        # Convertir la columna seleccionada a una lista
        d3 = d1.iloc[:, cod_columna-1].tolist()
        
        # Multiplicar por 100 los valores entre 0 y 1
        for i in range(len(d3)):
            if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
                d3[i] = round(d3[i] * 100, 2)
        
        # Formatear valores con 20 dígitos rellenados con ceros
        valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
        
        # Determinar el signo
        signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
        
        # Crear matriz01
        matriz01 = pd.DataFrame({
            'num_secuencia': num_secuencia,
            'tipo_registro': [str(tipo_registro)] * len(d1),
            'cod_formato': [str(cod_formato)] * len(d1),
            'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "10" 
            'unidad': [str(cod_unidad).zfill(2)] * len(d1),
            'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
            'signo': signo,
            'valor': valor
        })
        
        # Crear resul02
        resul02 = []
        for i in range(len(matriz01)):
            resul02.append(matriz01['num_secuencia'][i] + 
                          matriz01['tipo_registro'][i] + 
                          matriz01['cod_formato'][i] + 
                          matriz01['col'][i] + 
                          matriz01['unidad'][i] + 
                          matriz01['subcuenta'][i] + 
                          matriz01['signo'][i] + 
                          matriz01['valor'][i])
        
        # Añadir a bd
        bd.extend(resul02)

# UNIDAD DE CAPTURA 9 

In [361]:
# Leer el archivo Excel directamente con pandas
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=145,   # Saltar hasta llegar a la fila 146
    nrows=4,        # Leer 4 filas (equivalente a B146:O149)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 12    # Para acceder a la columna "Previsionales de Invalidez y Sobrevivencia"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "10" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 5 

In [362]:
# Leer el archivo Excel directamente con pandas
formato407_unidad05_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=92,    # Saltar hasta llegar a la fila 93
    nrows=8,        # Leer 8 filas (equivalente a B93:O100)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad05_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad05_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 5     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 13    # Para acceder a la columna "Demas_ramos"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "11" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 6 

In [363]:
# Leer el archivo Excel directamente con pandas
formato407_unidad06_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=101,   # Saltar hasta llegar a la fila 102
    nrows=24,       # Leer 24 filas (equivalente a B102:O125)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad06_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad06_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 6     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 13    # Para acceder a la columna "Demas_ramos"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "11" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 7  

In [364]:
# Leer el archivo Excel directamente con pandas
formato407_unidad07_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=126,   # Saltar hasta llegar a la fila 127
    nrows=12,       # Leer 12 filas (equivalente a B127:O138)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad07_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad07_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 7     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 13    # Para acceder a la columna "Demas_ramos"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "11" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 9

In [365]:
# Leer el archivo Excel directamente con pandas
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=145,   # Saltar hasta llegar a la fila 146
    nrows=4,        # Leer 4 filas (equivalente a B146:O149)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 13    # Para acceder a la columna "Demas_ramos"
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "11" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 5

In [366]:
# Leer el archivo Excel directamente con pandas
formato407_unidad05_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=92,    # Saltar hasta llegar a la fila 93
    nrows=8,        # Leer 8 filas (equivalente a B93:O100)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad05_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad05_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 5     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 14    # NOTA: Este valor es diferente al ejemplo anterior (ahora es 14)
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
# Nota: Como cod_columna = 14 pero solo hay 14 columnas (0-13 en índice de Python),
# necesitamos ajustar esto para evitar un error de índice fuera de rango
if cod_columna <= len(datos.columns):
    limpieza01 = datos.iloc[:, cod_columna-1] != 0
    d1 = datos[limpieza01].reset_index(drop=True)
else:
    # Si cod_columna está fuera de rango, usamos una columna válida o creamos un DataFrame vacío
    print(f"Advertencia: cod_columna={cod_columna} está fuera de rango. El DataFrame tiene {len(datos.columns)} columnas.")
    d1 = pd.DataFrame(columns=datos.columns)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "12" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 6 

In [367]:
# Leer el archivo Excel directamente con pandas
formato407_unidad06_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=101,   # Saltar hasta llegar a la fila 102
    nrows=24,       # Leer 24 filas (equivalente a B102:O125)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad06_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad06_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 6     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 14    # NOTA: Este valor es 14
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
# Nota: Como cod_columna = 14 pero solo hay 14 columnas (0-13 en índice de Python),
# necesitamos ajustar esto para evitar un error de índice fuera de rango
if cod_columna <= len(datos.columns):
    limpieza01 = datos.iloc[:, cod_columna-1] != 0
    d1 = datos[limpieza01].reset_index(drop=True)
else:
    # Si cod_columna está fuera de rango, usamos una columna válida o creamos un DataFrame vacío
    print(f"Advertencia: cod_columna={cod_columna} está fuera de rango. El DataFrame tiene {len(datos.columns)} columnas.")
    d1 = pd.DataFrame(columns=datos.columns)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "12" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 7

In [368]:
# Leer el archivo Excel directamente con pandas
formato407_unidad07_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,   # Primera hoja
    skiprows=126,   # Saltar hasta llegar a la fila 127
    nrows=12,       # Leer 12 filas (equivalente a B127:O138)
    usecols="B:O",  # Usar columnas B a O
    header=None     # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad07_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad07_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 7     # Colocar la unidad de captura usada

# Asumo que bd es una lista que ya existe
# Si no existe, créala así:
if 'bd' not in locals():
    bd = []

cod_columna = 14    # NOTA: Este valor es 14
secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
# Nota: Como cod_columna = 14 pero solo hay 14 columnas (0-13 en índice de Python),
# necesitamos ajustar esto para evitar un error de índice fuera de rango
if cod_columna <= len(datos.columns):
    limpieza01 = datos.iloc[:, cod_columna-1] != 0
    d1 = datos[limpieza01].reset_index(drop=True)
else:
    # Si cod_columna está fuera de rango, usamos una columna válida o creamos un DataFrame vacío
    print(f"Advertencia: cod_columna={cod_columna} está fuera de rango. El DataFrame tiene {len(datos.columns)} columnas.")
    d1 = pd.DataFrame(columns=datos.columns)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # Será "12" 
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02
    resul02 = []
    for i in range(len(matriz01)):
        resul02.append(matriz01['num_secuencia'][i] + 
                      matriz01['tipo_registro'][i] + 
                      matriz01['cod_formato'][i] + 
                      matriz01['col'][i] + 
                      matriz01['unidad'][i] + 
                      matriz01['subcuenta'][i] + 
                      matriz01['signo'][i] + 
                      matriz01['valor'][i])
    
    # Añadir a bd
    bd.extend(resul02)

# UNIDAD DE CAPTURA 9 

In [369]:
# Leer el archivo Excel
formato407_unidad09_COVI = pd.read_excel(
    "D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Proforma/MS_COVI_F407.xlsx",
    sheet_name=0,  # Primera hoja
    skiprows=145,  # Saltar las primeras 145 filas (para llegar a B146)
    nrows=4,       # Leer 4 filas (B146:O149)
    usecols="B:O", # Usar columnas B a O
    header=None    # No usar primera fila como encabezado
)

# Asignar nombres a las columnas
formato407_unidad09_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "Saldo", "Deterioro", "Exposición  Crediticia", "Valor Ponderado",
    "Educativo", "Vida Individual", "Pensiones con Conmutación Pensional", "Pensiones Ley 100",
    "Pensiones Voluntarias", "Riesgos Profesionales", "Previsionales de Invalidez y Sobrevivencia",
    "Demas_ramos"
]

# Criterios
datos = formato407_unidad09_COVI  # Colocar la bd usada
cod_formato = 407  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 9  # Colocar la unidad de captura usada
cod_columna = 14  # En este caso es 14

# Asumimos que bd es una variable global que ya existe
# Si no existe, la inicializamos como una lista vacía
try:
    bd
except NameError:
    bd = []

secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR

# Encontrar índices donde la columna cod_columna no es igual a 0
limpieza01 = datos.iloc[:, cod_columna-1] != 0  # Ajuste por indexación 0-based en Python
d1 = datos[limpieza01].reset_index(drop=True)

if len(d1) > 0:
    # Crear una secuencia de números con formato de 5 dígitos
    num_secuencia = [(str(i).zfill(5)) for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
    
    # Convertir la columna seleccionada a una lista
    d3 = d1.iloc[:, cod_columna-1].tolist()
    
    # Multiplicar por 100 los valores entre 0 y 1
    for i in range(len(d3)):
        if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
            d3[i] = round(d3[i] * 100, 2)
    
    # Formatear valores con 20 dígitos rellenados con ceros
    valor = [str(abs(v)).zfill(20) if isinstance(v, (int, float)) else "00000000000000000000" for v in d3]
    
    # Determinar el signo
    signo = ['+' if isinstance(v, (int, float)) and v > 0 else '-' for v in d1.iloc[:, cod_columna-1]]
    
    # Crear matriz01
    matriz01 = pd.DataFrame({
        'num_secuencia': num_secuencia,
        'tipo_registro': [str(tipo_registro)] * len(d1),
        'cod_formato': [str(cod_formato)] * len(d1),
        'col': [str(cod_columna-2).zfill(2)] * len(d1),  # cod_columna-2 (14-2=12)
        'unidad': [str(cod_unidad).zfill(2)] * len(d1),
        'subcuenta': [str(int(sc)).zfill(3) if isinstance(sc, (int, float)) else "000" for sc in d1['SUBCUENTA']],
        'signo': signo,
        'valor': valor
    })
    
    # Crear resul02 (concatenar todas las columnas)
    resul02 = []
    for i in range(len(matriz01)):
        row_str = (matriz01['num_secuencia'][i] + 
                  matriz01['tipo_registro'][i] + 
                  matriz01['cod_formato'][i] + 
                  matriz01['col'][i] + 
                  matriz01['unidad'][i] + 
                  matriz01['subcuenta'][i] + 
                  matriz01['signo'][i] + 
                  matriz01['valor'][i])
        resul02.append(row_str)
    
    # Añadir a bd
    bd.extend(resul02)

# CARGUE DEL FORMATO 481 POR UNIDAD DE CAPTURA 

# UNIDAD DE CAPTURA 1 

In [370]:
# Leer el archivo Excel
formato481_unidad01_COVI = pd.read_excel("D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Fuentes de Información COVI.xlsx", 
                                         sheet_name="Formato481", 
                                         header=None, 
                                         skiprows=12, 
                                         nrows=41,  # Equivalente a range = "B13:AD53"
                                         usecols=range(1, 30))  # Columnas B a AD

# Asignar nombres a las columnas
formato481_unidad01_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "ELIMINADA", "RESERVA MATEMÁTICA", "DEPÓSITO DE RESERVA A REASEGURADORES",
    "RESERVA DESVIACIÓN DE SINIESTRALIDAD", "RESERVA PARA SINIESTROS AVISADOS", "RESERVA PARA SINIESTROS NO AVISADOS",
    "RESERVA PARA SINIESTROS PENDIENTES GARANTIZADOS POR LA NACIÓN", "RESERVA ENFERMEDAD LABORAL",
    "TOTAL RESERVAS", "RESERVAS TECNICAS EN MONEDA NACIONAL", "RESERVAS TECNICAS EN MONEDA EXTRANJERA",
    "TOTAL ACTIVOS QUE RESPALDAN  RESERVAS TÉCNICAS", "INVERSIONES MONEDA NACIONAL QUE RESPALDAN LAS RESERVAS TECNICAS",
    "INVERSIONES MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TECNICAS", "TOTAL OTROS ACTIVOS QUE RESPALDAN LA RESERVAS TECNICAS",
    "DEFECTO", "RESERVA DE INSUFICIENCIA DE ACTIVOS", "RESERVA DE RIESGOS CATASTRÓFICOS", "ELIMINADA", "RESERVA DE PRIMA NO DEVENGADA",
    "RESERVA POR INSUFICIENCIA DE PRIMA", "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA NACIONAL",
    "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA EXTRANJERA", "TOTAL CONTINGENCIAS A CARGO DE REASEGURADORES",
    "TOTAL INVERSIONES QUE RESPALDAN  LAS RESERVAS TÉCNICAS", "OTROS ACTIVOS EN MONEDA NACIONAL  QUE RESPALDAN LAS RESERVAS TÉCNICAS",
    "OTROS ACTIVOS EN MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TÉCNICAS"
]

# Criterios
datos = formato481_unidad01_COVI  # Colocar la bd usada
cod_formato = 481  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 1  # Colocar la unidad de captura usada 

# Inicializar bd si no existe
try:
    bd
except NameError:
    bd = []

# Recorrer las columnas (equivalente a for (cod_columna in 1:(ncol(datos)-2)))
for cod_columna in range(1, len(datos.columns) - 2 + 1):  # +1 porque range es exclusivo al final
    secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
    
    # En R: limpieza01<-which(!datos[,cod_columna+2]==0)
    # En Python: el índice de columna es cod_columna+1 (por indexación base 0)
    limpieza01 = datos.iloc[:, cod_columna+1] != 0  # Crear máscara booleana
    limpieza01 = limpieza01 & ~datos.iloc[:, cod_columna+1].isna()  # Filtrar NaN
    d1 = datos[limpieza01].reset_index(drop=True)  # Resetear índices
    
    if len(d1) > 0:
        # Generar números de secuencia
        # En R: str_pad(seq(secuencia_numero_registro, secuencia_numero_registro+nrow(d1)), 5, pad = "0")
        num_secuencia = [f"{i:05d}" for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
        
        # En R: d3<-as.vector(as.matrix(d1[,cod_columna+2]))
        d3 = d1.iloc[:, cod_columna+1].tolist()  # Convertir a lista
        
        # En R: d3[which(d3>0 & d3<=1)]<- round(d3[which(d3>0 & d3<=1)]*100,2)
        # Multiplicar valores entre 0 y 1 por 100
        for i in range(len(d3)):
            if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
                d3[i] = round(d3[i] * 100, 2)
        
        # En R: valor=str_pad(d3,20,pad = "0")
        # Formatear valores con 20 dígitos
        valor = [f"{abs(v):020.0f}" if isinstance(v, (int, float)) and not pd.isna(v) else "00000000000000000000" for v in d3]
        
        # En R: signo=ifelse(d1[,cod_columna+2]>0,"+","-")
        # Determinar signo de valores
        signo = []
        for v in d1.iloc[:, cod_columna+1]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                signo.append("+" if v > 0 else "-")
            else:
                signo.append("-")  # Default para valores no numéricos
        
        # Crear matriz01 (DataFrame en Python)
        # Formatear subcuenta con 3 dígitos
        subcuenta = []
        for v in d1["SUBCUENTA"]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                subcuenta.append(f"{int(v):03d}")
            else:
                subcuenta.append("000")
        
        # Construir el DataFrame equivalente a matriz01
        matriz01 = pd.DataFrame({
            'num_secuencia': num_secuencia,
            'tipo_registro': [str(tipo_registro)] * len(d1),
            'cod_formato': [str(cod_formato)] * len(d1),
            'col': [f"{cod_columna:02d}"] * len(d1),
            'unidad': [f"{cod_unidad:02d}"] * len(d1),
            'subcuenta': subcuenta,
            'signo': signo,
            'valor': valor
        })
        
        # En R: resul02=paste0(matriz01[,1],matriz01[,2],...,matriz01[,8])
        # Concatenar las columnas para formar cada registro
        resul02 = []
        for i in range(len(matriz01)):
            registro = (
                matriz01['num_secuencia'][i] +
                matriz01['tipo_registro'][i] +
                matriz01['cod_formato'][i] +
                matriz01['col'][i] +
                matriz01['unidad'][i] +
                matriz01['subcuenta'][i] +
                matriz01['signo'][i] +
                matriz01['valor'][i]
            )
            resul02.append(registro)
        
        # En R: bd=c(bd,resul02)
        # Añadir a bd
        bd.extend(resul02)

#   UNIDAD DE CAPTURA 2 

In [371]:
# Leer el archivo Excel
formato481_unidad02_COVI = pd.read_excel("D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_407_SOLVENCIA/VIDA/Bases/Fuentes de Información COVI.xlsx", 
                                        sheet_name="Formato481", 
                                        header=None, 
                                        skiprows=54,  # Para iniciar en la fila 55
                                        nrows=2,      # Para leer 2 filas (55 y 56)
                                        usecols=range(1, 30))  # Columnas B a AD

# Asignar nombres a las columnas
formato481_unidad02_COVI.columns = [
    "SUBCUENTA", "CONCEPTO", "ELIMINADA", "RESERVA MATEMÁTICA", "DEPÓSITO DE RESERVA A REASEGURADORES",
    "RESERVA DESVIACIÓN DE SINIESTRALIDAD", "RESERVA PARA SINIESTROS AVISADOS", "RESERVA PARA SINIESTROS NO AVISADOS",
    "RESERVA PARA SINIESTROS PENDIENTES GARANTIZADOS POR LA NACIÓN", "RESERVA ENFERMEDAD LABORAL",
    "TOTAL RESERVAS", "RESERVAS TECNICAS EN MONEDA NACIONAL", "RESERVAS TECNICAS EN MONEDA EXTRANJERA",
    "TOTAL ACTIVOS QUE RESPALDAN  RESERVAS TÉCNICAS", "INVERSIONES MONEDA NACIONAL QUE RESPALDAN LAS RESERVAS TECNICAS",
    "INVERSIONES MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TECNICAS", "TOTAL OTROS ACTIVOS QUE RESPALDAN LA RESERVAS TECNICAS",
    "DEFECTO", "RESERVA DE INSUFICIENCIA DE ACTIVOS", "RESERVA DE RIESGOS CATASTRÓFICOS", "ELIMINADA", "RESERVA DE PRIMA NO DEVENGADA",
    "RESERVA POR INSUFICIENCIA DE PRIMA", "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA NACIONAL",
    "CONTINGENCIAS A CARGO DE REASEGURADORES EN MONEDA EXTRANJERA", "TOTAL CONTINGENCIAS A CARGO DE REASEGURADORES",
    "TOTAL INVERSIONES QUE RESPALDAN  LAS RESERVAS TÉCNICAS", "OTROS ACTIVOS EN MONEDA NACIONAL  QUE RESPALDAN LAS RESERVAS TÉCNICAS",
    "OTROS ACTIVOS EN MONEDA EXTRANJERA QUE RESPALDAN LAS RESERVAS TÉCNICAS"
]

# Criterios
datos = formato481_unidad02_COVI  # Colocar la bd usada
cod_formato = 481  # Colocar el tipo de formato usado
tipo_registro = 4  # Colocar el tipo usado
cod_unidad = 2  # Colocar la unidad de captura usada

# Inicializar bd si no existe
try:
    bd
except NameError:
    bd = []

# Recorrer las columnas (equivalente a for (cod_columna in 1:(ncol(datos)-2)))
for cod_columna in range(1, len(datos.columns) - 2 + 1):  # +1 porque range es exclusivo al final
    secuencia_numero_registro = len(bd) + 1  # NO SE DEBE MODIFICAR
    
    # En R: limpieza01<-which(!datos[,cod_columna+2]==0)
    # En Python: el índice de columna es cod_columna+1 (por indexación base 0)
    limpieza01 = datos.iloc[:, cod_columna+1] != 0  # Crear máscara booleana
    limpieza01 = limpieza01 & ~datos.iloc[:, cod_columna+1].isna()  # Filtrar NaN
    d1 = datos[limpieza01].reset_index(drop=True)  # Resetear índices
    
    if len(d1) > 0:
        # Generar números de secuencia
        # En R: str_pad(seq(secuencia_numero_registro, secuencia_numero_registro+nrow(d1)), 5, pad = "0")
        num_secuencia = [f"{i:05d}" for i in range(secuencia_numero_registro, secuencia_numero_registro + len(d1))]
        
        # En R: d3<-as.vector(as.matrix(d1[,cod_columna+2]))
        d3 = d1.iloc[:, cod_columna+1].tolist()  # Convertir a lista
        
        # En R: d3[which(d3>0 & d3<=1)]<- round(d3[which(d3>0 & d3<=1)]*100,2)
        # Multiplicar valores entre 0 y 1 por 100
        for i in range(len(d3)):
            if isinstance(d3[i], (int, float)) and 0 < d3[i] <= 1:
                d3[i] = round(d3[i] * 100, 2)
        
        # En R: valor=str_pad(d3,20,pad = "0")
        # Formatear valores con 20 dígitos
        valor = [f"{abs(v):020.0f}" if isinstance(v, (int, float)) and not pd.isna(v) else "00000000000000000000" for v in d3]
        
        # En R: signo=ifelse(d1[,cod_columna+2]>0,"+","-")
        # Determinar signo de valores
        signo = []
        for v in d1.iloc[:, cod_columna+1]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                signo.append("+" if v > 0 else "-")
            else:
                signo.append("-")  # Default para valores no numéricos
        
        # Crear matriz01 (DataFrame en Python)
        # Formatear subcuenta con 3 dígitos
        subcuenta = []
        for v in d1["SUBCUENTA"]:
            if isinstance(v, (int, float)) and not pd.isna(v):
                subcuenta.append(f"{int(v):03d}")
            else:
                subcuenta.append("000")
        
        # Construir el DataFrame equivalente a matriz01
        matriz01 = pd.DataFrame({
            'num_secuencia': num_secuencia,
            'tipo_registro': [str(tipo_registro)] * len(d1),
            'cod_formato': [str(cod_formato)] * len(d1),
            'col': [f"{cod_columna:02d}"] * len(d1),
            'unidad': [f"{cod_unidad:02d}"] * len(d1),
            'subcuenta': subcuenta,
            'signo': signo,
            'valor': valor
        })
        
        # En R: resul02=paste0(matriz01[,1],matriz01[,2],...,matriz01[,8])
        # Concatenar las columnas para formar cada registro
        resul02 = []
        for i in range(len(matriz01)):
            registro = (
                matriz01['num_secuencia'][i] +
                matriz01['tipo_registro'][i] +
                matriz01['cod_formato'][i] +
                matriz01['col'][i] +
                matriz01['unidad'][i] +
                matriz01['subcuenta'][i] +
                matriz01['signo'][i] +
                matriz01['valor'][i]
            )
            resul02.append(registro)
        
        # En R: bd=c(bd,resul02)
        # Añadir a bd
        bd.extend(resul02)


#  LINEA FINAL

In [372]:
# Generar el registro final
n_final = str(len(bd) + 1).zfill(5)  # Número de registro con padding de ceros
n_final = n_final + str(tipo_registro + 1)  # Concatenar con tipo_registro+1
bd.append(n_final)  # Añadir a la lista bd

# EXPORTAR FORMATO TXT 

In [373]:
# Suponiendo que bd ya contiene todos los registros generados
# Convertir bd a DataFrame
bd1 = pd.DataFrame(bd, columns=['registro'])

# Filtrar registros que contienen "nan" o valores cero
registros_filtrados = []
encabezado = None
pie = None

# Guardar el encabezado y el pie
if len(bd) > 0:
    encabezado = bd[0]
if len(bd) > 1:
    pie = bd[-1]

# Filtrar y corregir los demás registros (excluyendo encabezado y pie)
for idx, registro in enumerate(bd):
    # Saltar el primer y último registro
    if idx == 0 or idx == len(bd) - 1:
        continue
    
    # Verificar si el registro contiene "nan"
    if "nan" not in str(registro).lower():
        # Extraer el signo y el valor
        match = re.search(r'([-+])(\d+\.\d+|\d+)$', str(registro))
        if match:
            signo = match.group(1)
            valor_str = match.group(2)
            
            try:
                # Convertir a float
                valor_float = float(valor_str)
                
                # Verificar si es cero
                if valor_float == 0:
                    continue  # Saltar este registro
                
                # Determinar si es un valor porcentual (menores a 100 con decimales)
                if valor_float < 100 and '.' in valor_str:
                    # Para porcentajes, mantener el punto decimal y formatear a 2 decimales
                    valor_formateado = f"{valor_float:.2f}"
                    # Asegurar que tenga 20 caracteres incluyendo el punto
                    while len(valor_formateado) < 20:
                        valor_formateado = "0" + valor_formateado
                    nuevo_valor = valor_formateado
                else:
                    # Para valores grandes, eliminar el punto decimal
                    valor_int = int(valor_float)
                    nuevo_valor = str(valor_int).zfill(20)
                
                # Reconstruir el registro con el valor corregido
                prefijo = registro[:registro.rfind(signo) + 1]  # Todo hasta e incluyendo el signo
                nuevo_registro = prefijo + nuevo_valor
                
                registros_filtrados.append(nuevo_registro)
            except ValueError:
                # Si hay algún error en la conversión, mantener el registro original
                registros_filtrados.append(registro)
        else:
            # Si no tiene el formato esperado, mantenerlo
            registros_filtrados.append(registro)

# Renumerar los registros filtrados
registros_renumerados = []
for i, registro in enumerate(registros_filtrados):
    # Extraer las partes del registro (asumiendo formato fijo)
    nuevo_num = str(i + 2).zfill(5)  # Empezar desde 00002 porque 00001 es el encabezado
    resto_registro = registro[5:]  # Obtener todo excepto los primeros 5 caracteres
    registros_renumerados.append(nuevo_num + resto_registro)

# Reconstruir la lista completa con encabezado y pie
resultado_final = []
if encabezado:
    resultado_final.append(encabezado)
resultado_final.extend(registros_renumerados)
if pie:
    # Actualizar el número en el pie
    nuevo_pie = str(len(resultado_final) + 1).zfill(5) + pie[5:]
    resultado_final.append(nuevo_pie)

# Crear un nuevo DataFrame con los registros renumerados
bd1_final = pd.DataFrame(resultado_final, columns=['registro'])

# Paso 1: Escribir el archivo con todas las líneas excepto la última
with open("CLV.txt", "w") as f:
    for i in range(len(bd1_final) - 1):
        f.write(bd1_final.iloc[i, 0] + "\n")

# Paso 2: Añadir la última línea manualmente, sin el salto de línea final
with open("CLV.txt", "a") as f:
    f.write(bd1_final.iloc[-1, 0])

# Actualizar el contador de registros en el encabezado
if len(bd1_final) > 0:
    # Extraer el encabezado actual
    encabezado_actual = bd1_final.iloc[0, 0]
    
# Actualizar el encabezado con contador y sufijo correctos
if len(bd1_final) > 0:
    encabezado_actual = bd1_final.iloc[0, 0]

    # Tomar las partes del encabezado
    prefijo_base = encabezado_actual[:14]           # "00001114000001"
    fecha_encabezado = encabezado_actual[14:22]     # "Fecha"
    contador = str(len(bd1_final)).zfill(5)         # número exacto de registros
    sufijo = encabezado_actual[-15:]                # últimos 15 caracteres siempre son el sufijo

    # Armar nuevo encabezado
    nuevo_encabezado = prefijo_base + fecha_encabezado + contador + sufijo

    # Verificación final
    if len(nuevo_encabezado) != 42:
        raise ValueError(f"Encabezado mal formado: '{nuevo_encabezado}' tiene {len(nuevo_encabezado)} caracteres (deben ser 42)")

    # Reemplazar el encabezado
    bd1_final.iloc[0, 0] = nuevo_encabezado

    # Reescribir el archivo
    with open("CLV.txt", "w") as f:
        for i in range(len(bd1_final) - 1):
            f.write(bd1_final.iloc[i, 0] + "\n")
    with open("CLV.txt", "a") as f:
        f.write(bd1_final.iloc[-1, 0])

    
    # Reemplazar el encabezado
    bd1_final.iloc[0, 0] = nuevo_encabezado
    
    # Reescribir el archivo con el encabezado actualizado
    with open("CLV.txt", "w") as f:
        for i in range(len(bd1_final) - 1):
            f.write(bd1_final.iloc[i, 0] + "\n")
    
    # Añadir la última línea sin salto de línea
    with open("CLV.txt", "a") as f:
        f.write(bd1_final.iloc[-1, 0])

# Leer el contenido del archivo y verificar
with open("CLV.txt", "r") as f:
    lines = f.readlines()

# EXPORTAR BD ANALISIS 

In [374]:
if "Validación reaseguro1" in wb.sheetnames:
    wb.remove(wb["Validación reaseguro1"])
wb.create_sheet("Validación reaseguro1")

if "Validación coaseguro1" in wb.sheetnames:
    wb.remove(wb["Validación coaseguro1"])
wb.create_sheet("Validación coaseguro1")

# Escribir datos en las hojas

# Validación reaseguro1
sheet = wb["Validación reaseguro1"]
for col_num, column_title in enumerate(INFORMACION_REASEGURO.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(INFORMACION_REASEGURO.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Validación coaseguro1
sheet = wb["Validación coaseguro1"]
for col_num, column_title in enumerate(INFORMACION_COASEGURO.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)
for row_num, row in enumerate(INFORMACION_COASEGURO.values, 2):
    for col_num, cell_value in enumerate(row, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_value)

# Guardar el libro de Excel
wb.save("MS_VIDA.xlsx")
